REALLY HACKY FIX FOR OBTAINING Hidden State:
- Swapped the keys for the testing function (bic version) such that hidden state tensor is located at the 'network_activity' key, and hidden layer output activity is located at 'network_hidden_state'
- This means. IF I WANT TO RUN IT AGAIN BUT USE THE OUTPUT ACTIVITY FOR THE ANALYSIS AND THEN USE HIDDEN STATE FPS WITH RELU ACTIVATION TO PLOT ALONGSIDE THE HIDDEN LAYER OUTPUT ACTIVITY FROM TESTING TRIALS:
  -  NEED TO RE RUN WITH THE FOLLOWING:
    - Keys swapped back (in testing func)
    - hidden_key (in FPF input) == 'network_hidden_state'
    - (Easier to copy note book and rerun like this)

# CURRENT MODE == ANALYSE HIDDEN ACTIVITY (not hidden state)

# Importing Packages ((un)comment neuro and sb imports)

In [ ]:
# # # # Uncomment VV for session restart
!pip install neurogym
!pip install git+https://github.com/DLR-RM/stable-baselines3
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
!pip install deepdiff



# MODEL_IO_NSC = False
# MODEL_IO_SE2D = False
# MODEL_IO_SE3D = False
# MODEL_IO_EI = False
# MODEL_IO_NBIC = False

# Use models instead of re-training every time
## NEED TO MAKE SURE HAVE folders and files e.g NBIC : LRNN_NBIC/LRNN_NBIC.pth
## training funcs spit out LRNN_NBIC_best.pth but training in dev nb doesnt so not gonna change
MODEL_IO_NSC = True
MODEL_IO_SE2D = True
MODEL_IO_SE3D = True
MODEL_IO_EI = True
MODEL_IO_NBIC = True





  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-6575_315
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-6575_315
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit bf51a6233a8f934a68430f8f78e44360410d23ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinu

In [ ]:
# Imports
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
from pathlib import Path
import json
from copy import deepcopy
from sklearn.decomposition import PCA
import itertools # Unused if not gridsearching
import logging

logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

device = 'cpu'
import neurogym as ngym

import warnings

# Suppress UserWarnings specifically from the 'gymnasium' module
warnings.filterwarnings("ignore", category=UserWarning, module="gymnasium")

# Training Regime (Training with Validation Thresholding and Early Stopping)

## Validation Function

Component function to run eval style trials on network for a single dataset (task trial setup)

In [ ]:
def validate_network(net, dataset, num_trials=200):
    # same as test but no print statements
    # Reset environment
  env = dataset.env
  env.reset()

  # Initialise variables for logging
  # perf = 0
  activity_dict = {}  # recording activity
  trial_infos = {}  # recording trial information

  num_trial = num_trials
  for i in range(num_trial):
      # Neurogym boiler plate
      # Sample a new trial
      trial_info = env.new_trial()
      # Observation and groud-truth of this trial
      ob, gt = env.ob, env.gt
      # Convert to numpy, add batch dimension to input
      inputs = torch.from_numpy(ob[:, np.newaxis, :]).type(torch.float)

      # Run the network for one trial
      # inputs (SeqLen, Batch, InputSize)
      # action_pred (SeqLen, Batch, OutputSize)
      action_pred, _ = net(inputs)

      # Compute performance
      # First convert back to numpy

      action_pred = action_pred.detach().numpy()[:, 0, :]
      # Read out final choice at last time step
      choice = np.argmax(action_pred[-1, :])
      # Compare to ground truth
      correct = choice == gt[-1]

      # Record activity, trial information, choice, correctness
      # rnn_activity = rnn_activity[:, 0, :].detach().numpy() # removed for new output signature of hidden state based networks (neither need it for validation anyway)
      # activity_dict[i] = rnn_activity
      trial_infos[i] = trial_info  # trial_info is a dictionary
      trial_infos[i].update({'correct': correct})
  avg_trial_performance = np.mean([val['correct'] for val in trial_infos.values()])
  return trial_infos, activity_dict, avg_trial_performance

## Early Stop Validation Helper

 Calling Validation Component for each validation data set and returning Performances to main training function

In [ ]:
def early_stop_validation(model_in_training, validation_set_dict, validation_trials=200):
  performance_dict = {}
  for dataset_name, dataset in validation_set_dict.items():
    trial_infos_valid, activity_dict_valid, avg_trial_performance_valid = validate_network(model_in_training, dataset, num_trials=validation_trials)
    performance_dict[dataset_name] = avg_trial_performance_valid
    # print(f'{dataset_name} - Accuracy : {avg_trial_performance_vt}')
  return performance_dict


## Main Training Function

Uses Early Stop Validation Helper and a "patience" mechanism to implement early stopping for networks that have exceeded the require performance threshold across validation task trial setups but not exhibiting performance improvement from further training.

In [ ]:
# import math
# def training_with_early_stop(model, training_set, validation_set_dict, max_steps = 10000, min_validation_perf = 0.8, patience = 5, num_steps_for_early_stop_check = 500, num_validation_trials = 200, model_name = 'model_name', tr_output_mode = False):

#       # validation_set_dict is like = {'Short': <neurogym dataset object> , ... }
#       # training set is like <neurogym dataset object>

#       # PERFORMANCE is each individual validation set accuracy
#       # ACCURACY is the overall average of these
#       # ES is for early stopping, i.e. perf/accuracy of validations that meet the es conditions (>threshold)
#       # all is for all i.e. perf/accuracy of validations that haven't met the es conditions

#       # we need to retain running loss / validation performances for plotting learning curves
#       # Learning tracking
#       # Training Loss Tracking
#       tr_epochs_loss = []
#       tr_loss = []

#       # Validation Performance Tracking
#       tr_epochs_valid_perf = []
#       tr_valid_perfs_list_of_dicts = []
#       tr_valid_perfs_avg = []
#       n_steps_first_th = 0
#       n_steps_final = 0

#       # Setup
#       optimiser = optim.Adam(model.parameters(), lr=0.001)
#       criterion = nn.CrossEntropyLoss()
#       running_loss = 0
#       running_acc = 0
#       start_time = time.time()

#       # Loop over training batches
#       print('Training network...')

#       current_best_valid_accuracy_es = 0.0
#       current_best_valid_accuracy_all = 0.0


#       # set best_valid_performance_es to a list of len n containing zeros where n is number of keys in validation_set_dict
#       best_valid_performance_all = [0 for _ in validation_set_dict.keys()]
#       best_valid_performance_es = [0 for _ in validation_set_dict.keys()]
#       surpassed_threshold = False
#       current_patience = patience
#       model.train()
#       model_save_name = f'{model_name}_best.pth'
#       for i in range(max_steps):

#           # Generate input and target, convert to pytorch tensor
#           # inputs, labels = dataset()
#           inputs, labels = training_set()
#           inputs = torch.from_numpy(inputs).type(torch.float)
#           labels = torch.from_numpy(labels.flatten()).type(torch.long)
#                   # boiler plate pytorch training:
#           optimiser.zero_grad()   # zero the gradient buffers
#           output, _ = model(inputs)
#           # Reshape to (SeqLen x Batch, OutputSize)
#           output_size = 3
#           output = output.view(-1, output_size)

#           loss = criterion(output, labels)

#           loss.backward()
#           optimiser.step()    # Does the update
#           running_loss += loss.item()
#           if i % 100 == 99:
#             running_loss /= 100
#             print('Step {}, Loss {:0.4f}, Time {:0.1f}s'.format(
#                 i+1, running_loss, time.time() - start_time))
#             tr_epochs_loss.append(i+1)
#             tr_loss.append(running_loss)

#             running_loss = 0
#           if (i + 1) % num_steps_for_early_stop_check == 0:
#             model.eval()
#             with torch.no_grad():
#               current_perf_dict = early_stop_validation(model, validation_set_dict, num_validation_trials)
#               print(f'Current Performance Dict: {current_perf_dict}')
#               list_avg_perfs = list(current_perf_dict.values())
#               average_perf = np.mean(list_avg_perfs)

#               tr_epochs_valid_perf.append(i+1)
#               tr_valid_perfs_list_of_dicts.append(current_perf_dict) # Means we retain labels for performance scores for the plot
#               tr_valid_perfs_avg.append(average_perf)

#               print(f'Current Average Performance: {average_perf}')
#               status_list = [i>min_validation_perf for i in list_avg_perfs]

#               performance_th_list = [i>j for i,j in zip(list_avg_perfs, best_valid_performance_all)]

#               best_valid_performance_all = list_avg_perfs if all(performance_th_list) else best_valid_performance_all

#               current_best_valid_accuracy_all = average_perf if average_perf > current_best_valid_accuracy_all else current_best_valid_accuracy_all
#               if all(status_list):
#                 surpassed_threshold = True
#                 n_steps_first_th = i+1
#                 num_steps_for_early_stop_check = 100
#                 if average_perf > current_best_valid_accuracy_es:
#                   current_best_valid_accuracy_es = average_perf
#                   best_valid_performance_es = list_avg_perfs
#                   # create a file of current best model
#                   torch.save(model.state_dict(), model_save_name)
#                   # Reset patience
#                   current_patience = patience
#                 elif ((average_perf == current_best_valid_accuracy_es) and (current_patience > 0)):
#                   current_best_valid_accuracy_es = average_perf
#                   best_valid_performance_es = list_avg_perfs
#                   # create a file of current best model
#                   torch.save(model.state_dict(), model_save_name)
#                   # Dont Reset patience, continue to decrease (model trains a bit more beyond first attempt at 100)
#                   current_patience -= 1

#                 else:
#                   if current_patience <= 0:
#                     n_steps_final = i+1
#                     print('Early Stopping. I have run out of patience Grrr' )
#                     break
#                   else:
#                     current_patience -= 1
#             model.train()
#       learning_curve_dict = {
#             'training_loss_epochs_list': tr_epochs_loss,
#             'training_loss_values_list' : tr_loss,
#             'validation_perf_epochs_list': tr_epochs_valid_perf,
#             'validation_perf_dicts_list' : tr_valid_perfs_list_of_dicts,
#             'validation_perf_avg_values_list' : tr_valid_perfs_avg,
#             'n_steps_first_th' : n_steps_first_th,
#             'n_steps_final' : n_steps_final
#               }
#       # Try to Load best model from last saved file. Handle for if it doesn't exist!
#       try:
#         model.load_state_dict(torch.load(model_save_name))
#         print('Best model loaded')
#         # Print best performance stats
#         for valid_set_name, perf in zip(validation_set_dict.keys(),best_valid_performance_es):
#           print(f'Best {valid_set_name} performance: {perf}')

#         if tr_output_mode:
#           return model, learning_curve_dict

#         return model
#       except:
#         print('No best model that satisfies validation threshold found')
#         print(f'Best all time performance: {current_best_valid_accuracy_all}')
#         for valid_set_name, perf in zip(validation_set_dict.keys(),best_valid_performance_all):
#           print(f'Best {valid_set_name} performance: {perf}')

#         if tr_output_mode:
#           return model, learning_curve_dict

#         return model

### Training Function - BICs

In [ ]:

def training_with_early_stop_and_regularisation(
    model,
    training_set,
    validation_set_dict,
    WD_approach=False,
    WD_regulariser=None,
    wiring_beta=0.001,
    activity_regularisation=True,
    activity_beta=1e-3,
    max_steps=10000,
    min_validation_perf=0.8,
    patience=5,
    num_steps_for_early_stop_check=500,
    num_validation_trials=200,
    model_name='model_name',
    tr_output_mode=False
):
    """
    Trains mode (optional wiring distance and activity regularisation if used in BICs),
    keeps the early stop mechanism. Also tracks non task loss to plot if needed.
    (Baso a Cleaned up version of the main training function with additional functionality for BICs - keep other in case ive fd up)
    """
    # --- Setup ---
    output_size = model.output_size
    optimiser = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    start_time = time.time()
    model_save_name = f'{model_name}_best.pth'

    # --- Loss Tracking ---
    tr_epochs_loss = []
    tr_total_loss_values = []
    tr_task_loss_values = []
    tr_wiring_loss_values = []
    tr_activity_loss_values = []
    running_loss = 0
    running_task_loss = 0
    running_wiring_loss = 0
    running_activity_loss = 0

    # --- Validation and Early Stopping Tracking ---
    tr_epochs_valid_perf = []
    tr_valid_perfs_list_of_dicts = []
    tr_valid_perfs_avg = []
    n_steps_first_th = 0
    n_steps_final = 0
    current_best_valid_accuracy_es = 0.0
    current_best_valid_accuracy_all = 0.0
    best_valid_performance_all = [0 for _ in validation_set_dict.keys()]
    best_valid_performance_es = [0 for _ in validation_set_dict.keys()]
    surpassed_threshold = False
    current_patience = patience

    # --- Training Loop ---
    print('Training network...')
    model.train()
    for i in range(max_steps):
        # --- Data ---
        inputs, labels = training_set()
        inputs = torch.from_numpy(inputs).type(torch.float)
        labels = torch.from_numpy(labels.flatten()).type(torch.long)

        # --- Forward Pass and Loss Calc ---
        optimiser.zero_grad()
        output, rnn_output = model(inputs) # Now Capturing rnn_output for activity loss
        output = output.view(-1, output_size)

        # 1. Task Loss (styll)
        task_loss = criterion(output, labels)

        # 2. Wiring Loss (is optional, adding 0 has no effect)
        wiring_loss = torch.tensor(0.0)
        if WD_approach:
            wiring_loss = WD_regulariser(model)
        elif 'penalise_weight_distance' in dir(model): # Check if method exists

             wiring_loss = wiring_beta * model.penalise_weight_distance()

        # 3. Activity Loss (optional)
        activity_loss = torch.tensor(0.0)
        if activity_regularisation:
            activity_loss = activity_beta * torch.mean(rnn_output**2)

        # Total Loss
        loss = task_loss + wiring_loss + activity_loss

        # --- Backprop and optmise step ---
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimiser.step()

        # --- Tracking all loss components  ---
        running_loss += loss.item()
        running_task_loss += task_loss.item()
        running_wiring_loss += wiring_loss.item()
        running_activity_loss += activity_loss.item()
        if i <= 1:
          print("--- First Step Losses to Check Beta----")
          print(f"Task Loss: {task_loss.item()}")
          print(f"Wiring Loss: {wiring_loss.item()}")
          print(f"Activity Loss: {activity_loss.item()}")
        if (i + 1) % 100 == 0:
            # running losses
            running_loss /= 100
            running_task_loss /= 100
            running_wiring_loss /= 100
            running_activity_loss /= 100

            print(f'Step {i+1}, Total Loss: {running_loss:.4f}, Time: {time.time() - start_time:.1f}s')
            print(f' Loss Components -> Task: {running_task_loss:.4f}, Wiring: {running_wiring_loss:.4f}, Activity: {running_activity_loss:.4f}')

            # Append to tracking lists
            tr_epochs_loss.append(i + 1)
            tr_total_loss_values.append(running_loss)
            tr_task_loss_values.append(running_task_loss)
            tr_wiring_loss_values.append(running_wiring_loss)
            tr_activity_loss_values.append(running_activity_loss)

            # Reset running losses
            running_loss, running_task_loss, running_wiring_loss, running_activity_loss = 0, 0, 0, 0

        # --- Early Stopping Check ---
        if (i + 1) % num_steps_for_early_stop_check == 0:
            model.eval()
            with torch.no_grad():
                current_perf_dict = early_stop_validation(model, validation_set_dict, num_validation_trials)
                list_avg_perfs = list(current_perf_dict.values())
                average_perf = np.mean(list_avg_perfs)

                tr_epochs_valid_perf.append(i + 1)
                tr_valid_perfs_list_of_dicts.append(current_perf_dict)
                tr_valid_perfs_avg.append(average_perf)
                print(f'Validation at Step {i+1} | Avg Perf: {average_perf:.3f}')

                # Print all performances by key and perf value , without new line
                for key, value in current_perf_dict.items():
                    print(f'{key}: {value:.3f}', end=' ')
                print()

                status_list = [p > min_validation_perf for p in list_avg_perfs]
                if all(status_list):
                    if not surpassed_threshold:
                        surpassed_threshold = True
                        n_steps_first_th = i + 1
                        num_steps_for_early_stop_check = 100 # Check more frequently after threshold
                        print("Performance threshold surpassed. Checking more frequently.")

                    if average_perf > current_best_valid_accuracy_es:
                        current_best_valid_accuracy_es = average_perf
                        best_valid_performance_es = list_avg_perfs
                        torch.save(model.state_dict(), model_save_name)
                        current_patience = patience # Reset patience
                        print(f"New best model saved with avg perf: {average_perf:.3f}")
                    else:
                        current_patience -= 1
                        print(f"No improvement. Patience left: {current_patience}")

                if surpassed_threshold and current_patience <= 0:
                    n_steps_final = i + 1
                    print('Early Stopping. I have run out of patience Grrr')
                    break
            model.train()

    # final tracking dict
    learning_curve_dict = {
        'training_loss_epochs_list': tr_epochs_loss,
        'training_total_loss_values_list': tr_total_loss_values,
        'training_task_loss_values_list': tr_task_loss_values,
        'training_wiring_loss_values_list': tr_wiring_loss_values,
        'training_activity_loss_values_list': tr_activity_loss_values,
        'validation_perf_epochs_list': tr_epochs_valid_perf,
        'validation_perf_dicts_list': tr_valid_perfs_list_of_dicts,
        'validation_perf_avg_values_list': tr_valid_perfs_avg,
        'n_steps_first_th': n_steps_first_th,
        'n_steps_final': n_steps_final
    }

    try:
        model.load_state_dict(torch.load(model_save_name))
        print('Best model loaded.')
        for valid_set_name, perf in zip(validation_set_dict.keys(), best_valid_performance_es):
            print(f'Best {valid_set_name} performance: {perf}')

    except FileNotFoundError:
        print('No best model was saved that satisfied the validation threshold.')

    if tr_output_mode:
        return model, learning_curve_dict
    return model

### Learning Curve Visualisation

In [ ]:
def plot_learning_curve(
    learning_curve_dict,
    average_only=False,
    plot_loss_components=True,
    filename=None,
    show_legend=True,
    legend_location='upper center'
):
    """
    Plots training losses and validation accuracy from a learning_curve_dict.

    Args:
        learning_curve_dict: dict containing learning curve data.
        average_only: If True, plots only average validation performance (instead of perf on each validation set).
        plot_loss_components: If True, plots individual loss components (wiring, activity) if they were used.
        filename: If provided, saves the figure (need to end with '.png'!.
        show_legend: If True, displays the legend.
        legend_location: Location of the legend (see bookmarked matplotlib page).
    """
    # --- Unpack Data from Dictionary ---
    # Loss Data
    tr_epochs_loss = learning_curve_dict['training_loss_epochs_list']
    tr_total_loss = learning_curve_dict['training_total_loss_values_list']
    tr_task_loss = learning_curve_dict.get('training_task_loss_values_list', [])
    tr_wiring_loss = learning_curve_dict.get('training_wiring_loss_values_list', [])
    tr_activity_loss = learning_curve_dict.get('training_activity_loss_values_list', [])

    # Validation Data
    val_epochs = learning_curve_dict['validation_perf_epochs_list']
    val_dicts = learning_curve_dict['validation_perf_dicts_list']
    val_avg = learning_curve_dict['validation_perf_avg_values_list']

    # Early Stopping Data
    n_steps_final = learning_curve_dict.get('n_steps_final', None)
    th_location = np.where(np.array(val_avg) > 0.8)[0][0] if np.any(np.array(val_avg) > 0.8) else None
    n_steps_first_th = val_epochs[th_location] if th_location is not None else None

    # --- Plotting ---
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot Training Losses on the left y-axis (ax1)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.plot(tr_epochs_loss, tr_total_loss, color='tab:blue', label='Total Training Loss')

    if plot_loss_components:
        # Plot wiring loss if it was used
        if np.any(tr_wiring_loss):
            ax1.plot(tr_epochs_loss, tr_wiring_loss, color='tab:green', linestyle=':', label='Wiring Loss')
        # Plot activity loss if it was used
        if np.any(tr_activity_loss):
            ax1.plot(tr_epochs_loss, tr_activity_loss, color='tab:cyan', linestyle=':', label='Activity Loss')

    # Create a second y-axis for validation performance (ax2)
    ax2 = ax1.twinx()
    ax2.set_ylabel('Validation Performance', color='tab:red')
    ax2.tick_params(axis='y', labelcolor='tab:red')
    ax2.set_ylim(0, 1.05) # Set y-axis for performance from 0 to 1

    # Plot Validation Performance on the right y-axis (ax2)
    if average_only:
        ax2.plot(val_epochs, val_avg, color='tab:red', linestyle='--', label='Average Validation Performance')
    else:
        if val_dicts:
            validation_set_names = list(val_dicts[0].keys())
            # Use a colormap with highly contrasting colours
            colors = plt.cm.get_cmap('Set1')(np.linspace(0, 1, len(validation_set_names)))

            for i, name in enumerate(validation_set_names):
                values = [d[name] for d in val_dicts]
                ax2.plot(val_epochs, values, color=colors[i], linestyle='-', label=f'{name} Performance')

    # Plot vertical lines for early stopping events
    if n_steps_first_th is not None:
        ax2.axvline(x=n_steps_first_th, color='gray', linestyle='--', label='Perf. Threshold Met')
    if n_steps_final is not None and n_steps_final > 0:
         ax2.axvline(x=n_steps_final, color='black', linestyle='--', label='Early Stopping')

    # Combine legends from both axes
    if show_legend:
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax2.legend(lines1 + lines2, labels1 + labels2, loc=legend_location)

    plt.title('Learning Curve: Training Loss and Validation Performance vs. Epoch')
    fig.tight_layout()

    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved to {filename}")

    plt.show()

# Testing Regime

## Testing - Functions

### Testing Functions - Run Test Trials and Record Data Function

In [ ]:
def evaluate_network_on_dataset(network, dataset_to_evaluate, num_trials=200):
    """
    Evaluates the given network on NGym dataset and records trial data.

    Args:
        network: The network to evaluate.
        dataset_to_evaluate: The NeuroGym dataset to use for evaluation.
        num_trials (int): The number of trials to run for evaluation.

    Returns:
        dict: A dictionary containing environmental information and detailed trial data
              including network activity, stimulus values, and correctness.
    """
    # Reset environment
    environment = dataset_to_evaluate.env
    environment.reset()

    # Initialise variables for logging
    # Stores correctness for calculating average performance
    trial_correctness_records = {}
    # Stores detailed information and network activity for each trial
    trial_data_and_activity = {}

    # Store the information of the trial environment
    # For reporting or future devs with sequence isolation
    # NOTE: for trial envs with varying task period timings this will need to be called IN THE LOOP AND STORED ALONGSIDE EACH TRIAL
    # NOTE : Having trial envs with varying task period timings also cocks up the results data processing func so dont forget

    environment_info = {
            'dt' : environment.dt,
            'trial_start_ind': environment.start_ind,
            'trial_end_ind': environment.end_ind,
            'timing': environment.timing,
            'sigma': environment.sigma,
            'choices': environment.choices,
            'stim_values_array': environment.theta
            }


    for trial_index in range(num_trials):
        # Initialise dictionary for the current trial's data
        trial_data_and_activity[trial_index] = {}

        # Sample a new trial
        trial_information = environment.new_trial()

        # Observation and groud-truth of this trial
        observation, ground_truth = environment.ob, environment.gt

        # Convert to numpy, add batch dimension to input (for consistency)
        inputs_tensor = torch.from_numpy(observation[:, np.newaxis, :]).type(torch.float)

        # Run the network for one trial
        # inputs_tensor (SeqLen, Batch, InputSize)
        # predicted_actions (SeqLen, Batch, OutputSize) (output layer activity across all timesteps)
        # hidden_activity (seq_len, batch, hidden layer size)
        prediction_action, hidden_activity = network(inputs_tensor)

        # Compute performance
        # First convert back to numpy
        prediction_action_np = prediction_action.detach().numpy()[:, 0, :]
        # Read out final choice at last time step
        network_choice = np.argmax(prediction_action_np[-1, :])
        # Compare to ground truth
        is_correct = network_choice == ground_truth[-1]

        # Record activity, trial information, choice, correctness
        hidden_activity_np = hidden_activity[:, 0, :].detach().numpy() # becomes (seq_len, hidden_size)

        # Update all logging of trial information and network activity
        trial_correctness_records[trial_index] = {}
        trial_correctness_records[trial_index].update({'correct': is_correct})

        trial_data_and_activity[trial_index]['network_activity'] = hidden_activity_np
        trial_data_and_activity[trial_index]['sample_stim_value'] = trial_information['sample_theta']
        trial_data_and_activity[trial_index]['test_stim_value'] = trial_information['test_theta']
        trial_data_and_activity[trial_index]['test_equals_sample'] = True if int(trial_information['ground_truth']) == 1 else False
        trial_data_and_activity[trial_index]['network_correct'] = is_correct

    # Print Average trial performance during testing
    average_trial_performance = np.mean([val['correct'] for val in trial_correctness_records.values()])
    print('Average performance', average_trial_performance)

    # PACKING THE DICTS
    full_testing_data = {
        'testing_env_info' : environment_info,
        'testing_trial_and_activity': trial_data_and_activity,
        'testing_trial_performance': average_trial_performance
    }

    return full_testing_data

In [ ]:
def bic_testing_w_state_tracking(network, dataset_to_evaluate, num_trials=200):
    """
    Evaluates the given network on a NGym dataset and records trial data.
    Needs to have .forward_for_fpf_ics(self, x) method implemented.

    Args:
        network: The network to evaluate.
        dataset_to_evaluate: The NeuroGym dataset to use for evaluation.
        num_trials: The number of trials to run for evaluation.

    Returns:
        dict: A dictionary containing environmental information and detailed trial data
              including network activity, stimulus values, and correctness.
    """
    # Reset environment
    environment = dataset_to_evaluate.env
    environment.reset()

    # Initialise variables for logging
    # Stores correctness for calculating average performance
    trial_correctness_records = {}
    # Stores detailed information and network activity for each trial
    trial_data_and_activity = {}

    # Store the information of the trial environment
    # For reporting or future devs with sequence isolation
    # NOTE: for trial envs with varying task period timings this will need to be called IN THE LOOP AND STORED ALONGSIDE EACH TRIAL
    # NOTE : Having trial envs with varying task period timings also cocks up the results data processing func so dont forget

    environment_info = {
            'dt' : environment.dt,
            'trial_start_ind': environment.start_ind,
            'trial_end_ind': environment.end_ind,
            'timing': environment.timing,
            'sigma': environment.sigma,
            'choices': environment.choices,
            'stim_values_array': environment.theta
            }


    for trial_index in range(num_trials):
        # Initialise dictionary for the current trial's data
        trial_data_and_activity[trial_index] = {}

        # Sample a new trial
        trial_information = environment.new_trial()

        # Observation and groud-truth of this trial
        observation, ground_truth = environment.ob, environment.gt

        # Convert to numpy, add batch dimension to input (for consistency)
        inputs_tensor = torch.from_numpy(observation[:, np.newaxis, :]).type(torch.float)

        # Run the network for one trial
        # inputs_tensor (SeqLen, Batch, InputSize)
        # predicted_actions (SeqLen, Batch, OutputSize) (output layer activity across all timesteps)
        # hidden_activity (seq_len, batch, hidden layer size)
        # prediction_action, hidden_activity = network(inputs_tensor)
        prediction_action, hidden_activity, hidden_state_tensor = network.forward_for_fpf_ics(inputs_tensor)

        # Compute performance
        # First convert back to numpy
        prediction_action_np = prediction_action.detach().numpy()[:, 0, :]
        # Read out final choice at last time step
        network_choice = np.argmax(prediction_action_np[-1, :])
        # Compare to ground truth
        is_correct = network_choice == ground_truth[-1]

        # Record activity, trial information, choice, correctness
        hidden_activity_np = hidden_activity[:, 0, :].detach().numpy() # becomes (seq_len, hidden_size)
        hidden_state_np = hidden_state_tensor[:, 0, :].detach().numpy()
        if hidden_activity_np.shape[0] != hidden_state_np.shape[0]:
          print('gotcha')
        # Update all logging of trial information and network activity
        trial_correctness_records[trial_index] = {}
        trial_correctness_records[trial_index].update({'correct': is_correct}) # Redundant but kept for original output structure

        # trial_data_and_activity[trial_index]['network_activity'] = hidden_state_np ### NOTE THIS HAS BEEN CHANGED TO SEE HOW ANALYSIS CHANGES ### IF THESE ARE SWAPPED BACK (i.e. activity = output activity), the hidden_key input for bic fpf needs to be changed to = 'network_hidden_state'
        # trial_data_and_activity[trial_index]['network_hidden_state'] = hidden_activity_np
        trial_data_and_activity[trial_index]['network_activity'] = hidden_activity_np ### NOTE THIS HAS BEEN CHANGED TO SEE HOW ANALYSIS CHANGES ### IF THESE ARE SWAPPED BACK (i.e. activity = output activity), the hidden_key input for bic fpf needs to be changed to = 'network_hidden_state'
        trial_data_and_activity[trial_index]['network_hidden_state'] = hidden_state_np
        trial_data_and_activity[trial_index]['sample_stim_value'] = trial_information['sample_theta']
        trial_data_and_activity[trial_index]['test_stim_value'] = trial_information['test_theta']
        trial_data_and_activity[trial_index]['test_equals_sample'] = True if int(trial_information['ground_truth']) == 1 else False
        trial_data_and_activity[trial_index]['network_correct'] = is_correct

    # Print Average trial performance during testing
    average_trial_performance = np.mean([val['correct'] for val in trial_correctness_records.values()])
    print('Average performance', average_trial_performance)

    # PACKING THE DICTS - cut down so now returning essentials
    full_testing_data = {
        'testing_env_info' : environment_info,
        'testing_trial_and_activity': trial_data_and_activity,
        'testing_trial_performance': average_trial_performance
    }

    return full_testing_data

# ----- ANALYSIS FUNCTIONS ------



## VISUALISATION FUNCTIONS

### Single Trial Unit Activty Plotting Function

In [ ]:
# Maybe this should take the whole trial dict, not just the activity? would be good for the title
def plot_unit_activity_over_time(trial_activity, env_info, unit_indices_to_plot=None, legend = False, filename=None):
    """
    plots the activty of all the units in a trial over time.
    can also just plot a specefic subset of them if you want.
    """
    seq_len, hidden_size = trial_activity.shape
    dt = env_info['dt']
    timing = env_info['timing']

    # make the time axis
    time_points = np.arange(seq_len) * dt

    # get the fig and ax so we can save it later
    fig, ax = plt.subplots(figsize=(15, 7))

    # figure out which units to plot
    units_to_plot = range(hidden_size)
    if unit_indices_to_plot is not None:
        units_to_plot = unit_indices_to_plot
        # just make sure the indicies are actually in the array
        units_to_plot = [i for i in units_to_plot if 0 <= i < hidden_size]


    for i in units_to_plot:
        ax.plot(time_points, trial_activity[:, i], label=f'Unit {i+1}')

    # draw the lines for the different task bits
    current_time = 0
    # bit fiddly, just makes sure we dont get duplicate labels in the legend
    existing_legend_labels = [l.get_text() for l in ax.get_legend().get_texts()] if legend and ax.get_legend() else []

    for phase, duration in timing.items():
        # only works for fixed timings for now
        if isinstance(duration, (int, float)):
            current_time += duration
            # don't draw a line if it's off the edge of the plot
            if current_time < time_points[-1]:
                # add the line, but only give it a label if we want a legend
                # and if that label isn't already there
                ax.axvline(x=current_time, color='k', linestyle='--', label=phase if legend and phase not in existing_legend_labels else "")


    ax.set_title('Network Unit Activity Over Time')
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Activity')
    if legend:
      # put a legend on if we asked for one. can get a bit crowded
      ax.legend()
    ax.grid(True)

    # save the plot if we got a filename, otherwise just show it
    if filename:
        try:
            # make sure the directory exists
            if os.path.dirname(filename):
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
            # save it out
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {filename}")
            # close the plot so it doesn't just pop up anyway / saves colab ram
            plt.close(fig)
        except Exception as e:
            # if saving went wrong for some reason, just show the plot instead
            print(f"Error saving figure to {filename}: {e}")
            plt.show()
    else:
        plt.show()

### Average Unit Activity Function

In [ ]:
def plot_average_unit_activity_over_time(trial_data_dict, env_info, unit_indices_to_plot=None, legend=False, filename=None):
    """
    Averages the unit activty across a number of trials and then plots it.
    You can also provide a list of speific units to plot.
    """
    if not trial_data_dict:
        print("No trial data provided for plotting.")
        return

    # First, get the shape info from one of the trials
    first_trial_key = list(trial_data_dict.keys())[0]
    first_trial_activity = trial_data_dict[first_trial_key]['network_activity']
    seq_len, hidden_size = first_trial_activity.shape
    dt = env_info['dt']
    timing = env_info['timing']

    # Go through all the trials and stack their activities together
    all_activities = []
    for trial_data in trial_data_dict.values():
        # just a check to make sure the shapes match up
        if trial_data['network_activity'].shape == (seq_len, hidden_size):
            all_activities.append(trial_data['network_activity'])
        else:
            # never happens but just incase
            print(f"Warning: Skipping trial with inconsistent shape: {trial_data['network_activity'].shape}")

    if not all_activities:
        print("No valid trial data found for averaging.")
        return

    all_activities_stacked = np.stack(all_activities, axis=0)

    # Now we can average across all the trials
    average_activity = np.mean(all_activities_stacked, axis=0)

    # create the time axis for the plot
    time_points = np.arange(seq_len) * dt

    # get the fig and ax so we can save the plot
    fig, ax = plt.subplots(figsize=(15, 7))


    # Decide which units we're actually plotting
    units_to_plot = range(hidden_size)
    if unit_indices_to_plot is not None:
        units_to_plot = unit_indices_to_plot
        # Make sure the requested indicies are valid
        units_to_plot = [i for i in units_to_plot if 0 <= i < hidden_size]


    for i in units_to_plot:
        ax.plot(time_points, average_activity[:, i], label=f'Avg Unit {i+1}')

    # Add the vertical lines for task phases
    current_time = 0
    # a check to prevent duplicate labels appearing in the legend
    existing_legend_labels = [l.get_text() for l in ax.get_legend().get_texts()] if legend and ax.get_legend() else []

    for phase, duration in timing.items():
        # This only works for the fixed timings
        if isinstance(duration, (int, float)):
            current_time += duration
            # don't draw lines that would be off the chart
            if current_time < time_points[-1]:
                ax.axvline(x=current_time, color='k', linestyle='--', label=phase if legend and phase not in existing_legend_labels else "")
        else:
            # For handling variable timings in future if needed
            print(f"Skipping phase '{phase}' with variable timing for plotting vertical lines.")


    ax.set_title('Average Network Unit Activity Over Time Across Trials')
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Average Activity')
    # The legend is optional as it can sometimes mess up the plot
    if legend:
      ax.legend()
    ax.grid(True)

    # Save the figure or show it
    if filename:
        try:
            # make sure the directory exists first
            if os.path.dirname(filename):
                    os.makedirs(os.path.dirname(filename), exist_ok=True)
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {filename}")
            # close the plot so it doesn't show up after saving
            plt.close(fig)
        except Exception as e:
            # if saving fails, show the plot instead
            print(f"Error saving figure to {filename}: {e}")
            plt.show()
    else:
        plt.show()

### Heat Map for Network Structure - BIC Change

In [ ]:
import os
import matplotlib.colors as mcolors
import matplotlib.patches as patches

def plot_recurrent_weights_heatmap_BIC(net, filename = None, mask=None, weights_passed=False):
    """
    Makes a heatmap of the recurrent weights from the RNN's h2h layer.
    It expects the network to have a net.rnn.h2h.weight structure.
    """
    try:
        if mask is not None:
            effective_weights = net.rnn.h2h.weight * mask
            weights = effective_weights.detach().cpu().numpy()
        elif weights_passed:
            weights = net
        else:
            # get the recurrent weights from the linear layer
            weights = net.rnn.h2h.weight.detach().cpu().numpy()
    except AttributeError:
        print("Error: Could not find 'net.rnn.h2h.weight'.")
        return

    num_units = weights.shape[0]

    # set up a symmetric colour range for the plot
    max_abs_val = np.max(np.abs(weights))
    norm = mcolors.Normalize(vmin=-max_abs_val, vmax=max_abs_val)

    # create the plot
    fig, ax = plt.subplots(figsize=(12, 10))

    # use imshow to create the heatmap itself
    im = ax.imshow(weights, cmap='bwr_r', norm=norm, interpolation='none')

    # configure the colour bar
    cbar = fig.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('Connection Weight', rotation=270, labelpad=20, fontsize=12)

    # configure the axes and labels
    ax.set_title('Recurrent Connection Weights Heatmap', fontsize=16, pad=20)
    ax.set_ylabel('To Unit Index', fontsize=12)
    ax.set_xlabel('From Unit Index', fontsize=12)

    # configure the main ticks to be centered and rotated
    tick_labels = np.arange(num_units)
    ax.set_xticks(tick_labels)
    ax.set_yticks(tick_labels)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)

    plt.setp(
        ax.get_xticklabels(),
        rotation=90,
        ha="center",
        rotation_mode="default"
    )

    # add grid lines and borders inside the heatmap
    # set minor ticks to sit between the major ones, which is where the grid lines go
    ax.set_xticks(np.arange(-.5, num_units, 1), minor=True)
    ax.set_yticks(np.arange(-.5, num_units, 1), minor=True)

    # add some faint gridlines to all the cells
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.25, alpha=0.5)

    # this is important - hide the minor tick marks on the axes themselves
    ax.tick_params(which='minor', bottom=False, left=False)

    # add a slightly more prominent border for the diagonal cells
    for i in range(num_units):
        rect = patches.Rectangle(
            (i - 0.5, i - 0.5), 1, 1,
            linewidth=0.75, edgecolor='black', facecolor='none', alpha=0.7
        )
        ax.add_patch(rect)

    fig.tight_layout(pad=1.5)

    # save or display the figure
    if filename:
      plt.savefig(filename, dpi=300, bbox_inches='tight')
      print(f"Figure saved to {filename}")
      plt.close(fig)
    else:
        plt.show()

### Heatmap for Trial Phase Period Selectivity (TBD)

## DIMENSIONALITY REDUCTION

### dPCA (I/O)

#### (O)

In [ ]:
import json
import numpy as np

def export_nested_dict_to_json(data, filename):
    """
    Exports a nested dictionary containing NumPy arrays to a JSON file.

    Recursively converts NumPy arrays to lists before exporting and adds an
    indicator to mark lists that were originally NumPy arrays.

    Args:
        data (dict): The nested dictionary to export.
        filename (str): The name of the JSON file to create.
    """
    def convert_numpy_to_list(obj):
        """
        Recursively converts NumPy arrays within an object to lists and adds a marker.
        """
        if isinstance(obj, np.ndarray):
            # Convert to list and add a marker at the beginning
            return ['NUMPY'] + obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert_numpy_to_list(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_to_list(item) for item in obj]
        elif isinstance(obj, (np.float64, np.bool_, np.int64, np.int32)): # Added more scalar types
             return obj.item() # Convert NumPy scalar types to Python native types
        else:
            return obj

    export_ready_data = convert_numpy_to_list(data)

    try:
        with open(filename, 'w') as f:
            json.dump(export_ready_data, f, indent=4)
        print(f"Data successfully exported to {filename}")
    except Exception as e:
        print(f"Error exporting data to {filename}: {e}")


#### (I)

In [ ]:
import json

def import_nested_dict_from_json(filename):
    """
    Loads data from a JSON file that was saved with the other function.
    It finds any lists marked as 'NUMPY' and turns them back into numpy arrays.
    """
    def convert_list_to_numpy_array(obj):
        """The helper that does the actual work of finding and converting the lists."""
        if isinstance(obj, dict):
            return {k: convert_list_to_numpy_array(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            # look for our 'NUMPY' marker at the start of a list
            if obj and isinstance(obj[0], str) and obj[0] == 'NUMPY':
                # if we find it, chop the marker off and convert the rest to an array
                try:
                    # process inner elements first, in case of nested arrays
                    list_without_marker = [convert_list_to_numpy_array(item) for item in obj[1:]]
                    return np.array(list_without_marker)
                except Exception as e:
                    print(f"Warning: Could not convert list marked as 'NUMPY' to array. Returning list. Error: {e}")
                    # if something goes wrong, just give back the list as it was
                    return obj
            else:
                # if there's no marker, just keep digging through the list
                return [convert_list_to_numpy_array(item) for item in obj]
        else:
            return obj

    try:
        with open(filename, 'r') as f:
            imported_data = json.load(f)

        reconstructed_data = convert_list_to_numpy_array(imported_data)
        print(f"Data successfully imported from {filename}")
        return reconstructed_data

    except FileNotFoundError:
        print(f"Error: File not found at {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {filename}")
        return None
    except Exception as e:
        print(f"Error importing data from {filename}: {e}")
        return None


Accompanying Import Function (also located in dPCA notebook)

#### Check I/O

Function to Check if the dict you are exporting will be imported as an exact match in the dPCA notebook

In [ ]:
import deepdiff

def compare_dicts(dict1, dict2):
    """
    Compares two dictionaries and reports the differences.
    Handles NumPy arrays by comparing their content.

    Args:
        dict1 (dict): The first dictionary.
        dict2 (dict): The second dictionary.

    Returns:
        dict: A dictionary containing the differences.
    """

    diff = deepdiff.DeepDiff(dict1, dict2, ignore_order=False, verbose_level=0)

    if not diff:
        print("Dictionaries are identical.")
    else:
        print("Differences found:")
        print(diff)

    return diff


### PCA

#### Fit PCA

In [ ]:
def fit_pca_on_selected_data(dict_of_trial_dicts, pca_components=2, report_var_expls = False):
    '''
    Fits a PCA model on the network activity from a dictionary of trials.

    This function can be used in two ways:
    1. To explore the data: set pca_components to None and report_var_expls to True.
       This will print out variance info and show scree plots to help you
       decide how many componants to use.

    2. To get a model: set pca_components to a number (e.g. 2).
       This will return the actual fitted PCA object.
    '''
    list_of_activity_arrays = []
    for trial_num, trial_dict in dict_of_trial_dicts.items():
        list_of_activity_arrays.append(trial_dict['network_activity'])

    # combine all the trial activities into one big array for the PCA
    activity_for_fit = np.concatenate(list_of_activity_arrays, axis=0)
    # just a quick check on the shape to make sure it's right
    # print(f'Checking Dimensionality should be (<seqlen*n_trials>, hidden size)')
    # print(f'{activity_for_fit.shape}')
    pca = PCA(n_components=pca_components)
    pca.fit(activity_for_fit)

    if report_var_expls:

        print("--- Detailed PCA Variance Explanation ---")

        # print the variance explained by each component as a percentage
        for i, ratio in enumerate(pca.explained_variance_ratio_):
            print(f"Principal Component {i+1}: {ratio*100:.2f}% of total variance explained")

        # and the cumulative variance
        cumulative_variance_percent = np.cumsum(pca.explained_variance_ratio_) * 100
        for i, cum_percent in enumerate(cumulative_variance_percent):
            print(f"Cumulative Variance Explained by first {i+1} components: {cum_percent:.2f}%")

        print(f"Total variance explained by all {pca.n_components_} selected components: {pca.explained_variance_ratio_.sum()*100:.2f}%")

        # make a scree plot
        plt.figure(figsize=(10, 6))
        plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
        plt.xlabel('Principal Component Number')
        plt.ylabel('Proportion of Variance Explained')
        plt.title('Scree Plot: Explained Variance Ratio per Component')
        plt.xticks(range(1, len(pca.explained_variance_ratio_) + 1))
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

        # and another plot for the cumulative variance
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(cumulative_variance_percent) + 1), cumulative_variance_percent, marker='o', linestyle='-')
        plt.xlabel('Number of Principal Components')
        plt.ylabel('Cumulative Explained Variance (%)')
        plt.title('Cumulative Explained Variance vs. Number of Components')
        plt.xticks(range(1, len(cumulative_variance_percent) + 1))
        plt.grid(True, linestyle='--', alpha=0.7)
        # an example threshold line
        plt.axhline(y=90, color='r', linestyle='--', label='90% Threshold')
        plt.legend()
        plt.show()
    else:
        return pca

#### Plot PC1 against time Function

------WONT WORK FOR DIFF TASK------

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import defaultdict

def plot_pca_trajectories(pca_object, data_to_transform_dict, testing_env_info, num_trials_to_plot=100, plot_title=None, save_filename=None):
    """
    Takes a fitted PCA object and some data and plots the average PC1 trajectries
    over time. It colours the lines based on the sample and test stimulus values,
    using the project's style guide.

    The plot_title argument is ignored to make sure the plots all look the same.
    """
    if not data_to_transform_dict:
        print("No trial data provided for plotting PCA trajectories.")
        return

    # ---- Style guide definitions ----
    # these are the specific colours for the different trial conditions
    STYLE_GUIDE_COLORS = {
        (0.0, 0.0):   {'color': '#68d3e8', 'label': 'S: 0.00, T: 0.00'},
        (3.14, 0.0):  {'color': '#e69010', 'label': 'S: 3.14, T: 0.00'},
        (0.0, 3.14):  {'color': '#6b08bd', 'label': 'S: 0.00, T: 3.14'},
        (3.14, 3.14): {'color': '#69420c', 'label': 'S: 3.14, T: 3.14'},
    }
    # we only need one linestyle for this plot
    LINESTYLE = '-'

    # ---- Preparing the data ----
    # first, group the trials by their stimulus condition
    grouped_trajectories = defaultdict(list)
    limited_trials = dict(list(data_to_transform_dict.items())[:num_trials_to_plot])

    for trial_dict in limited_trials.values():
        stim_condition = (
            round(trial_dict['sample_stim_value'], 2),
            round(trial_dict['test_stim_value'], 2)
        )
        if stim_condition in STYLE_GUIDE_COLORS:
            activity = np.array(trial_dict['network_activity'])
            # just the first principal component
            pc1_trajectory = pca_object.transform(activity)[:, 0]
            grouped_trajectories[stim_condition].append(pc1_trajectory)

    # ---- Setting up the plot ----
    fig, ax = plt.subplots(figsize=(16, 9))
    max_plotted_time = 0

    # ---- Plot the mean trajectories ----
    for condition, trajectories in grouped_trajectories.items():
        if trajectories:
            # average the trajectories for this condition
            mean_trajectory = np.mean(np.array(trajectories), axis=0)

            # get the right colour and label
            colour = STYLE_GUIDE_COLORS[condition]['color']
            label = STYLE_GUIDE_COLORS[condition]['label']

            # make the time axis
            time_points = np.arange(len(mean_trajectory)) * testing_env_info['dt']
            max_plotted_time = max(max_plotted_time, time_points[-1])

            ax.plot(time_points, mean_trajectory, color=colour, linestyle=LINESTYLE, label=label, linewidth=2.5)

    # ---- Add lines and labels for task phases ----
    task_env_dt = testing_env_info['dt']
    ax.axvline(x=0, color='k', linestyle='--', linewidth=1)

    for end_index in testing_env_info['trial_end_ind'].values():
        end_time_ms = (end_index-1) * task_env_dt
        if end_time_ms <= max_plotted_time + task_env_dt:
            ax.axvline(x=end_time_ms, color='k', linestyle='--', linewidth=1)

    y_min, y_max = ax.get_ylim()
    # put the phase labels near the top of the plot
    annotation_y_pos = y_max - (y_max - y_min) * 0.05

    for phase, start_index in testing_env_info['trial_start_ind'].items():
        start_time_ms = start_index * task_env_dt
        end_time_ms = (testing_env_info['trial_end_ind'][phase] - 1) * task_env_dt
        center_time_ms = (start_time_ms + end_time_ms) / 2
        if center_time_ms <= max_plotted_time:
            ax.text(center_time_ms, annotation_y_pos, phase.capitalize(),
                    ha='center', va='top', fontsize=12, color='black')

    # ---- Final tweaks to the plot ----
    ax.set_xlabel('Time (ms)', fontsize=14)
    ax.set_ylabel('PC1 (from PCA fit)', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)

    # add a bit of padding to the y-axis so the lines don't hit the top/bottom
    y_min, y_max = ax.get_ylim()
    y_range = y_max - y_min
    ax.set_ylim(y_min - y_range * 0.1, y_max + y_range * 0.1)

    ax.legend(title="Trial Conditions", loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=10)
    fig.tight_layout(rect=[0, 0, 0.85, 1]) # make space for the legend outside

    # ---- Save or show the plot ----
    if save_filename:
        try:
            # make sure the directory exists
            if os.path.dirname(save_filename):
                os.makedirs(os.path.dirname(save_filename), exist_ok=True)
            plt.savefig(save_filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {save_filename}")
        except Exception as e:
            print(f"Error saving figure to {save_filename}: {e}")
        plt.close(fig)
    else:
        plt.show()

#### 2 Datasets 1 PCA Plotting 1D (For Leisoning)

------WONT WORK FOR DIFF TASK------

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
from collections import defaultdict

def plot_pca_trajectories_two_datasets(pca_object, data_to_transform_dict1, data_to_transform_dict2, testing_env_info, num_trials_to_plot=100, plot_title=None, save_filename=None, label1="Unlesioned", label2="Lesioned"):
    """
    Plots and compares the mean PC1 trajectories from two different datasets,
    for example an 'Unlesioned' vs 'Lesioned' network.

    The line colour shows the task condition, while the line style (solid/dashed)
    shows which dataset it came from. The plot title is ignored for consistancy.
    """
    if not data_to_transform_dict1 and not data_to_transform_dict2:
        print("No trial data provided from either dataset for plotting.")
        return

    # ---- Style guide definitions ----
    STYLE_GUIDE_COLORS = {
        (0.0, 0.0):   {'color': '#68d3e8', 'label': 'S: 0.00, T: 0.00'},
        (3.14, 0.0):  {'color': '#e69010', 'label': 'S: 3.14, T: 0.00'},
        (0.0, 3.14):  {'color': '#6b08bd', 'label': 'S: 0.00, T: 3.14'},
        (3.14, 3.14): {'color': '#69420c', 'label': 'S: 3.14, T: 3.14'},
    }
    # solid for dataset 1, dashed for dataset 2
    DATASET_STYLES = [
        {'linestyle': '-', 'label': label1},
        {'linestyle': '--', 'label': label2}
    ]

    fig, ax = plt.subplots(figsize=(16, 9))
    max_plotted_time = 0
    all_plotted_conditions = set()

    datasets = [(data_to_transform_dict1, DATASET_STYLES[0]), (data_to_transform_dict2, DATASET_STYLES[1])]

    # loop through both datasets and plot their trajectories
    for data_dict, style in datasets:
        if not data_dict:
            continue

        # group the trials by their stimulus condition
        grouped_trajectories = defaultdict(list)
        limited_trials = dict(list(data_dict.items())[:num_trials_to_plot])

        for trial_dict in limited_trials.values():
            stim_condition = (round(trial_dict['sample_stim_value'], 2), round(trial_dict['test_stim_value'], 2))
            if stim_condition in STYLE_GUIDE_COLORS:
                activity = np.array(trial_dict['network_activity'])
                pc1_trajectory = pca_object.transform(activity)[:, 0]
                grouped_trajectories[stim_condition].append(pc1_trajectory)
                all_plotted_conditions.add(stim_condition)

        # now plot the mean trajectory for each condition in this dataset
        for condition, trajectories in grouped_trajectories.items():
            if trajectories:
                mean_trajectory = np.mean(np.array(trajectories), axis=0)
                colour = STYLE_GUIDE_COLORS[condition]['color']
                time_points = np.arange(len(mean_trajectory)) * testing_env_info['dt']
                max_plotted_time = max(max_plotted_time, time_points[-1])

                ax.plot(time_points, mean_trajectory, color=colour, linestyle=style['linestyle'], linewidth=2.5)

    # ---- Add lines and labels for task phases ----
    task_env_dt = testing_env_info['dt']
    ax.axvline(x=0, color='k', linestyle='--', linewidth=1)
    for end_index in testing_env_info['trial_end_ind'].values():
        end_time_ms = (end_index-1) * task_env_dt
        if end_time_ms <= max_plotted_time + task_env_dt:
            ax.axvline(x=end_time_ms, color='k', linestyle='--', linewidth=1)

    y_min, y_max = ax.get_ylim()
    annotation_y_pos = y_max - (y_max - y_min) * 0.05
    for phase, start_index in testing_env_info['trial_start_ind'].items():
        start_time_ms = start_index * task_env_dt
        end_time_ms = (testing_env_info['trial_end_ind'][phase]-1) * task_env_dt
        center_time_ms = (start_time_ms + end_time_ms) / 2
        if center_time_ms <= max_plotted_time:
            ax.text(center_time_ms, annotation_y_pos, phase.capitalize(), ha='center', va='top', fontsize=12)

    # ---- Final plot tweaks ----
    ax.set_xlabel('Time (ms)', fontsize=14)
    ax.set_ylabel('PC1 (from PCA fit)', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    y_min, y_max = ax.get_ylim()
    y_range = y_max - y_min
    ax.set_ylim(y_min - y_range * 0.1, y_max + y_range * 0.1)

    # ---- Build the legend manually ----
    legend_handles = []
    # First, the legend entries for the line styles (the two datasets)
    legend_handles.append(Line2D([0], [0], color='k', lw=2, linestyle=DATASET_STYLES[0]['linestyle'], label=DATASET_STYLES[0]['label']))
    legend_handles.append(Line2D([0], [0], color='k', lw=2, linestyle=DATASET_STYLES[1]['linestyle'], label=DATASET_STYLES[1]['label']))

    # add an invisible spacer to seperate the two parts of the legend
    legend_handles.append(Line2D([0], [0], color='w', label=''))

    # Second, the legend entries for the colours (the trial conditions)
    sorted_conditions = sorted(list(all_plotted_conditions))
    for condition in sorted_conditions:
        style = STYLE_GUIDE_COLORS[condition]
        legend_handles.append(Line2D([0], [0], color=style['color'], lw=2.5, label=style['label']))

    ax.legend(handles=legend_handles, title="Legend", loc='upper left', bbox_to_anchor=(1.02, 1))
    fig.tight_layout(rect=[0, 0, 0.85, 1]) # make space for the legend

    # ---- Save or show the plot ----
    if save_filename:
        try:
            if os.path.dirname(save_filename):
                os.makedirs(os.path.dirname(save_filename), exist_ok=True)
            plt.savefig(save_filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {save_filename}")
        except Exception as e:
            print(f"Error saving figure to {save_filename}: {e}")
        plt.close(fig)
    else:
        plt.show()

#### Plot PC1(hla) vs PC2(hla)  

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
from collections import defaultdict

def plot_pca_trajectories_2d(pca_object, data_to_transform_dict, testing_env_info,
                             num_trials_to_plot=100, plot_title=None, save_filename=None,
                             add_phase_markers=True):
    """
    Plots the average PC1 vs PC2 trajectories from network activity.
    The lines are coloured based on the trial condition, and it adds markers
    for the start, end, and phase transitions of each trajectory.

    The plot title is ignored for consistancy.
    """
    if not data_to_transform_dict:
        print("No trial data provided for plotting PCA trajectories.")
        return

    if pca_object.n_components < 2:
        print(f"Error: PCA object must be fit with n_components>=2, but has {pca_object.n_components}")
        return

    # ---- Style guide definitions ----
    STYLE_GUIDE_COLORS = {
        (0.0, 0.0):   {'color': '#68d3e8', 'label': 'S: 0.00, T: 0.00'},
        (3.14, 0.0):  {'color': '#e69010', 'label': 'S: 3.14, T: 0.00'},
        (0.0, 3.14):  {'color': '#6b08bd', 'label': 'S: 0.00, T: 3.14'},
        (3.14, 3.14): {'color': '#69420c', 'label': 'S: 3.14, T: 3.14'},
    }
    # single dataset so all lines are solid
    LINESTYLE = '-'

    # ---- Preparing the data ----
    grouped_trajectories = defaultdict(list)
    limited_trials = dict(list(data_to_transform_dict.items())[:num_trials_to_plot])

    for trial_dict in limited_trials.values():
        stim_condition = (
            round(trial_dict['sample_stim_value'], 2),
            round(trial_dict['test_stim_value'], 2)
        )
        if stim_condition in STYLE_GUIDE_COLORS:
            activity = np.array(trial_dict['network_activity'])
            # grab the first two principal components (PC1 and PC2)
            pc_trajectory = pca_object.transform(activity)[:, :2]
            grouped_trajectories[stim_condition].append(pc_trajectory)

    # ---- Setting up the plot ----
    fig, ax = plt.subplots(figsize=(12, 12))
    # add a faint grid and centre lines
    ax.grid(True, linestyle='--', color='lightgrey')
    ax.axhline(0, color='grey', linestyle='-', linewidth=0.8)
    ax.axvline(0, color='grey', linestyle='-', linewidth=0.8)
    # make sure the plot is square
    ax.set_aspect('equal', adjustable='box')

    # ---- Plot the mean trajectories and markers ----
    plotted_conditions = []
    for condition, trajectories in grouped_trajectories.items():
        if trajectories:
            plotted_conditions.append(condition)
            mean_trajectory = np.mean(np.array(trajectories), axis=0)
            colour = STYLE_GUIDE_COLORS[condition]['color']

            # plot the main trajectory line
            ax.plot(mean_trajectory[:, 0], mean_trajectory[:, 1], color=colour, linestyle=LINESTYLE, linewidth=2.5)

            # add a black square for the start point
            ax.plot(mean_trajectory[0, 0], mean_trajectory[0, 1], marker='s', color='k', markersize=8, linestyle='None')

            # add a black cross for the end point
            ax.plot(mean_trajectory[-1, 0], mean_trajectory[-1, 1], marker='X', color='k', markersize=10, mew=2, linestyle='None')

            # optionally add gold stars for phase transitions
            if add_phase_markers:
                for end_index in testing_env_info['trial_end_ind'].values():
                    marker_idx = end_index - 1
                    if marker_idx < len(mean_trajectory):
                        ax.plot(mean_trajectory[marker_idx, 0], mean_trajectory[marker_idx, 1],
                                marker='*', color='gold', markersize=12, linestyle='None')

    # ---- Build the legend manually ----
    legend_handles = []
    # first, the handles for the different coloured lines
    for condition in sorted(plotted_conditions):
          style = STYLE_GUIDE_COLORS[condition]
          legend_handles.append(Line2D([0], [0], color=style['color'], lw=2.5, label=style['label']))

    # next, the handles for the different marker types
    legend_handles.append(Line2D([0], [0], marker='s', color='k', label='Start', linestyle='None', markersize=8))
    legend_handles.append(Line2D([0], [0], marker='X', color='k', label='End', linestyle='None', markersize=10, mew=2))
    if add_phase_markers and plotted_conditions:
        legend_handles.append(Line2D([0], [0], marker='*', color='gold', label='Phase Transition', linestyle='None', markersize=12))

    ax.legend(handles=legend_handles, title="Legend", loc='upper left', bbox_to_anchor=(1.02, 1))

    # ---- Final plot tweaks ----
    ax.set_xlabel('Principal Component 1', fontsize=14)
    ax.set_ylabel('Principal Component 2', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    fig.tight_layout(rect=[0, 0, 0.85, 1])

    # ---- Save or show the plot ----
    if save_filename:
        try:
            if os.path.dirname(save_filename):
                os.makedirs(os.path.dirname(save_filename), exist_ok=True)
            plt.savefig(save_filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {save_filename}")
        except Exception as e:
            print(f"Error saving figure to {save_filename}: {e}")
        plt.close(fig)
    else:
        plt.show()

#### Plot 2 Datasets 1 PCA(2D)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
from collections import defaultdict

def plot_pca_trajectories_two_datasets_2d(pca_object, data_to_transform_dict1, data_to_transform_dict2, testing_env_info, num_trials_to_plot=100, plot_title=None, save_filename=None, add_phase_markers=True, label1="Unlesioned", label2="Lesioned"):
    """
    Plots and compares the mean PC1 vs PC2 trajectories from two datasets.

    Line colour indicates the task condition, line style (solid/dashed) shows
    the dataset, and markers show key events like start, end, and phase
    transitions. The plot title is ignored for consistancy.
    """
    if not data_to_transform_dict1 and not data_to_transform_dict2:
        print("No trial data provided from either dataset for plotting.")
        return

    if pca_object.n_components < 2:
        print(f"Error: PCA object must be fit with n_components>=2, but has {pca_object.n_components}")
        return

    # ---- Style guide definitions ----
    STYLE_GUIDE_COLORS = {
        (0.0, 0.0):   {'color': '#68d3e8', 'label': 'S: 0.00, T: 0.00'},
        (3.14, 0.0):  {'color': '#e69010', 'label': 'S: 3.14, T: 0.00'},
        (0.0, 3.14):  {'color': '#6b08bd', 'label': 'S: 0.00, T: 3.14'},
        (3.14, 3.14): {'color': '#69420c', 'label': 'S: 3.14, T: 3.14'},
    }
    # solid for dataset 1, dashed for dataset 2
    DATASET_STYLES = [
        {'linestyle': '-', 'label': label1},
        {'linestyle': '--', 'label': label2}
    ]

    # ---- Setting up the plot ----
    fig, ax = plt.subplots(figsize=(12, 12))
    # add a faint grid and centre lines
    ax.grid(True, linestyle='--', color='lightgrey')
    ax.axhline(0, color='grey', linestyle='-', linewidth=0.8)
    ax.axvline(0, color='grey', linestyle='-', linewidth=0.8)
    # make sure the plot is square
    ax.set_aspect('equal', adjustable='box')

    datasets = [(data_to_transform_dict1, DATASET_STYLES[0]), (data_to_transform_dict2, DATASET_STYLES[1])]
    all_plotted_conditions = set()

    # ---- Plot the trajectories and markers ----
    for data_dict, style in datasets:
        if not data_dict:
            continue

        grouped_trajectories = defaultdict(list)
        limited_trials = dict(list(data_dict.items())[:num_trials_to_plot])

        for trial_dict in limited_trials.values():
            stim_condition = (round(trial_dict['sample_stim_value'], 2), round(trial_dict['test_stim_value'], 2))
            if stim_condition in STYLE_GUIDE_COLORS:
                activity = np.array(trial_dict['network_activity'])
                pc_trajectory = pca_object.transform(activity)[:, :2]
                grouped_trajectories[stim_condition].append(pc_trajectory)
                all_plotted_conditions.add(stim_condition)

        for condition, trajectories in grouped_trajectories.items():
            if trajectories:
                mean_trajectory = np.mean(np.array(trajectories), axis=0)
                colour = STYLE_GUIDE_COLORS[condition]['color']

                # plot the main trajectory
                ax.plot(mean_trajectory[:, 0], mean_trajectory[:, 1], color=colour, linestyle=style['linestyle'], linewidth=2.5)
                # add the start and end markers
                ax.plot(mean_trajectory[0, 0], mean_trajectory[0, 1], marker='s', color='k', markersize=8, linestyle='None')
                ax.plot(mean_trajectory[-1, 0], mean_trajectory[-1, 1], marker='X', color='k', markersize=10, mew=2, linestyle='None')

                # and the phase transition markers if requested
                if add_phase_markers:
                    for end_index in testing_env_info['trial_end_ind'].values():
                        marker_idx = end_index - 1
                        if marker_idx < len(mean_trajectory):
                            ax.plot(mean_trajectory[marker_idx, 0], mean_trajectory[marker_idx, 1], marker='*', color='gold', markersize=12, linestyle='None')

    # ---- Build the legend manually in three parts ----
    legend_handles = []
    # Part 1: handles for the line styles (datasets)
    legend_handles.append(Line2D([0], [0], color='k', lw=2, linestyle=DATASET_STYLES[0]['linestyle'], label=DATASET_STYLES[0]['label']))
    legend_handles.append(Line2D([0], [0], color='k', lw=2, linestyle=DATASET_STYLES[1]['linestyle'], label=DATASET_STYLES[1]['label']))
    legend_handles.append(Line2D([0], [0], color='w', label='')) # a spacer

    # Part 2: handles for the line colours (conditions)
    sorted_conditions = sorted(list(all_plotted_conditions))
    for condition in sorted_conditions:
        style = STYLE_GUIDE_COLORS[condition]
        legend_handles.append(Line2D([0], [0], color=style['color'], lw=2.5, label=style['label']))
    legend_handles.append(Line2D([0], [0], color='w', label='')) # another spacer

    # Part 3: handles for the different markers (start, end, etc.)
    legend_handles.append(Line2D([0], [0], marker='s', color='k', label='Start', linestyle='None', markersize=8))
    legend_handles.append(Line2D([0], [0], marker='X', color='k', label='End', linestyle='None', markersize=10, mew=2))
    if add_phase_markers and all_plotted_conditions:
        legend_handles.append(Line2D([0], [0], marker='*', color='gold', label='Phase Transition', linestyle='None', markersize=12))

    ax.legend(handles=legend_handles, title="Legend", loc='upper left', bbox_to_anchor=(1.02, 1))

    # ---- Final plot tweaks ----
    ax.set_xlabel('Principal Component 1', fontsize=14)
    ax.set_ylabel('Principal Component 2', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    fig.tight_layout(rect=[0, 0, 0.85, 1])

    # ---- Save or show the plot ----
    if save_filename:
        try:
            if os.path.dirname(save_filename) and not os.path.exists(os.path.dirname(save_filename)):
                os.makedirs(os.path.dirname(save_filename))
            plt.savefig(save_filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {save_filename}")
        except Exception as e:
            print(f"Error saving figure to {save_filename}: {e}")
        plt.close(fig)
    else:
        plt.show()

## UNIT SELECTIVITY

### Calc Selectivity (and Inds)

In [ ]:
def calculate_selectivity_indexes(task_condition_1_data_dict, task_condition_2_data_dict):
  """
  Calculates a selectivity index for each hidden unit between two task conditions.
  The selectivity index is based on the absolute difference in mean activity
  between the two conditions, normalised by the pooled standard deviation.
  This provides a measure of how strongly a unit's activity differentiates
  between the two conditions.

  Args:
      task_condition_1_data_dict :  keys are trial numbers, values are dicts containing 'network_activity' for condition 1.
      task_condition_2_data_dict : A dictionary with the same structure as task_condition_1_data_dict, for condition 2.

  Returns:
      A tuple containing:
          - abs_selectivity_for_cond1xcond2 : Absolute selectivity index for each unit, in the order of hidden units.
          - sorted_indexes : Indices of the hidden units sorted by their selectivity index in ascending order.
  """
  # Uses absolute selectivity
  mean_activity = []
  std_activity = []

  # Iterate through the two task condition data dictionaries
  for task_condition in [task_condition_1_data_dict, task_condition_2_data_dict]:
    # Concatenate network activity across all trials for the current condition
    task_condition_activity = np.concatenate([value['network_activity'] for value in task_condition.values()], axis=0)

    # Calculate the mean activity for each unit across all time steps and trials
    mean_activity.append(np.mean(task_condition_activity, axis=0))

    # Calculate the standard deviation of activity for each unit
    std_activity.append(np.std(task_condition_activity, axis=0))

  # Calculate the absolute difference between the mean activities of the two conditions
  abs_selectivity_for_cond1xcond2 = abs((mean_activity[0] - mean_activity[1]))

  # Normalise the absolute difference by the pooled standard deviation
  # Adding a small epsilon (1e-7) to the denominator to prevent division by zero
  abs_selectivity_for_cond1xcond2 /= np.sqrt((std_activity[0]**2 + std_activity[1]**2 + 1e-7)/2)

  # Get the indices of the units sorted by their selectivity in ascending order
  sorted_indexes = np.argsort(abs_selectivity_for_cond1xcond2)

  # Return the selectivity values and the sorted unit indexes
  return abs_selectivity_for_cond1xcond2, sorted_indexes

## NETWORK LEISONING !!!! NEEDS BIC RELATED MODIFICATIONS

In [ ]:
def lesion_network(trained_network, units_to_lesion):
  # May not work for all network configs (designed on LRNN NB 1 HL(LeakyRNN))
    try:
        # Create a deep copy of the network
        lesioned_network = deepcopy(trained_network)
        print("Created a deep copy of the network.")

        # Locate the h2h Linear layer #### !!!! ---- NEEDS BIC Related Modifications ---- (Layer Attributes and Effective Weights) ------ !!!!
        if hasattr(lesioned_network, 'rnn') and hasattr(lesioned_network.rnn, 'h2h'):
            h2h_layer = lesioned_network.rnn.h2h # Direct ref?
            print(f"Found h2h layer: {h2h_layer}")

            # Get the weight tensor
            h2h_weights = h2h_layer.weight.data.cpu().numpy()
            print(f"Original h2h weights shape: {h2h_weights.shape}")

            # Check if the provided unit indices are valid
            hidden_size = h2h_weights.shape[0] # Assuming output dim == input dim for recurrent weights
            valid_units_to_lesion = [u for u in units_to_lesion if 0 <= u < hidden_size]

            if len(valid_units_to_lesion) != len(units_to_lesion):
                print("Warning: Some provided unit indices were out of bounds.")
                print(f"Valid hidden unit range is [0, {hidden_size - 1}].")
                print(f"Lesioning units: {valid_units_to_lesion}")
            else:
                print(f"Lesioning recurrent connections for units: {valid_units_to_lesion}")


            # Set incoming and outgoing recurrent weights for the specified units to zero
            # Recurrent weights shape: (output_features, input_features) or (to_units, from_units)
            for unit_index in valid_units_to_lesion:
                # Set weights *to* this unit from all other units in the h2h layer
                h2h_weights[unit_index, :] = 0

                # Set weights *from* this unit to all other units in the h2h layer
                h2h_weights[:, unit_index] = 0
            print("Recurrent weights for specified units set to zero.")

            # Update the weight tensor in the lesioned network
            h2h_layer.weight.data = torch.from_numpy(h2h_weights).to(h2h_layer.weight.data.device) # Depends on if h2h_layer is direct ref or copy (wouldve seen probs in non BIC)


            return lesioned_network

        else:
            print("Error: The network structure is not as expected (missing .rnn.h2h).")
            return None

    except Exception as e:
        print(f"An error occurred during network lesioning: {e}")
        return None



## SELECTIVITY AND LEISONING FRAMEWORK

### Leisoned Network for n% most selective Units and Analysis

In [ ]:
def leison_network_for_top_n_and_test(trained_network, list_of_task_cond_1_cond_2_data_dicts_for_selectivity, testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = None, fig_file_name = 'leisoned_unleisoned' , file_ext='.png', unleisoned_PCA_2d = None, return_results = False):
  ## Note:
  # calculate_selectivity_indexes is expecting the dicts in list_of_task... to be <trial_num>: 'network_activity',... i.e. testing_trial_and_activity structured so use data func with return like input = false
  # Note : some things in this rough func wont work for a diff task or 2d nets

  # Produce Leisoned Network
  abs_selectivity_, sorted_indexes_ = calculate_selectivity_indexes(list_of_task_cond_1_cond_2_data_dicts_for_selectivity[0], list_of_task_cond_1_cond_2_data_dicts_for_selectivity[1])
  units_to_leison_ = sorted_indexes_[-int(len(sorted_indexes_)*n_perc_leison/100):]


  leisoned_network_ = lesion_network(trained_network, units_to_leison_)

  # Test uneleisoned network
  unleisoned_full_testing_data_ = evaluate_network_on_dataset(trained_network, testing_data_set, num_trials=2000)
  # Test Leisoned Network
  leisoned_full_testing_data_ = evaluate_network_on_dataset(leisoned_network_, testing_data_set, num_trials=2000)


  testing_env_info_ = unleisoned_full_testing_data_['testing_env_info']

  print(f"Unleisoned Network Performance: {unleisoned_full_testing_data_['testing_trial_performance'] * 100} %")

  print(f"Leisoned Network Performance: {leisoned_full_testing_data_['testing_trial_performance'] * 100} %")

  print(f"Performance Difference: {((unleisoned_full_testing_data_['testing_trial_performance'] - leisoned_full_testing_data_['testing_trial_performance'])/(unleisoned_full_testing_data_['testing_trial_performance'])) * 100} %")

  # Print a PCA Transformed Plot of the test activities for both networks


  if unleisoned_PCA_1d:
    one_d_pca_filename = fig_file_name + '_1d_pca' + file_ext
    plot_pca_trajectories_two_datasets(unleisoned_PCA_1d, unleisoned_full_testing_data_['testing_trial_and_activity'], leisoned_full_testing_data_['testing_trial_and_activity'], testing_env_info_, num_trials_to_plot=100, plot_title='(Un)Leisoned for HL Activity in PC1 across t', save_filename=one_d_pca_filename, label1="Unleisoned", label2="Leisoned")

  if unleisoned_PCA_2d:
    two_d_pca_filename = fig_file_name + '_2d_pca' + file_ext
    plot_pca_trajectories_two_datasets_2d(unleisoned_PCA_2d, unleisoned_full_testing_data_['testing_trial_and_activity'], leisoned_full_testing_data_['testing_trial_and_activity'], testing_env_info_, num_trials_to_plot=100, plot_title='(Un)Leisoned for HL Activity in PC1xPC2', save_filename=two_d_pca_filename, add_phase_markers=True, label1="Unleisoned", label2="Leisoned")

  all_results_ = {
      'abs_selectivity': abs_selectivity_,
      'sorted_indexes': sorted_indexes_,
      'units_to_leison': units_to_leison_,
      'unleisoned_full_testing_data': unleisoned_full_testing_data_,
      'leisoned_full_testing_data': leisoned_full_testing_data_
  }
  if return_results:
    return all_results_
  else:
    print('Done')



## SELECTIVITY AND LEISONING Changes for EI and BICS

### Selectivity - For BICS

In [ ]:
def calculate_selectivity_indexes_BIC(task_condition_1_data_dict, task_condition_2_data_dict):
  """
  Calculates a selectivity index for each unit between two task conditions.

  The index is just the difference in mean activity, normalised by the
  pooled standard deviation. Basically, it shows how much a unit's firing
  rate changes between the two conditions.

  Returns the raw selectivity values (directional), indexes sorted by
  the absolute selectivity, and indexes sorted within E/I populations.
  """
  mean_activity = []
  std_activity = []

  # go through both sets of data
  for task_condition in [task_condition_1_data_dict, task_condition_2_data_dict]:
    # first, stack all the trial activities into one big array
    task_condition_activity = np.concatenate([value['network_activity'] for value in task_condition.values()], axis=0)

    # get the mean activity for each unit across all time steps and trials
    mean_activity.append(np.mean(task_condition_activity, axis=0))

    # and the standard deviation of activity for each unit
    std_activity.append(np.std(task_condition_activity, axis=0))

  # now calculate the selectivity (the difference in means)
  selectivity_for_cond1xcond2 = (mean_activity[0] - mean_activity[1])

  # normalise it by the pooled standard deviation
  # add eps to the bottom to stop it dividing by zero
  selectivity_for_cond1xcond2 /= np.sqrt((std_activity[0]**2 + std_activity[1]**2 + 1e-7)/2)

  # get the unit indexes sorted by their absolute selectivity
  abs_sorted_indexes = np.argsort(abs(selectivity_for_cond1xcond2))

  # this bit sorts the indexes within the E and I populations
  E_Prop = 0.8
  e_size = int(E_Prop * len(abs_sorted_indexes))
  ei_sorted_indexes = (np.argsort(selectivity_for_cond1xcond2[:e_size]), np.argsort(selectivity_for_cond1xcond2[e_size:]+e_size))

  # returns raw selectivity (directional), absolute indexes for lesioning, and ei sorted indexes for other analysis (DOES THIS REGARDLESS OF EI OR NOT)
  return selectivity_for_cond1xcond2, abs_sorted_indexes, ei_sorted_indexes

### Leisoning for BICs

In [ ]:
def lesion_network_BIC(trained_network, units_to_lesion):
    """
    Takes a trained network and a list of units, and returns a new
    network with those units lesioned. It works by setting the incoming
    and outgoing recurrent weights for those units to zero.
    """
    # layer_to_leision=None,  this is fine for now. Only need to change when multi layer (use f string for has attr 'rnn')
    # Might not work for all network configs (designed on LRNN NB 1 HL(LeakyRNN))
    try:
        # make a deep copy of the network so we don't break the original
        lesioned_network = deepcopy(trained_network)
        print("Created a deep copy of the network.")

        # find the h2h Linear layer #### ----  doesnt acc need modification, all weights defined at same attribute location
        if hasattr(lesioned_network, 'rnn') and hasattr(lesioned_network.rnn, 'h2h'):
            # is this a direct ref?
            h2h_layer = lesioned_network.rnn.h2h
            print(f"Found h2h layer: {h2h_layer}")

            # get the weight tensor
            h2h_weights = h2h_layer.weight.data.cpu().numpy()
            print(f"Original h2h weights shape: {h2h_weights.shape}")

            # check the unit indexes are actually valid
            hidden_size = h2h_weights.shape[0] # assumes output dim == input dim for recurrent weights
            valid_units_to_lesion = [u for u in units_to_lesion if 0 <= u < hidden_size]

            if len(valid_units_to_lesion) != len(units_to_lesion):
                print("Warning: Some provided unit indices were out of bounds.")
                print(f"Valid hidden unit range is [0, {hidden_size - 1}].")
                print(f"Lesioning units: {valid_units_to_lesion}")
            else:
                print(f"Lesioning recurrent connections for units: {valid_units_to_lesion}")


            # set the incoming and outgoing recurrent weights for the units to zero
            # recurrent weights shape is (to_units, from_units)
            for unit_index in valid_units_to_lesion:
                # set weights *to* this unit from all other units
                h2h_weights[unit_index, :] = 0

                # set weights *from* this unit to all other units
                h2h_weights[:, unit_index] = 0
            print("Recurrent weights for specified units set to zero.")

            # update the weights in the new lesioned network
            # this depends on if h2h_layer is a direct ref or a copy (we would've seen problems in the non-BIC version if it was wrong)
            h2h_layer.weight.data = torch.from_numpy(h2h_weights).to(h2h_layer.weight.data.device)


            return lesioned_network

        else:
            print("Error: The network structure is not as expected (missing .rnn.h2h).")
            return None

    except Exception as e:
        print(f"An error occurred during network lesioning: {e}")
        return None

### Full Selectivity and Leisoning (Accomodate for BICs)

In [ ]:
def leison_network_for_top_n_and_test_BIC(trained_network, list_of_task_cond_1_cond_2_data_dicts_for_selectivity, testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = None, fig_file_name = 'leisoned_unleisoned' , file_ext='.png', unleisoned_PCA_2d = None, return_results = True):
  """
  A wrapper function to find the most selective units, lesion them,
  and then test the network's performance before and after. Also plots stuff.
  """
  # get selectivity and find the top n% of units to lesion
  selectivity_for_cond1xcond2_, abs_sorted_indexes_, ei_sorted_indexes_ = calculate_selectivity_indexes_BIC(list_of_task_cond_1_cond_2_data_dicts_for_selectivity[0], list_of_task_cond_1_cond_2_data_dicts_for_selectivity[1])
  units_to_leison_ = abs_sorted_indexes_[-int(len(abs_sorted_indexes_)*n_perc_leison/100):]

  # create the lesioned network
  leisoned_network_ = lesion_network_BIC(trained_network, units_to_leison_)

  # evaluate both networks on the test data
  unleisoned_full_testing_data_ = evaluate_network_on_dataset(trained_network, testing_data_set, num_trials=2000)
  leisoned_full_testing_data_ = evaluate_network_on_dataset(leisoned_network_, testing_data_set, num_trials=2000)


  testing_env_info_ = unleisoned_full_testing_data_['testing_env_info']

  # print out the results
  print(f"Unleisoned Network Performance: {unleisoned_full_testing_data_['testing_trial_performance'] * 100} %")
  print(f"Leisoned Network Performance: {leisoned_full_testing_data_['testing_trial_performance'] * 100} %")
  print(f"Performance Difference: {((unleisoned_full_testing_data_['testing_trial_performance'] - leisoned_full_testing_data_['testing_trial_performance'])/(unleisoned_full_testing_data_['testing_trial_performance'])) * 100} %")

  # plot PCA trajectories if a pca object was given
  if unleisoned_PCA_1d:
    one_d_pca_filename = fig_file_name + '_1d_pca' + file_ext
    plot_pca_trajectories_two_datasets(unleisoned_PCA_1d, unleisoned_full_testing_data_['testing_trial_and_activity'], leisoned_full_testing_data_['testing_trial_and_activity'], testing_env_info_, num_trials_to_plot=100, plot_title='(Un)Leisoned for HL Activity in PC1 across t', save_filename=one_d_pca_filename, label1="Unleisoned", label2="Leisoned")

  if unleisoned_PCA_2d:
    two_d_pca_filename = fig_file_name + '_2d_pca' + file_ext
    plot_pca_trajectories_two_datasets_2d(unleisoned_PCA_2d, unleisoned_full_testing_data_['testing_trial_and_activity'], leisoned_full_testing_data_['testing_trial_and_activity'], testing_env_info_, num_trials_to_plot=100, plot_title='(Un)Leisoned for HL Activity in PC1xPC2', save_filename=two_d_pca_filename, add_phase_markers=True, label1="Unleisoned", label2="Leisoned")

  # bundle up the results
  all_results_ = {
      'raw_selectivity': selectivity_for_cond1xcond2_,
      'abs_sorted_indexes': abs_sorted_indexes_,
      'ei_sorted_indexes': ei_sorted_indexes_,
      'units_to_leison': units_to_leison_,
      'unleisoned_full_testing_data': unleisoned_full_testing_data_,
      'leisoned_full_testing_data': leisoned_full_testing_data_
  }
  if return_results:
    return all_results_
  else:
    print('Done')

## FULL FPF FUCNTION LIBRARY

### Import code stub

In [ ]:
# WHEN RUNNING FPF NEED TO DO THIS FIRST (LUCKILY SEEMS TO WORK IN THIS NOTEBOOK UNLIKE DPCA)
# also need to add fixed point finder package folder from wd to notebook drive thing
# import sys
# sys.path.append('/fixed-point-finder-master')
# %cd fixed-point-finder-master
# from FixedPointFinderTorch import FixedPointFinderTorch as FixedPointFinder
# import torch

### TRIAL DATA EXTRACTION HELPER (NEW ONE)

In [ ]:
def extract_dict_of_trial_data_for_conds_and_hla_index(full_testing_data_dict, dict_of_conds, hla_index=None, return_like_full = False):
  """
  A really important helper function.
  It pulls out trial data for specific conditions and can also slice the
  network activity data to specific time periods (using an int, tuple, or task phase name).
  Replaces that other stupid data processing function I made.
  """
  # TODO: maybe move this to the top of the notebook and integrate better
  full_testing_data_dict_ = deepcopy(full_testing_data_dict)
  env_info = full_testing_data_dict_['testing_env_info'] # must be there
  trial_testing_data_dict_ = full_testing_data_dict_['testing_trial_and_activity'] # must be there

  # work out how to slice the data based on hla_index
  slicer = None
  if isinstance(hla_index, int):
    slicer = hla_index
  if isinstance(hla_index, tuple):
    if hla_index[0] < hla_index[1]:
      slicer = slice(hla_index[0], hla_index[1])
    else:
      slicer = slice(hla_index[1], hla_index[0])
  if isinstance(hla_index, str):
    if hla_index in env_info['trial_start_ind'] and hla_index in env_info['trial_end_ind']:
      slicer = slice(env_info['trial_start_ind'][hla_index], env_info['trial_end_ind'][hla_index])
  if isinstance(hla_index, list):
    items_in_list_in_env_info = [item for item in hla_index if item in env_info['trial_start_ind'] and item in env_info['trial_end_ind']]
    if items_in_list_in_env_info:
      slicer = slice(min([env_info['trial_start_ind'][i] for i in hla_index]), max([env_info['trial_end_ind'][i] for i in hla_index]))

  # filter the dictionary based on the conditions
  cond_filtered_dict = {}
  for trial_id, trial_data in trial_testing_data_dict_.items():
    if all([trial_data[f'{cond_key}'] == cond_value for cond_key, cond_value in dict_of_conds.items()]):
      if slicer:
        trial_data['network_activity'] = trial_data['network_activity'][slicer]

      cond_filtered_dict[trial_id] = trial_data

  # adding this in so i can reuse output of just conditional to perform the index slicing too!
  if return_like_full:
    return {'testing_trial_and_activity': cond_filtered_dict, 'testing_env_info': env_info}
  else:
    return cond_filtered_dict

In [ ]:
def extract_dict_of_trial_data_for_conds_and_hla_index_BICFPF(full_testing_data_dict, dict_of_conds, hla_index=None, return_like_full = False):
  """
  The new version of the really important helper function.

  Filters trials based on conditions and slices the activity data to specific
  time periods. This one also handles slicing both 'network_activity' and
  'network_hidden_state' if it exists.
  """
  # TODO: maybe move this to the top of the notebook and integrate better
  full_testing_data_dict_ = deepcopy(full_testing_data_dict)
  env_info = full_testing_data_dict_['testing_env_info'] # must be there
  trial_testing_data_dict_ = full_testing_data_dict_['testing_trial_and_activity'] # must be there

  # work out how to slice the data based on hla_index
  slicer = None
  if isinstance(hla_index, int):
    slicer = hla_index
  if isinstance(hla_index, tuple):
    if hla_index[0] < hla_index[1]:
      slicer = slice(hla_index[0], hla_index[1])
    else:
      slicer = slice(hla_index[1], hla_index[0])
  if isinstance(hla_index, str):
    if hla_index in env_info['trial_start_ind'] and hla_index in env_info['trial_end_ind']:
      slicer = slice(env_info['trial_start_ind'][hla_index], env_info['trial_end_ind'][hla_index])
  if isinstance(hla_index, list):
    items_in_list_in_env_info = [item for item in hla_index if item in env_info['trial_start_ind'] and item in env_info['trial_end_ind']]
    if items_in_list_in_env_info:
      slicer = slice(min([env_info['trial_start_ind'][i] for i in hla_index]), max([env_info['trial_end_ind'][i] for i in hla_index]))

  # filter the dictionary based on the conditions
  cond_filtered_dict = {}
  for trial_id, trial_data in trial_testing_data_dict_.items():
    if all([trial_data[f'{cond_key}'] == cond_value for cond_key, cond_value in dict_of_conds.items()]):
      # if we're slicing, apply it to both activity and hidden_state if they exist
      if slicer:
        trial_data['network_activity'] = trial_data['network_activity'][slicer]
        network_hidden_state_ = trial_data.get('network_hidden_state', None)
        if network_hidden_state_ is not None:
          trial_data['network_hidden_state'] = network_hidden_state_[slicer]

      cond_filtered_dict[trial_id] = trial_data

  # adding this in so i can reuse output of just conditional to perform the index slicing too!
  if return_like_full:
    return {'testing_trial_and_activity': cond_filtered_dict, 'testing_env_info': env_info}
  else:
    return cond_filtered_dict

### FPF Specific Helper Funcs

In [ ]:
import numpy as np

def get_random_samples_with_opt_noise(list_of_arrays, num_samples, std_pc=0.0, random=False, random_range_modifier = 0.0, protected_indexes=None):
    """
    Gets samples from a list of arrays. Has two main modes.
    1. random=True: creates brand new random samples within the range of the original data.
    2. random=False: samples from the original data (with replacement) and can add
       some gaussian noise if you want.
    """
    # make sure we're working with a numpy array
    if not isinstance(list_of_arrays, np.ndarray):
        list_of_arrays_np = np.array(list_of_arrays)
    else:
        list_of_arrays_np = list_of_arrays

    # get the number of dimensions
    n_dim = list_of_arrays_np.shape[-1] if list_of_arrays_np.ndim > 1 else list_of_arrays_np.shape[0] # Check this (alt notebook now?)

    # if random=True, make completely new samples
    if random:
        # find the min/max for each unit to create a plausible range
        min_per_unit = np.min(list_of_arrays_np, axis=0) * (1.0 - random_range_modifier)
        max_per_unit = np.max(list_of_arrays_np, axis=0) * (1.0 + random_range_modifier)

        # now create the random arrays
        # np.random.uniform(low, high, size)
        sampled_arrays = np.random.uniform(low=min_per_unit,
                                           high=max_per_unit,
                                           size=(num_samples, n_dim))
        return sampled_arrays

    # otherwise, sample from the existing arrays
    # samples with replacement
    random_indices = np.random.choice(len(list_of_arrays_np), size=num_samples)
    sampled_arrays = list_of_arrays_np[random_indices]

    # if we want noise, add it now
    if std_pc > 0.0:
        # calculate noise based on a percentage of each unit's std dev
        std_dev_per_unit = np.std(list_of_arrays_np, axis=0)
        noise_scale_per_unit = std_pc * std_dev_per_unit
        noise = np.random.randn(num_samples, n_dim) * noise_scale_per_unit

        # don't add noise to any protected units
        if protected_indexes is not None:
            noise[:, protected_indexes] = 0.0
        sampled_arrays += noise
        return sampled_arrays

    # if no noise, just return the samples as they are
    return sampled_arrays

### Input and IC Data Construction Func

In [ ]:
import numpy as np

def obtain_data_for_fpf_setup_bic(filtered_trial_data_dict, num_init_conds, noise_std_or_random_range_modifier = 0.0, random_ics = False, noise_protected_indexes_hla = None, list_of_input_arrays = None, hidden_key='network_activity'):
    """
    Prepares the initial conditions for the fixed point finder.

    It creates a set of initial hidden layer activities and corresponding inputs
    based on the provided trial data.
    """
    if list_of_input_arrays is None:
        # default inputs if none are provided (what shape does this go to?)
        list_of_input_arrays = [np.array([0.0, 0.0, 1.0]),np.array([0.0,1.0,0.0]),np.array([1.0,0.0,0.0])]

    list_of_hidden_activity = [trial_data[f'{hidden_key}'] for trial_data in filtered_trial_data_dict.values()]
    list_of_hidden_activity_proc = []

    # flatten the list of activities into a list of 1d arrays
    for hla_arr in list_of_hidden_activity:
        if hla_arr.ndim == 2:
            list_of_hidden_activity_proc.extend([arr for arr in hla_arr])
        else:
            list_of_hidden_activity_proc.append(hla_arr)

    list_of_hidden_activity_proc = np.array(list_of_hidden_activity_proc)

    if random_ics:
        init_hla_cond_array = get_random_samples_with_opt_noise(list_of_hidden_activity_proc, num_init_conds, std_pc=0.0, random=True, random_range_modifier = noise_std_or_random_range_modifier, protected_indexes=None)
    else:
        init_hla_cond_array = get_random_samples_with_opt_noise(list_of_hidden_activity_proc, num_init_conds, std_pc=noise_std_or_random_range_modifier, random=False, random_range_modifier = 0.0, protected_indexes=noise_protected_indexes_hla)


    # make sure we have the same number of inputs as initial conditions
    input_arrays = get_random_samples_with_opt_noise(list_of_input_arrays, num_init_conds)

    return init_hla_cond_array, input_arrays

### Fixed Point Finding

In [ ]:
def perform_fixed_point_finding(rnn_layer, initial_hla_array, fixed_input_array, n_iters_fpf = 20000):
  """
  A wrapper for the FixedPointFinder. It takes an RNN layer,
  a set of initial hidden activities, and a fixed input, then
  goes and finds the fixed points.
  """
  fp_finder = None
  fp_finder = FixedPointFinder(rnn_layer, max_iters = n_iters_fpf)

  # convert to numpy just in case (shouldnt be needed though)
  if torch.is_tensor(initial_hla_array):
    initial_hla_array = initial_hla_array.cpu().numpy()
  if torch.is_tensor(fixed_input_array):
    fixed_input_array = fixed_input_array.cpu().numpy()

  if fixed_input_array.shape[0] != initial_hla_array.shape[0] and fixed_input_array.shape[0] != 1:
    print('Fixed input should either be 1 vector (1,n_input_dim) shaped, or have same number of "rows" as initial conditions')
    # not sure how fpf handles this, might need to raise an error here later
    print('Expect error but continuing.')


  unique_fps, all_fps = fp_finder.find_fixed_points(initial_hla_array, fixed_input_array)
  fp_finder = None # clean up
  # can change to return all_fps if needed, hasn't been so far
  print(unique_fps.xstar.shape)
  print(all_fps.xstar.shape)
  return unique_fps

### Fixed Point PCA Plotter

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
from collections import defaultdict

def plot_pca_trajectories_with_fps_BICS(
    fixed_points_obj,   # A single FixedPoints object
    pca_object,         # fitted PCA object (assumes n_components>=2)
    trial_data_dict=None,# pre-filtered dict of trial data for the specific scenario
    trial_env_info=None, # env info (dt, trial_start_ind, trial_end_ind)
    task_phase_str=None, # e.g., 'delay', 'sample' - corresponds to the scenario's phase
    plot_prev=True,      # If True, plot pre-phase activity in faint black
    plot_title=None,     # Optional plot title (ignored)
    filename=None,       # Optional filename to save (triggers save and close)
    max_trials_to_plot=100, # Max number of trial trajectories to plot
    fixed_axis_limits=None, # (xmin, xmax, ymin, ymax) tuple for fixed axis limits
    activate_fps = False # set dep. on if using activity or state
    ):
    """
    This is the big one. It plots the fixed points and the average 2D PCA
    trajectories for a specific task phase.

    It can show the activity from before the main phase in faint black and
    has a pretty complex legend for all the different elements.
    """
    # ---- Style guide definitions ----
    STYLE_GUIDE_COLORS = {
        (0.0, 0.0):   {'color': '#68d3e8', 'label': 'S: 0.00, T: 0.00'},
        (3.14, 0.0):  {'color': '#e69010', 'label': 'S: 3.14, T: 0.00'},
        (0.0, 3.14):  {'color': '#6b08bd', 'label': 'S: 0.00, T: 3.14'},
        (3.14, 3.14): {'color': '#69420c', 'label': 'S: 3.14, T: 3.14'},
    }
    STYLE_GUIDE_MARKERS = {
        'stable_fp':    {'marker': 'D', 'color': 'g', 'label': 'Stable Fixed Point', 'size': 120},
        'unstable_fp':  {'marker': 'o', 'color': 'r', 'label': 'Unstable Fixed Point', 'size': 120},
        'start':        {'marker': 's', 'color': 'k', 'label': 'Start', 'size': 64},
        'phase_trans':  {'marker': '*', 'color': 'gold', 'label': 'Phase Transition', 'size': 144}
    }

    # ---- Plotting Setup ----
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.grid(True, linestyle='--', color='lightgrey')
    ax.axhline(0, color='grey', linestyle='-', linewidth=0.8)
    ax.axvline(0, color='grey', linestyle='-', linewidth=0.8)
    ax.set_aspect('equal', adjustable='box')

    legend_handles = []
    plotted_fp_types = set()
    plotted_conditions = set()

    # ---- 1. Plot the fixed points ----
    if fixed_points_obj:
        fp_activity = fixed_points_obj.xstar
        if activate_fps:
             # Assumes ReLU
            fp_activity = np.maximum(0, fp_activity)

        transformed_fps = pca_object.transform(fp_activity)[:, :2]

        for i in range(len(transformed_fps)):
            is_stable = fixed_points_obj.is_stable[i]
            style_key = 'stable_fp' if is_stable else 'unstable_fp'
            style = STYLE_GUIDE_MARKERS[style_key]

            ax.scatter(transformed_fps[i, 0], transformed_fps[i, 1],
                       marker=style['marker'], facecolor=style['color'], s=style['size'],
                       edgecolor='k', linewidth=1.0, zorder=10)
            plotted_fp_types.add(style_key)

    # ---- 2. Plot the trial trajectories ----
    if trial_data_dict and trial_env_info and task_phase_str:
        phase_start_idx = trial_env_info['trial_start_ind'].get(task_phase_str)
        phase_end_idx = trial_env_info['trial_end_ind'].get(task_phase_str)
        phase_order = list(trial_env_info.get('trial_start_ind', {}).keys())

        if phase_start_idx is not None and phase_end_idx is not None:
            grouped_trajectories = defaultdict(list)
            limited_trials = dict(list(trial_data_dict.items())[:max_trials_to_plot])

            for trial_dict in limited_trials.values():
                stim_condition = (round(trial_dict['sample_stim_value'], 2), round(trial_dict['test_stim_value'], 2))
                if stim_condition in STYLE_GUIDE_COLORS:
                    activity = np.array(trial_dict['network_activity'])
                    grouped_trajectories[stim_condition].append(pca_object.transform(activity)[:, :2])

            for condition, trajectories in grouped_trajectories.items():
                if not trajectories: continue
                plotted_conditions.add(condition)
                mean_traj = np.mean(np.array(trajectories), axis=0)

                # plot the bit before this phase in faint black
                if plot_prev and phase_start_idx > 0:
                    pre_phase_seg = mean_traj[0:phase_start_idx]
                    ax.plot(pre_phase_seg[:, 0], pre_phase_seg[:, 1], color='k', lw=1.0, alpha=0.8)

                # plot the main phase bit in its proper colour
                in_phase_seg = mean_traj[phase_start_idx-1:phase_end_idx]
                ax.plot(in_phase_seg[:, 0], in_phase_seg[:, 1], color=STYLE_GUIDE_COLORS[condition]['color'], lw=2.5)

                # add the start marker (always at timepoint 0)
                start_style = STYLE_GUIDE_MARKERS['start']
                ax.plot(mean_traj[0, 0], mean_traj[0, 1], marker=start_style['marker'], color=start_style['color'],
                        markersize=np.sqrt(start_style['size']), linestyle='None', zorder=5)

                # add markers for phase transitions
                indices_to_mark = []
                if task_phase_str in phase_order:
                    current_phase_position = phase_order.index(task_phase_str)
                    indices_to_mark.append(phase_end_idx - 1) # end of current phase

                    if plot_prev:
                        # ends of all previous phases
                        for i in range(current_phase_position):
                            prev_end_idx = trial_env_info['trial_end_ind'].get(phase_order[i])
                            if prev_end_idx:
                                indices_to_mark.append(prev_end_idx - 1)

                phase_trans_style = STYLE_GUIDE_MARKERS['phase_trans']
                # Use set to plot each unique index only once
                for end_idx in set(filter(None, indices_to_mark)):
                    if end_idx >= 0 and end_idx < len(mean_traj):
                        ax.plot(mean_traj[end_idx, 0], mean_traj[end_idx, 1],
                                marker=phase_trans_style['marker'],
                                color=phase_trans_style['color'],
                                markersize=np.sqrt(phase_trans_style['size']),
                                linestyle='None', zorder=5)

    # ---- 3. Build the fancy legend ----
    # fixed point markers
    for fp_type in sorted(list(plotted_fp_types)):
        style = STYLE_GUIDE_MARKERS[fp_type]
        legend_handles.append(Line2D([0], [0], marker=style['marker'], color='w', label=style['label'],
                                     markerfacecolor=style['color'], markeredgecolor='k', markersize=np.sqrt(style['size'])))
    if plotted_fp_types: legend_handles.append(Line2D([0], [0], color='w', label=''))

    # trial condition lines
    for condition in sorted(list(plotted_conditions)):
        style = STYLE_GUIDE_COLORS[condition]
        legend_handles.append(Line2D([0], [0], color=style['color'], lw=2.5, label=style['label']))
    if plotted_conditions: legend_handles.append(Line2D([0], [0], color='w', label=''))

    # other markers (start, phase transition)
    if plotted_conditions:
        start_style = STYLE_GUIDE_MARKERS['start']
        legend_handles.append(Line2D([0], [0], marker=start_style['marker'], color='w', label=start_style['label'],
                                     markerfacecolor=start_style['color'], markersize=np.sqrt(start_style['size'])))
        trans_style = STYLE_GUIDE_MARKERS['phase_trans']
        legend_handles.append(Line2D([0], [0], marker=trans_style['marker'], color='w', label=trans_style['label'],
                                     markerfacecolor=trans_style['color'], markersize=np.sqrt(trans_style['size'])))

    ax.legend(handles=legend_handles, title="Legend", loc='upper left', bbox_to_anchor=(1.02, 1))

    # ---- 4. Final plot tweaks ----
    if fixed_axis_limits:
        ax.set_xlim(fixed_axis_limits[0], fixed_axis_limits[1])
        ax.set_ylim(fixed_axis_limits[2], fixed_axis_limits[3])

    ax.set_xlabel('Principal Component 1', fontsize=14)
    ax.set_ylabel('Principal Component 2', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    fig.tight_layout(rect=[0, 0, 0.85, 1])

    # ---- 5. Save or show the plot ----
    if filename:
        try:
            if os.path.dirname(filename) and not os.path.exists(os.path.dirname(filename)):
                os.makedirs(os.path.dirname(filename))
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {filename}")
        except Exception as e:
            print(f"Error saving figure to {filename}: {e}")
        plt.close(fig)
    else:
        plt.show()

### Function to run full FPF (based on our implementation)

In [ ]:
def run_fpf_2d_pca_plot_BIC(rnn_for_fpf, testing_trial_data_for_fpa, testing_trial_env_for_fpa, pca_object_for_plotting = None, ic_period = 'sample', fixed_input_array_ = [[1.0,0.0,1.0]], trial_cond_for_plot = {'sample_stim_value': 0.0}, use_random_ics=False, use_noisey_ics = False, filename_for_plot = 'figure.png', title_for_plot = 'Sample Period 0.0 Stimulus Input Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100, hidden_key='network_hidden_state'):
    """
    A high-level wrapper to run the whole analysis pipeline:
    1. Extracts data for a specific task period.
    2. Generates initial conditions (straight, random, or with noise).
    3. Runs the fixed-point finder.
    4. Plots the results (trajectories and fixed points) on a 2D PCA plot.
    """
    # if using hidden state for FPF (AND HIDDEN ACTIVITY FOR REST OF ANALYSIS), need to activate the FPs (e.g. ReLU) before plotting
    _activate_fps = True if hidden_key == 'network_hidden_state' else False

    # get the data from the very first timestep of the specified period to generate ICs from
    period_i_data_test_period = extract_dict_of_trial_data_for_conds_and_hla_index_BICFPF(testing_trial_data_for_fpa, {'network_correct': True}, return_like_full=True, hla_index= ic_period)
    period_i_0_trial_data = extract_dict_of_trial_data_for_conds_and_hla_index_BICFPF(period_i_data_test_period, {'network_correct': True}, return_like_full=False, hla_index=0)

    # 1. Generate the initial conditions for the FPF
    list_of_ic_arrs = []
    # always include the straight samples
    ic_arr_0, _ = obtain_data_for_fpf_setup_bic(period_i_0_trial_data, num_init_conds=1000, noise_std_or_random_range_modifier = 0.0, random_ics = False, noise_protected_indexes_hla = None, list_of_input_arrays = None, hidden_key=hidden_key)
    list_of_ic_arrs.append(ic_arr_0)
    # optionally add random and noisy ICs
    if use_random_ics:
        ic_arr_1, _ = obtain_data_for_fpf_setup_bic(period_i_0_trial_data, num_init_conds = 100, noise_std_or_random_range_modifier = 0.1, random_ics = True, noise_protected_indexes_hla = None, list_of_input_arrays = None, hidden_key=hidden_key)
        list_of_ic_arrs.append(ic_arr_1)
    if use_noisey_ics:
        ic_arr_2, _ = obtain_data_for_fpf_setup_bic(period_i_0_trial_data, num_init_conds = 100, noise_std_or_random_range_modifier = 0.1, random_ics = False, noise_protected_indexes_hla = None, list_of_input_arrays = None, hidden_key=hidden_key)
        list_of_ic_arrs.append(ic_arr_2)
    period_i_0_init_conds = np.concatenate(list_of_ic_arrs, axis=0)

    # 2. Run the fixed-point finder
    unique_fps_period_i_input_j = perform_fixed_point_finding(rnn_layer=rnn_for_fpf, initial_hla_array=period_i_0_init_conds, fixed_input_array=fixed_input_array_)

    # 3. Get the trial data for plotting the trajectories
    period_cond_trial_data = extract_dict_of_trial_data_for_conds_and_hla_index(testing_trial_data_for_fpa, trial_cond_for_plot, return_like_full=False)

    # 4. Make the final plot
    plot_pca_trajectories_with_fps_BICS(
      fixed_points_obj = unique_fps_period_i_input_j,
      pca_object = pca_object_for_plotting,
      trial_data_dict=period_cond_trial_data,
      trial_env_info=testing_trial_env_for_fpa,
      task_phase_str= ic_period,
      plot_prev=plot_prev,
      plot_title=title_for_plot,
      filename=filename_for_plot,
      max_trials_to_plot=max_trials_to_plot,
      activate_fps=_activate_fps
      )

# --- Trial Run with 'Functionised' Notebook ---


# DEFINING BIC NETWORKS (LAST UPDATE : 1507_4:07pm)

## LeakyRNN Layer - No Self Connections, State,Output Separation

In [ ]:
# Leaky RNN with No Self connections
from torch.nn import functional as F

class LeakyRNN_NSC_MS(nn.Module):
    """Leaky RNN No Self Connections and Maintained Unit State

    Parameters:
        input_size: Number of input neurons
        hidden_size: Number of hidden neurons
        dt: discretization time step in ms.
            If None, dt equals time constant tau

    Inputs:
        input: tensor of shape (seq_len, batch, input_size)
        hidden: tensor of shape (batch, hidden_size), initial hidden activity
            if None, hidden is initialised through self.init_hidden()

    Outputs:
        output: tensor of shape (seq_len, batch, hidden_size)
        hidden: tensor of shape (batch, hidden_size), final hidden activity
    """

    def __init__(self, input_size, hidden_size, dt, tau=100, **kwargs): # dt is now required
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.tau = tau
        self.alpha = dt / self.tau # Alpha is always dt/tau
        self.batch_first = False # For fixed point finder
        # Check for stability (/biological plausibility)
        if self.alpha > 1.0:
            print(f"Warning: dt ({dt}) is greater than tau ({tau}). Alpha ({self.alpha:.2f}) > 1.0. This can lead to numerical instability.")

        # self.nsc_mask = self.create_no_self_conn_mask()
        mask = torch.ones(self.hidden_size, self.hidden_size) - torch.eye(self.hidden_size, self.hidden_size)
        self.register_buffer('nsc_mask', mask)
        self.input2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.reset_parameters()


    def reset_parameters(self):
      # Initialise input-to-hidden layer with Kaiming Uniform
      init.kaiming_uniform_(self.input2h.weight, a=math.sqrt(5))
      if self.input2h.bias is not None:
          fan_in, _ = init._calculate_fan_in_and_fan_out(self.input2h.weight)
          bound = 1 / math.sqrt(fan_in)
          init.uniform_(self.input2h.bias, -bound, bound)

      # Initialise hidden-to-hidden layer with Orthogonal
      init.orthogonal_(self.h2h.weight)
      if self.h2h.bias is not None:
          fan_in, _ = init._calculate_fan_in_and_fan_out(self.h2h.weight)
          bound = 1 / math.sqrt(fan_in)
          init.uniform_(self.h2h.bias, -bound, bound)


    def init_hidden(self, input):
      batch_size = input.shape[1]
      return (torch.zeros(batch_size, self.hidden_size).to(input.device), torch.zeros(batch_size, self.hidden_size).to(input.device))



    def recurrence(self, input, hidden):
        """Run network for one time step. (Now using a maintained internal state)

        Inputs:
            input: tensor of shape (batch, input_size)
            hidden: tensor of shape (batch, hidden_size)

        Outputs:
            h_new: tensor of shape (batch, hidden_size),
                network activity at the next time step
        """

        state, output = hidden

        effective_h2h_weights = self.h2h.weight * self.nsc_mask

        recurrent_component = F.linear(output, effective_h2h_weights, self.h2h.bias)

        total_input = self.input2h(input) + recurrent_component
        state = state * (1 - self.alpha) + total_input * self.alpha
        output = torch.relu(state)
        return state, output

    def forward(self, input, hidden=None):
        """Propogate input through the network."""

        # If hidden activity is not provided, initialise it
        if hidden is None:
            hidden = self.init_hidden(input)

        # Loop through time
        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden = self.recurrence(input[i], hidden)
            output.append(hidden[1])

        # Stack together output from all time steps
        output = torch.stack(output, dim=0)  # (seq_len, batch, hidden_size)


        return output, hidden # Note, hidden is now tuple with: (final state, final output) which are (shape?) (previously it was just (final output))

    def recurrence_helper_fpf(self, input, hidden_state_only):
      state, output = hidden_state_only, torch.relu(hidden_state_only)
      effective_h2h_weights = self.h2h.weight * self.nsc_mask
      recurrent_component = F.linear(output, effective_h2h_weights, self.h2h.bias)
      total_input = self.input2h(input) + recurrent_component
      state = state * (1 - self.alpha) + total_input * self.alpha
      return state


    def forward_helper_fpf(self, input, hidden_state_only=None):
      if hidden_state_only is None:
          hidden_state_only = self.init_hidden(input)[1]
      elif isinstance(hidden_state_only, tuple):
          hidden_state_only = hidden_state_only[1]

      output = []
      steps = range(input.size(0))
      for i in steps:
          hidden_state_only = self.recurrence_helper_fpf(input[i], hidden_state_only)
          hidden_output = torch.relu(hidden_state_only)
          output.append(hidden_output)
      output = torch.stack(output, dim=0)
      return output, hidden_state_only

    def forward_helper_fpf_ICs(self, input, hidden_state_only=None):
        """Propogate input through the network."""
        if hidden_state_only is None:
            hidden_state_only = self.init_hidden(input)[0]
        output = []
        steps = range(input.size(0))
        hidden_state_only_list = []
        # hidden_state_only_list.append(hidden_state_only)
        for i in steps:
            hidden_state_only = self.recurrence_helper_fpf(input[i], hidden_state_only)
            hidden_state_only_list.append(hidden_state_only)
            hidden_output = torch.relu(hidden_state_only)
            output.append(hidden_output)
        output = torch.stack(output, dim=0)
        hidden_state_only_tensor = torch.stack(hidden_state_only_list, dim=0)
        return output, hidden_state_only_tensor # hidden_state_only_tensor is shaped (seq_len, batch_size, hidden_size) and contains the raw states, we want to track this in FPF testing to obtain ICs


## LRNN-NSC : Main Network (s/o sep)

In [ ]:
class RNNNet_MS(nn.Module):
    """Full Network with a Leaky Recurrent Layer. That uses the NSC and Maintained State version of the LRNN Layer

    Parameters:
        input_size: int, input size
        hidden_size: int, hidden size
        output_size: int, output size

    Inputs:
        x: tensor of shape (Seq Len, Batch, Input size)

    Outputs:
        out: tensor of shape (Seq Len, Batch, Output size)
        rnn_output: tensor of shape (Seq Len, Batch, Hidden size)
    """
    def __init__(self, input_size, hidden_size, output_size, **kwargs):
        super().__init__()
        self.num_layers = 1
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        # Leaky RNN
        self.rnn = LeakyRNN_NSC_MS(input_size, hidden_size, **kwargs)
        # Add an output layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.all_layers = [self.fc]
        self.reset_parameters()


    def reset_parameters(self):
      for layer in self.all_layers:
        if isinstance(layer, nn.Linear):
          init.kaiming_uniform_(layer.weight, a=math.sqrt(5))
        if layer.bias is not None:
          fan_in, _ = init._calculate_fan_in_and_fan_out(layer.weight)
          bound = 1 / math.sqrt(fan_in)
          init.uniform_(layer.bias, -bound, bound)

    def forward(self, x):
        rnn_output, _ = self.rnn(x)
        out = self.fc(rnn_output)
        return out, rnn_output # So this is actually unchanged except for the self.rnn = LeakyRNN_NSC_MS(..)


    def forward_for_fpf_ics(self, x):
      rnn_outputs, hidden_state_tensor = self.rnn.forward_helper_fpf_ICs(x)
      out = self.fc(rnn_outputs)
      return out, rnn_outputs, hidden_state_tensor

## Spatial Embedding with WD Penalty

### 3D Approach - WD Regulariser Module (used in training with other network)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial
class Reg_WD(torch.nn.Module):

    """A regulariser for spatially embedded RNNs.
  Applies L1 regularisation to recurrent kernel of
  RNN which is weighted by the distance of units
  in predefined 3D space.
  Calculation:
      reg_WD * sum[distance_matrix o recurrent_kernel]
  Attributes:
      reg_WD: Float; Weighting of Reg_WD regularisation term.
      network_structure: Defines a 3D grid specifying the
      dimensions of a 3D space where neurons are placed.
      The tuple specifies the range of coordinates along
      each of the three axes in this 3D space.
  """

    def __init__(self, reg_WD=0.01, neuron_num=64, network_structure=(4,4,4), coordinates_list=None, distance_power=1, distance_metric='euclidean'):
        super(Reg_WD, self).__init__()

        self.distance_power = distance_power
        self.reg_WD = torch.tensor([reg_WD], dtype=torch.float32)

        # Set up tensor with distance matrix
        nx = np.arange(network_structure[0])
        ny = np.arange(network_structure[1])
        nz = np.arange(network_structure[2])

        # Set up coordinate grid
        x, y, z = np.meshgrid(nx, ny, nz)
        self.coordinates = [x.ravel(), y.ravel(), z.ravel()]

        # Override coordinate grid if provided in init
        if coordinates_list is not None:
            self.coordinates = coordinates_list

        # Check neuron number / number of coordinates
        if (len(self.coordinates[0]) == neuron_num) and (len(self.coordinates[1]) == neuron_num) and (len(self.coordinates[2]) == neuron_num):
            pass
        else:
            raise ValueError('Network / coordinate structure does not match the number of neurons.')

        # Calculate the euclidean distance matrix
        euclidean_vector = scipy.spatial.distance.pdist(np.transpose(self.coordinates), metric=distance_metric)
        euclidean = scipy.spatial.distance.squareform(euclidean_vector ** self.distance_power)
        self.distance_matrix = torch.tensor(euclidean, dtype=torch.float32)

    def forward(self, net):
        eff_weight_matrix = net.rnn.h2h.weight * net.rnn.nsc_mask
        abs_weight_matrix = torch.abs(eff_weight_matrix)
        WD_loss = self.reg_WD * torch.sum(abs_weight_matrix * self.distance_matrix)
        return WD_loss

    def _check_penalty_number(self, x):
        if not isinstance(x, (float, int)):
            raise ValueError(('Value: {} is not a valid regularization penalty number, '
                              'expected an int or float value').format(x))

    def visualise_distance_matrix(self):
        plt.imshow(self.distance_matrix.numpy())
        plt.colorbar()
        plt.show()

    def visualise_neuron_structure(self):
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(self.coordinates[0], self.coordinates[1], self.coordinates[2], c='b', marker='.')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        plt.show()

    def get_config(self):
        return {'reg_WD': float(self.reg_WD)}


### 2D Approach - SE_RNNNet (a Main Network)

In [ ]:
class SE_RNNNet(nn.Module):
    """Full Network with a Leaky Recurrent Layer. (ALSO UPDATED FOR Maintained State Version)

    Parameters:
        input_size: int, input size
        hidden_size: int, hidden size
        output_size: int, output size

    Inputs:
        x: tensor of shape (Seq Len, Batch, Input size)

    Outputs:
        out: tensor of shape (Seq Len, Batch, Output size)
        rnn_output: tensor of shape (Seq Len, Batch, Hidden size)
    """
    def __init__(self, input_size, hidden_size, output_size, spatial_embedding_dim = 2 , **kwargs):
        super().__init__()
        self.num_layers = 1
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.ndim = spatial_embedding_dim
        self.spatial_embedding_arrangement = self.spatial_embedding() # Renamed to avoid conflict with method name
        self.distance_matrix_computed = self.distance_matrix() # Renamed to avoid conflict with method name
        # Leaky RNN
        # self.rnn = LeakyRNN(input_size, hidden_size, **kwargs)
        self.rnn = LeakyRNN_NSC_MS(input_size, hidden_size, **kwargs)
        # Add an output layer
        self.fc = nn.Linear(hidden_size, output_size)



    def spatial_embedding(self):
      len_dimensions = int(self.hidden_size ** (1/self.ndim)) # Convert to integer
      print(len_dimensions)
      spatial_embedding_arrangement = np.zeros((len_dimensions, len_dimensions))
      _unit = 0
      for i in range(len_dimensions):
        for j in range(len_dimensions):
          spatial_embedding_arrangement[i,j] = _unit
          _unit += 1
      return spatial_embedding_arrangement

    def distance_matrix(self):

      len_dimensions = int(self.hidden_size ** (1/self.ndim)) # Convert to integer
      spatial_embedding_arrangement = np.zeros((len_dimensions, len_dimensions))
      _unit = 0
      for i in range(len_dimensions):
        for j in range(len_dimensions):
          spatial_embedding_arrangement[i,j] = _unit
          _unit += 1

      distance_matrix = np.zeros((self.hidden_size, self.hidden_size))
      # distance matrix i_j = distance from i to j in the spatial embedding
      for i in range(self.hidden_size):
        [unit_i_location_x,unit_i_location_y] = np.where(spatial_embedding_arrangement == i)
        unit_i_location = np.array([unit_i_location_x,unit_i_location_y])
        for j in range(self.hidden_size):
          [unit_j_location_x,unit_j_location_y] = np.where(spatial_embedding_arrangement == j)
          unit_j_location = np.array([unit_j_location_x,unit_j_location_y])
          distance_matrix[i,j] = np.linalg.norm(unit_i_location - unit_j_location)
      return torch.from_numpy(distance_matrix.T).float() # For consistency as Pytorch .weights (for h2h linear) is arranged such that weight from i to j = W[j,i] #here are distance matrix is symmetric so doesnt matter but good for understanding

    def penalise_weight_distance(self):
      # Hidden Layer Weight Matrix
      # Effective Weight Matrix
      eff_weight_matrix = self.rnn.h2h.weight * self.rnn.nsc_mask
      weight_matrix_penalty = torch.abs(eff_weight_matrix)
      # Weight Distance elementwise product
      # wd_penalty_raw =  weight_matrix_penalty * torch.square(self.distance_matrix_computed)
      wd_penalty_raw =  weight_matrix_penalty * self.distance_matrix_computed # try L1
      L2_penalty_term = torch.sum(wd_penalty_raw)
      return L2_penalty_term


    def forward(self, x):
        rnn_output, _ = self.rnn(x)
        out = self.fc(rnn_output)
        return out, rnn_output

    def forward_for_fpf_ics(self, x):
      rnn_outputs, hidden_state_tensor = self.rnn.forward_helper_fpf_ICs(x)
      out = self.fc(rnn_outputs)
      return out, rnn_outputs, hidden_state_tensor

## E-I RNN

### EI - Effective Weight Module

In [ ]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.nn import functional as F
import math


class EIRecLinear(nn.Module):

    """Recurrent E-I Linear transformation.

    This module implements a linear transformation with recurrent E-I dynamics,
    where part of the units are excitatory and the rest are inhibitory.

    Args:
        hidden_size: int, the number of units in the layer.
        e_prop: float between 0 and 1, the proportion of excitatory units.
        bias: bool, if True, adds a learnable bias to the output.
    """

    __constants__ = ['bias', 'hidden_size', 'e_prop']

    def __init__(self, hidden_size, e_prop, bias=True):
        super().__init__()
        self.hidden_size = hidden_size
        self.e_prop = e_prop
        self.e_size = int(e_prop * hidden_size) # Number of excitatory units
        self.i_size = hidden_size - self.e_size # Number of inhibitory units

        # Weight matrix for the recurrent connections
        self.weight = nn.Parameter(torch.Tensor(hidden_size, hidden_size))

        # Create a mask to define the E-I interactions
        # The mask has ones for E to E/I and negative ones for I to E/I, except the diagonal

        mask_no_diag = np.ones((self.hidden_size,self.hidden_size)) -  np.diag(np.ones((self.hidden_size)))

        E_I_unit_list = np.concatenate((np.ones((self.e_size,1)),-1*np.ones((self.i_size,1)))).T

        mask = mask_no_diag*E_I_unit_list
        # self.register_buffer('mask', torch.tensor(mask, dtype=torch.float32))
        self.mask = torch.tensor(mask, dtype=torch.float32)

        # Optionally add a bias term
        if bias:
            self.bias = nn.Parameter(torch.Tensor(hidden_size))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        # Initialise weights and biases
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        # Scale the weights for the excitatory neurons
        self.weight.data[:, :self.e_size] /= (self.e_size/self.i_size)

        # Initialise biases
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def effective_weight(self):
        # Apply the mask you have already created to the weights after applying rectification to get the effective weight
        # This ensures that weights from excitatory neurons are positive,
        # and weights from inhibitory neurons are negative.
        eff_W = F.relu(self.weight)*self.mask
        return eff_W

    def forward(self, input):
        # Apply the linear transformation using the effective weights and biases
        # The weights used are non-negative due to the absolute value in effective_weight.
        return F.linear(input, self.effective_weight(), self.bias)


### EI - RNN Layer

In [ ]:

class EIRNN(nn.Module):
    """E-I RNN.

    Reference:
        Song, H.F., Yang, G.R. and Wang, X.J., 2016.
        Training excitatory-inhibitory recurrent neural networks
        for cognitive tasks: a simple and flexible framework.
        PLoS computational biology, 12(2).

    Args:
        input_size: Number of input neurons
        hidden_size: Number of hidden neurons

    Inputs:
        input: (seq_len, batch, input_size)
        hidden: (batch, hidden_size)
        e_prop: float between 0 and 1, proportion of excitatory neurons
    """

    def __init__(self, input_size, hidden_size, dt=None,
                 e_prop=0.8, sigma_rec=0, **kwargs):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.e_size = int(hidden_size * e_prop)
        self.i_size = hidden_size - self.e_size
        self.num_layers = 1
        self.tau = 100
        if dt is None:
            alpha = 1
        else:
            alpha = dt / self.tau
        self.alpha = alpha
        self.oneminusalpha = 1 - alpha
        # Recurrent noise parameter, scaled by the discretization (sqrt(2*alpha)) and noise level (sigma_rec)
        # This adds stochasticity to the recurrent dynamics, possibly simulating biological neural variability
        self._sigma_rec = np.sqrt(2*alpha) * sigma_rec

        self.input2h = nn.Linear(input_size, hidden_size)
        self.h2h = EIRecLinear(hidden_size, e_prop=0.8)

    def init_hidden(self, input):
        batch_size = input.shape[1]
        return (torch.zeros(batch_size, self.hidden_size).to(input.device),
                torch.zeros(batch_size, self.hidden_size).to(input.device))

    def recurrence(self, input, hidden):
        """Recurrence helper."""
        state, output = hidden
        total_input = self.input2h(input) + self.h2h(output)
        state = state * self.oneminusalpha + total_input * self.alpha
        state += self._sigma_rec * torch.randn_like(state)
        output = torch.relu(state)
        return state, output


    def forward(self, input, hidden=None):
        """Propogate input through the network."""
        if hidden is None:
            hidden = self.init_hidden(input)

        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden = self.recurrence(input[i], hidden)
            output.append(hidden[1])
        output = torch.stack(output, dim=0)
        return output, hidden


    def recurrence_helper_fpf(self, input, hidden_state_only):
        """Recurrence helper."""
        state, output = hidden_state_only, torch.relu(hidden_state_only)
        total_input = self.input2h(input) + self.h2h(output)
        state = state * self.oneminusalpha + total_input * self.alpha
        state += self._sigma_rec * torch.randn_like(state)
        return state

    def forward_helper_fpf(self, input, hidden_state_only=None):
        """Propogate input through the network."""
        if hidden_state_only is None:
            hidden_state_only = self.init_hidden(input)[0]
        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden_state_only = self.recurrence_helper_fpf(input[i], hidden_state_only)
            hidden_output = torch.relu(hidden_state_only)
            output.append(hidden_output)
        output = torch.stack(output, dim=0)
        return output, hidden_state_only
    def forward_helper_fpf_ICs(self, input, hidden_state_only=None):
        """Propogate input through the network."""
        if hidden_state_only is None:
            hidden_state_only = self.init_hidden(input)[0]
        output = []
        steps = range(input.size(0))
        hidden_state_only_list = []
        # hidden_state_only_list.append(hidden_state_only)
        for i in steps:
            hidden_state_only = self.recurrence_helper_fpf(input[i], hidden_state_only)
            hidden_state_only_list.append(hidden_state_only)
            hidden_output = torch.relu(hidden_state_only)
            output.append(hidden_output)
        output = torch.stack(output, dim=0)
        hidden_state_only_tensor = torch.stack(hidden_state_only_list, dim=0)
        return output, hidden_state_only_tensor # hidden_state_only_tensor is shaped (seq_len, batch_size, hidden_size) and contains the raw states, we want to track this in FPF testing to obtain ICs



### EI - Main Network

In [ ]:

class EINet(nn.Module):
    """Recurrent network model.

    Args:
        input_size: int, input size
        hidden_size: int, hidden size
        output_size: int, output size
        rnn: str, type of RNN, lstm, rnn, ctrnn, or eirnn
    """
    def __init__(self, input_size, hidden_size, output_size, **kwargs):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.input_size = input_size
        # Excitatory-inhibitory RNN
        self.rnn = EIRNN(input_size, hidden_size, **kwargs)
        self.fc = nn.Linear(self.rnn.e_size, output_size)

    def forward(self, x):
        rnn_activity, _ = self.rnn(x)
        rnn_e = rnn_activity[:, :, :self.rnn.e_size]
        out = self.fc(rnn_e)
        return out, rnn_activity

    def forward_for_fpf_ics(self, x):
        rnn_outputs, hidden_state_tensor = self.rnn.forward_helper_fpf_ICs(x)
        rnn_e = rnn_outputs[:, :, :self.rnn.e_size]
        out = self.fc(rnn_e)
        return out, rnn_outputs, hidden_state_tensor


# Initialising for BIC Networks

Good for temp testing of diff functionality and using the current Standard Implementation to guide. If we implement three at a time, is that dumb or can we get the outputs in a way that informs the analysis / write up narrative

## Starting From Pretrained / Training

### Training Data Sets

In [ ]:
# Dataset Setups

### SHARED ACROSS TRAINING VALID TEST
timestep = 50 #ms
noise = 1.0
batch_size = 64

# Assign Timing for each task period (same as default but enables calculation of sequence length)
timing_valid_s = {
            'fixation': 300,
            'sample': 500,
            'delay': 300,
            'test': 500,
            'decision': 900}
timing_valid_m = {
            'fixation': 300,
            'sample': 500,
            'delay': 500,
            'test': 500,
            'decision': 900}
timing_valid_l = {
            'fixation': 300,
            'sample': 500,
            'delay': 900,
            'test': 500,
            'decision': 900}

timing_valid_ll = {
            'fixation': 300,
            'sample': 500,
            'delay': 1000,
            'test': 500,
            'decision': 900}

timing_training = {
            'fixation': 300,
            'sample': 500,
            'delay': ('choice', (300, 400, 500, 600, 700, 800, 900)),
            'test': 500,
            'decision': 900}

# Assign Time step size and calculate sequence length of a full trial
# Short
trial_time_s = sum(timing_valid_s.values()) #ms
seq_len_valid_s = int(trial_time_s/timestep)
# Medium
trial_time_m = sum(timing_valid_m.values()) #ms
seq_len_valid_m = int(trial_time_m/timestep)
# Long
trial_time_l = sum(timing_valid_l.values()) #ms
seq_len_valid_l = int(trial_time_l/timestep)
# X-Long
trial_time_ll = sum(timing_valid_ll.values()) #ms
seq_len_valid_ll = int(trial_time_ll/timestep)

task_name = 'DelayMatchSample-v0'

kwargs_valid_s = {'dt' : timestep, 'timing': timing_valid_s, 'sigma': noise}
kwargs_valid_m = {'dt' : timestep, 'timing': timing_valid_m, 'sigma': noise}
kwargs_valid_l = {'dt' : timestep, 'timing': timing_valid_l, 'sigma': noise}
kwargs_valid_ll = {'dt' : timestep, 'timing': timing_valid_ll, 'sigma': noise}
kwargs_training = {'dt' : timestep, 'timing': timing_training, 'sigma': noise}

dataset_valid_s = ngym.Dataset(task_name, env_kwargs=kwargs_valid_s, batch_size=batch_size, seq_len=seq_len_valid_s)
dataset_valid_m = ngym.Dataset(task_name, env_kwargs=kwargs_valid_m, batch_size=batch_size, seq_len=seq_len_valid_m)
dataset_valid_l = ngym.Dataset(task_name, env_kwargs=kwargs_valid_l, batch_size=batch_size, seq_len=seq_len_valid_l)
dataset_valid_ll = ngym.Dataset(task_name, env_kwargs=kwargs_valid_ll, batch_size=batch_size, seq_len=seq_len_valid_ll)
dataset_training = ngym.Dataset(task_name, env_kwargs=kwargs_training, batch_size=batch_size, seq_len=124)

validation_set_dictionary = {
    'Short': dataset_valid_s,
    'Medium': dataset_valid_m,
    'Long': dataset_valid_l,
    'UnseenLong' : dataset_valid_ll
}

### Initialising LRNN-NSC from .pth / TRAINING

In [ ]:
# Initialising Parameters
hidden_size_NSC = 64
input_size_NSC = 3
output_size_NSC = 3
dt_NSC = 50
tau_NSC = 100
# Initialising base model
LRNN_NSC = RNNNet_MS(input_size=input_size_NSC, hidden_size = hidden_size_NSC, output_size = output_size_NSC, dt=dt_NSC, tau=tau_NSC)
# MODEL_IO_NSC = False

if MODEL_IO_NSC:
  LRNN_NSC_TRAINED = LRNN_NSC
  # LRNN_NSC_TRAINED.load_state_dict(torch.load('CURRENT_MODEL_PTHS/NSC_LRNN_MODEL.pth'))
  LRNN_NSC_TRAINED.load_state_dict(torch.load('LRNN_NSC/LRNN_NSC.pth'))
  LRNN_NSC_TRAINED.eval()
else:
  LRNN_NSC_TRAINED, LRNN_NSC_training_dict = training_with_early_stop_and_regularisation(
    model=LRNN_NSC,
    training_set= dataset_training,
    validation_set_dict = validation_set_dictionary,
    WD_approach=False,
    WD_regulariser=None,
    wiring_beta=0.001,
    activity_regularisation=True,
    activity_beta=1e-1,
    max_steps=100000,
    min_validation_perf=0.8,
    patience=5,
    num_steps_for_early_stop_check=1500,
    num_validation_trials=200,
    model_name='LRNN_NSC_best',
    tr_output_mode= True)



In [ ]:

  # plot_learning_curve(
  #   learning_curve_dict=LRNN_NSC_training_dict,
  #   average_only=False,
  #   plot_loss_components=True,
  #   filename='LRNN_NSC/Learning_Curve_NSC_all.png',
  #   show_legend=True,
  #   legend_location='center right')


### Initialising LRNN-SE_2D from .pth

In [ ]:
# Initialising Parameters
hidden_size_SE2D = 64
input_size_SE2D = 3
output_size_SE2D = 3
dt_SE2D = 50
tau_SE2D = 100
# Initialising base model
LRNN_SE2D = SE_RNNNet(input_size=input_size_SE2D, hidden_size = hidden_size_SE2D, output_size = output_size_SE2D, dt=dt_SE2D, tau=tau_SE2D)
# MODEL_IO_SE2D = False

if MODEL_IO_SE2D:
  LRNN_SE2D_TRAINED = LRNN_SE2D
  # LRNN_SE2D_TRAINED.load_state_dict(torch.load('CURRENT_MODEL_PTHS/2D_WIRING_MODEL.pth'))
  LRNN_SE2D_TRAINED.load_state_dict(torch.load('LRNN_SE2D/LRNN_SE2D.pth'))
  LRNN_SE2D_TRAINED.eval()

else:
  LRNN_SE2D_TRAINED, LRNN_SE2D_training_dict = training_with_early_stop_and_regularisation(
    model=LRNN_SE2D,
    training_set= dataset_training,
    validation_set_dict = validation_set_dictionary,
    WD_approach=False,
    WD_regulariser=None,
    wiring_beta=0.00001,
    activity_regularisation=True,
    activity_beta=1e-1,
    max_steps=100000,
    min_validation_perf=0.8,
    patience=10,
    num_steps_for_early_stop_check=500,
    num_validation_trials=200,
    model_name='LRNN_SE2D_best',
    tr_output_mode= True)
  plot_learning_curve(
    learning_curve_dict=LRNN_SE2D_training_dict,
    average_only=False,
    plot_loss_components=True,
    filename='LRNN_SE2D/Learning_Curve_SE2D_all.png',
    show_legend=True,
    legend_location='center right')





8


### Initialising LRNN-SE_3D from .pth

In [ ]:
WD_regulariser = Reg_WD(reg_WD=0.0001, neuron_num=64, network_structure=(4,4,4))

In [ ]:
# # Initialising Parameters
hidden_size_SE3D = 64
input_size_SE3D = 3
output_size_SE3D = 3
dt_SE3D = 50
tau_SE3D = 100
# Initialising base model
LRNN_SE3D = RNNNet_MS(input_size=input_size_SE3D, hidden_size = hidden_size_SE3D, output_size = output_size_SE3D, dt=dt_SE3D, tau=tau_SE3D)
# MODEL_IO_SE3D = False

if MODEL_IO_SE3D:
  LRNN_SE3D_TRAINED = LRNN_SE3D
  LRNN_SE3D_TRAINED.load_state_dict(torch.load('LRNN_SE3D/LRNN_SE3D.pth'))
  LRNN_SE3D_TRAINED.eval()
else:
  LRNN_SE3D_TRAINED, LRNN_SE3D_training_dict = training_with_early_stop_and_regularisation(
    model=LRNN_SE3D,
    training_set= dataset_training,
    validation_set_dict = validation_set_dictionary,
    WD_approach=True,
    WD_regulariser=WD_regulariser,
    wiring_beta=0.0001,
    activity_regularisation=True,
    activity_beta=1e-1,
    max_steps=100000,
    min_validation_perf=0.8,
    patience=10,
    num_steps_for_early_stop_check=500,
    num_validation_trials=200,
    model_name='LRNN_SE3D_best',
    tr_output_mode= True)
  plot_learning_curve(
    learning_curve_dict=LRNN_SE3D_training_dict,
    average_only=False,
    plot_loss_components=True,
    filename='LRNN_SE3D/Learning_Curve_SE3D_all.png',
    show_legend=True,
    legend_location= 'center right' )


### Initialising EI-LRNN from .pth

In [ ]:
# Initialising Parameters
hidden_size_EI = 64
input_size_EI = 3
output_size_EI = 3
dt_EI = 50
tau_EI = 100
# Initialising base model # Does not take tau, maybe change this if want to use EI Layers in 2c
LRNN_EI = EINet(input_size=input_size_EI, hidden_size = hidden_size_EI, output_size = output_size_EI, dt=dt_EI)
# MODEL_IO_EI = False

if MODEL_IO_EI:
  LRNN_EI_TRAINED = LRNN_EI
  LRNN_EI_TRAINED.load_state_dict(torch.load('LRNN_EI/LRNN_EI.pth'))
  LRNN_EI_TRAINED.eval()

else:
  LRNN_EI_TRAINED, LRNN_EI_training_dict = training_with_early_stop_and_regularisation(
    model=LRNN_EI,
    training_set= dataset_training,
    validation_set_dict = validation_set_dictionary,
    WD_approach=False,
    WD_regulariser=None,
    wiring_beta=0.001,
    activity_regularisation=True,
    activity_beta=5e-2,
    max_steps=100000,
    min_validation_perf=0.8,
    patience=10,
    num_steps_for_early_stop_check=500,
    num_validation_trials=200,
    model_name='LRNN_EI_best',
    tr_output_mode= True)
  plot_learning_curve(
    learning_curve_dict=LRNN_EI_training_dict,
    average_only=False,
    plot_loss_components=True,
    filename='LRNN_EI/Learning_Curve_EI_all.png',
    show_legend=True,
    legend_location= 'center right' )

# Implementing Testing

## Initialising Testing Dataset

In [ ]:
# Create a dataset for testing
# Using Unseen 'delay' at 1100 : extrapolation on working memory time period
timing_testing = {
            'fixation': 300,
            'sample': 500,
            'delay': 1100,
            'test': 500,
            'decision': 900}

timestep = 50
noise = 0.0 # Consider setting to 0 (check with gem)
batch_size = 64 # Consistent with Training (dont think it has that much of an effect)
# Testing Trials
trial_time_testing = sum(timing_testing.values()) #ms
seq_len_testing = int(trial_time_testing/timestep)

task_name = 'DelayMatchSample-v0'


kwargs_testing = {'dt' : timestep, 'timing': timing_testing, 'sigma': noise}

# kwargs_testing = {'dt' : timestep, 'timing': timing_testing, 'sigma': 0.0}

dataset_testing = ngym.Dataset(task_name, env_kwargs=kwargs_testing, batch_size=batch_size, seq_len=seq_len_testing)

## Running Testing for Each Network

### Testing for LRNN - NSC

In [ ]:
# LRNN_NSC_full_testing_data = evaluate_network_on_dataset(LRNN_NSC_TRAINED, dataset_testing, num_trials=2000)
LRNN_NSC_full_testing_data = bic_testing_w_state_tracking(LRNN_NSC_TRAINED, dataset_testing, num_trials=2000)

Average performance 1.0


### Testing for LRNN - SE2D

In [ ]:
# LRNN_SE2D_full_testing_data = evaluate_network_on_dataset(LRNN_SE2D_TRAINED, dataset_testing, num_trials=2000)
LRNN_SE2D_full_testing_data = bic_testing_w_state_tracking(LRNN_SE2D_TRAINED, dataset_testing, num_trials=2000)

Average performance 1.0


### Testing for LRNN - SE3D

In [ ]:
# LRNN_SE3D_full_testing_data = evaluate_network_on_dataset(LRNN_SE3D_TRAINED, dataset_testing, num_trials=2000)
LRNN_SE3D_full_testing_data = bic_testing_w_state_tracking(LRNN_SE3D_TRAINED, dataset_testing, num_trials=2000)

Average performance 1.0


### Testing for EI - LRNN

In [ ]:
# LRNN_EI_full_testing_data = evaluate_network_on_dataset(LRNN_EI_TRAINED, dataset_testing, num_trials=2000)
LRNN_EI_full_testing_data = bic_testing_w_state_tracking(LRNN_EI_TRAINED, dataset_testing, num_trials=2000)

Average performance 1.0


# Performing Data Partitioning for Each Network's Testing Data

## Data Partitioning LRNN - NSC

In [ ]:
## NOTE : THESE ARE ALL OF THE SAME SHAPE AS THE ORIGINAL INPUT DICT
# trial_activity_matching
cond_dict_matching = {
    'test_equals_sample' : True
}
LRNN_NSC_testing_data_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, cond_dict_matching, return_like_full=True)

# trial_activity_non_matching

cond_dict_non_matching = {
    'test_equals_sample' : False
}
LRNN_NSC_testing_data_non_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, cond_dict_non_matching, return_like_full=True)

# trial_activity_0p0_sample
cond_dict_0p0_sample = {
    'sample_stim_value' : 0.0
}
LRNN_NSC_testing_data_0p0_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'sample_stim_value' : 0.0}, return_like_full=True)

# trial_activity_0p0_test
cond_dict_0p0_test = {
    'test_stim_value' : 0.0
}
LRNN_NSC_testing_data_0p0_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'test_stim_value' : 0.0}, return_like_full=True)


# trial_activity_3p1_sample
cond_dict_3p1_sample = {
    'sample_stim_value' : np.pi
}
LRNN_NSC_testing_data_3p1_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'sample_stim_value' : np.pi}, return_like_full=True)

# trial_activity_3p1_test
cond_dict_3p1_test = {
    'test_stim_value' : np.pi
}
LRNN_NSC_testing_data_3p1_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'test_stim_value' : np.pi}, return_like_full=True)

# trial_activity_0p0_3p1
cond_dict_0p0_3p1 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : np.pi
}
LRNN_NSC_testing_data_0p0_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : np.pi}, return_like_full=True)

#trial_activity_0p0_0p0
cond_dict_0p0_0p0 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : 0.0
}
LRNN_NSC_testing_data_0p0_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_0p0
cond_dict_3p1_0p0 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : 0.0
}
LRNN_NSC_testing_data_3p1_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_3p1
cond_dict_3p1_3p1 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : np.pi
}
LRNN_NSC_testing_data_3p1_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : np.pi}, return_like_full=True)

## Data Partitioning LRNN - SE2D

In [ ]:
## NOTE : THESE ARE ALL OF THE SAME SHAPE AS THE ORIGINAL INPUT DICT
# trial_activity_matching

cond_dict_matching = {
    'test_equals_sample' : True
}
LRNN_SE2D_testing_data_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, cond_dict_matching, return_like_full=True)

# trial_activity_non_matching

cond_dict_non_matching = {
    'test_equals_sample' : False
}
LRNN_SE2D_testing_data_non_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, cond_dict_non_matching, return_like_full=True)

# trial_activity_0p0_sample
cond_dict_0p0_sample = {
    'sample_stim_value' : 0.0
}
LRNN_SE2D_testing_data_0p0_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'sample_stim_value' : 0.0}, return_like_full=True)

# trial_activity_0p0_test
cond_dict_0p0_test = {
    'test_stim_value' : 0.0
}
LRNN_SE2D_testing_data_0p0_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'test_stim_value' : 0.0}, return_like_full=True)


# trial_activity_3p1_sample
cond_dict_3p1_sample = {
    'sample_stim_value' : np.pi
}
LRNN_SE2D_testing_data_3p1_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'sample_stim_value' : np.pi}, return_like_full=True)

# trial_activity_3p1_test
cond_dict_3p1_test = {
    'test_stim_value' : np.pi
}
LRNN_SE2D_testing_data_3p1_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'test_stim_value' : np.pi}, return_like_full=True)

# trial_activity_0p0_3p1
cond_dict_0p0_3p1 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : np.pi
}
LRNN_SE2D_testing_data_0p0_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : np.pi}, return_like_full=True)

#trial_activity_0p0_0p0
cond_dict_0p0_0p0 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : 0.0
}
LRNN_SE2D_testing_data_0p0_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_0p0
cond_dict_3p1_0p0 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : 0.0
}
LRNN_SE2D_testing_data_3p1_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_3p1
cond_dict_3p1_3p1 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : np.pi
}
LRNN_SE2D_testing_data_3p1_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : np.pi}, return_like_full=True)

## Data Partitioning LRNN - SE3D

In [ ]:
# ## NOTE : THESE ARE ALL OF THE SAME SHAPE AS THE ORIGINAL INPUT DICT
# # trial_activity_matching
cond_dict_matching = {
    'test_equals_sample' : True
}

LRNN_SE3D_testing_data_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, cond_dict_matching, return_like_full=True)

# trial_activity_non_matching

cond_dict_non_matching = {
    'test_equals_sample' : False
}
LRNN_SE3D_testing_data_non_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, cond_dict_non_matching, return_like_full=True)

# trial_activity_0p0_sample
cond_dict_0p0_sample = {
    'sample_stim_value' : 0.0
}
LRNN_SE3D_testing_data_0p0_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'sample_stim_value' : 0.0}, return_like_full=True)

# trial_activity_0p0_test
cond_dict_0p0_test = {
    'test_stim_value' : 0.0
}
LRNN_SE3D_testing_data_0p0_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'test_stim_value' : 0.0}, return_like_full=True)


# trial_activity_3p1_sample
cond_dict_3p1_sample = {
    'sample_stim_value' : np.pi
}
LRNN_SE3D_testing_data_3p1_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'sample_stim_value' : np.pi}, return_like_full=True)

# trial_activity_3p1_test
cond_dict_3p1_test = {
    'test_stim_value' : np.pi
}
LRNN_SE3D_testing_data_3p1_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'test_stim_value' : np.pi}, return_like_full=True)

# trial_activity_0p0_3p1
cond_dict_0p0_3p1 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : np.pi
}
LRNN_SE3D_testing_data_0p0_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : np.pi}, return_like_full=True)

#trial_activity_0p0_0p0
cond_dict_0p0_0p0 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : 0.0
}
LRNN_SE3D_testing_data_0p0_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_0p0
cond_dict_3p1_0p0 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : 0.0
}
LRNN_SE3D_testing_data_3p1_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_3p1
cond_dict_3p1_3p1 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : np.pi
}
LRNN_SE3D_testing_data_3p1_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : np.pi}, return_like_full=True)

## Data Partitioning LRNN - E-I

In [ ]:
## NOTE : THESE ARE ALL OF THE SAME SHAPE AS THE ORIGINAL INPUT DICT
# trial_activity_matching
cond_dict_matching = {
    'test_equals_sample' : True
}

LRNN_EI_testing_data_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, cond_dict_matching, return_like_full=True)

# trial_activity_non_matching

cond_dict_non_matching = {
    'test_equals_sample' : False
}
LRNN_EI_testing_data_non_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, cond_dict_non_matching, return_like_full=True)

# trial_activity_0p0_sample
cond_dict_0p0_sample = {
    'sample_stim_value' : 0.0
}
LRNN_EI_testing_data_0p0_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'sample_stim_value' : 0.0}, return_like_full=True)

# trial_activity_0p0_test
cond_dict_0p0_test = {
    'test_stim_value' : 0.0
}
LRNN_EI_testing_data_0p0_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'test_stim_value' : 0.0}, return_like_full=True)


# trial_activity_3p1_sample
cond_dict_3p1_sample = {
    'sample_stim_value' : np.pi
}
LRNN_EI_testing_data_3p1_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'sample_stim_value' : np.pi}, return_like_full=True)

# trial_activity_3p1_test
cond_dict_3p1_test = {
    'test_stim_value' : np.pi
}
LRNN_EI_testing_data_3p1_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'test_stim_value' : np.pi}, return_like_full=True)

# trial_activity_0p0_3p1
cond_dict_0p0_3p1 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : np.pi
}
LRNN_EI_testing_data_0p0_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : np.pi}, return_like_full=True)

#trial_activity_0p0_0p0
cond_dict_0p0_0p0 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : 0.0
}
LRNN_EI_testing_data_0p0_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_0p0
cond_dict_3p1_0p0 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : 0.0
}
LRNN_EI_testing_data_3p1_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_3p1
cond_dict_3p1_3p1 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : np.pi
}
LRNN_EI_testing_data_3p1_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : np.pi}, return_like_full=True)

# Plotting Unit Activity

## Plotting Unit Activity - LRNN NSC

### LRNN NSC - Single Trial Unit Activity

In [ ]:
#Single trial of each sample x test combination
LRNN_NSC_one_trial_0p0_0p0 = LRNN_NSC_testing_data_0p0_0p0['testing_trial_and_activity'][list(LRNN_NSC_testing_data_0p0_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NSC_one_trial_0p0_3p1 = LRNN_NSC_testing_data_0p0_3p1['testing_trial_and_activity'][list(LRNN_NSC_testing_data_0p0_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NSC_one_trial_3p1_0p0 = LRNN_NSC_testing_data_3p1_0p0['testing_trial_and_activity'][list(LRNN_NSC_testing_data_3p1_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NSC_one_trial_3p1_3p1 = LRNN_NSC_testing_data_3p1_3p1['testing_trial_and_activity'][list(LRNN_NSC_testing_data_3p1_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NSC_one_trial_plot_env_info = LRNN_NSC_testing_data_0p0_0p0['testing_env_info'] # Same for each



In [ ]:
# 0p0_0p0
plot_unit_activity_over_time(LRNN_NSC_one_trial_0p0_0p0, LRNN_NSC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NSC/1trial_0p0_0p0_LRNN_NSC.png') # plot all units, no legend for now
#0p0_3p1
plot_unit_activity_over_time(LRNN_NSC_one_trial_0p0_3p1, LRNN_NSC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NSC/1trial_0p0_3p1_LRNN_NSC.png') # plot all units, no legend for now
# 3p1_0p0
plot_unit_activity_over_time(LRNN_NSC_one_trial_3p1_0p0, LRNN_NSC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NSC/1trial_3p1_0p0_LRNN_NSC.png')
#3p1_3p1
plot_unit_activity_over_time(LRNN_NSC_one_trial_3p1_3p1, LRNN_NSC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NSC/1trial_3p1_3p1_LRNN_NSC.png')


Figure saved to LRNN_NSC/1trial_0p0_0p0_LRNN_NSC.png
Figure saved to LRNN_NSC/1trial_0p0_3p1_LRNN_NSC.png
Figure saved to LRNN_NSC/1trial_3p1_0p0_LRNN_NSC.png
Figure saved to LRNN_NSC/1trial_3p1_3p1_LRNN_NSC.png


### LRNN NSC - Average Unit Activity Plot

In [ ]:

LRNN_NSC_all_trial_0p0_0p0 = LRNN_NSC_testing_data_0p0_0p0['testing_trial_and_activity']
LRNN_NSC_all_trial_0p0_3p1 = LRNN_NSC_testing_data_0p0_3p1['testing_trial_and_activity']
LRNN_NSC_all_trial_3p1_0p0 = LRNN_NSC_testing_data_3p1_0p0['testing_trial_and_activity']
LRNN_NSC_all_trial_3p1_3p1 = LRNN_NSC_testing_data_3p1_3p1['testing_trial_and_activity']
LRNN_NSC_all_trial_plot_env_info = LRNN_NSC_testing_data_0p0_0p0['testing_env_info'] # Same for each


In [ ]:

#0p0_0p0
plot_average_unit_activity_over_time(LRNN_NSC_all_trial_0p0_0p0, LRNN_NSC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NSC/avg_0p0_0p0_LRNN_NSC.png')

#0p0_3p1
plot_average_unit_activity_over_time(LRNN_NSC_all_trial_0p0_3p1, LRNN_NSC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NSC/avg_0p0_3p1_LRNN_NSC.png')

#3p1_0p0
plot_average_unit_activity_over_time(LRNN_NSC_all_trial_3p1_0p0, LRNN_NSC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NSC/avg_3p1_0p0_LRNN_NSC.png')

#3p1_3p1
plot_average_unit_activity_over_time(LRNN_NSC_all_trial_3p1_3p1, LRNN_NSC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NSC/avg_3p1_3p1_LRNN_NSC.png')


Figure saved to LRNN_NSC/avg_0p0_0p0_LRNN_NSC.png
Figure saved to LRNN_NSC/avg_0p0_3p1_LRNN_NSC.png
Figure saved to LRNN_NSC/avg_3p1_0p0_LRNN_NSC.png
Figure saved to LRNN_NSC/avg_3p1_3p1_LRNN_NSC.png


## Plotting Unit Activity - LRNN SE2D

### LRNN SE 2D - Single Trial Unit

In [ ]:
#Single trial of each sample x test combination
LRNN_SE2D_one_trial_0p0_0p0 = LRNN_SE2D_testing_data_0p0_0p0['testing_trial_and_activity'][list(LRNN_SE2D_testing_data_0p0_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE2D_one_trial_0p0_3p1 = LRNN_SE2D_testing_data_0p0_3p1['testing_trial_and_activity'][list(LRNN_SE2D_testing_data_0p0_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE2D_one_trial_3p1_0p0 = LRNN_SE2D_testing_data_3p1_0p0['testing_trial_and_activity'][list(LRNN_SE2D_testing_data_3p1_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE2D_one_trial_3p1_3p1 = LRNN_SE2D_testing_data_3p1_3p1['testing_trial_and_activity'][list(LRNN_SE2D_testing_data_3p1_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE2D_one_trial_plot_env_info = LRNN_SE2D_testing_data_0p0_0p0['testing_env_info'] # Same for each



In [ ]:
# 0p0_0p0
plot_unit_activity_over_time(LRNN_SE2D_one_trial_0p0_0p0, LRNN_SE2D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE2D/1trial_0p0_0p0_LRNN_SE2D.png') # plot all units, no legend for now
#0p0_3p1
plot_unit_activity_over_time(LRNN_SE2D_one_trial_0p0_3p1, LRNN_SE2D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE2D/1trial_0p0_3p1_LRNN_SE2D.png') # plot all units, no legend for now
# 3p1_0p0
plot_unit_activity_over_time(LRNN_SE2D_one_trial_3p1_0p0, LRNN_SE2D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE2D/1trial_3p1_0p0_LRNN_SE2D.png')
#3p1_3p1
plot_unit_activity_over_time(LRNN_SE2D_one_trial_3p1_3p1, LRNN_SE2D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE2D/1trial_3p1_3p1_LRNN_SE2D.png')

Figure saved to LRNN_SE2D/1trial_0p0_0p0_LRNN_SE2D.png
Figure saved to LRNN_SE2D/1trial_0p0_3p1_LRNN_SE2D.png
Figure saved to LRNN_SE2D/1trial_3p1_0p0_LRNN_SE2D.png
Figure saved to LRNN_SE2D/1trial_3p1_3p1_LRNN_SE2D.png


### LRNN SE 2D - Average Unit Activity Plot

In [ ]:
LRNN_SE2D_all_trial_0p0_0p0 = LRNN_SE2D_testing_data_0p0_0p0['testing_trial_and_activity']
LRNN_SE2D_all_trial_0p0_3p1 = LRNN_SE2D_testing_data_0p0_3p1['testing_trial_and_activity']
LRNN_SE2D_all_trial_3p1_0p0 = LRNN_SE2D_testing_data_3p1_0p0['testing_trial_and_activity']
LRNN_SE2D_all_trial_3p1_3p1 = LRNN_SE2D_testing_data_3p1_3p1['testing_trial_and_activity']
LRNN_SE2D_all_trial_plot_env_info = LRNN_SE2D_testing_data_0p0_0p0['testing_env_info'] # Same for each



In [ ]:
#0p0_0p0
plot_average_unit_activity_over_time(LRNN_SE2D_all_trial_0p0_0p0, LRNN_SE2D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE2D/avg_0p0_0p0_LRNN_SE2D.png')

#0p0_3p1
plot_average_unit_activity_over_time(LRNN_SE2D_all_trial_0p0_3p1, LRNN_SE2D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE2D/avg_0p0_3p1_LRNN_SE2D.png')

#3p1_0p0
plot_average_unit_activity_over_time(LRNN_SE2D_all_trial_3p1_0p0, LRNN_SE2D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE2D/avg_3p1_0p0_LRNN_SE2D.png')

#3p1_3p1
plot_average_unit_activity_over_time(LRNN_SE2D_all_trial_3p1_3p1, LRNN_SE2D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE2D/avg_3p1_3p1_LRNN_SE2D.png')

Figure saved to LRNN_SE2D/avg_0p0_0p0_LRNN_SE2D.png
Figure saved to LRNN_SE2D/avg_0p0_3p1_LRNN_SE2D.png
Figure saved to LRNN_SE2D/avg_3p1_0p0_LRNN_SE2D.png
Figure saved to LRNN_SE2D/avg_3p1_3p1_LRNN_SE2D.png


## Plotting Unit Activity - LRNN SE 3D

### LRNN SE 3D - Single Trial Unit

In [ ]:
#Single trial of each sample x test combination
LRNN_SE3D_one_trial_0p0_0p0 = LRNN_SE3D_testing_data_0p0_0p0['testing_trial_and_activity'][list(LRNN_SE3D_testing_data_0p0_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE3D_one_trial_0p0_3p1 = LRNN_SE3D_testing_data_0p0_3p1['testing_trial_and_activity'][list(LRNN_SE3D_testing_data_0p0_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE3D_one_trial_3p1_0p0 = LRNN_SE3D_testing_data_3p1_0p0['testing_trial_and_activity'][list(LRNN_SE3D_testing_data_3p1_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE3D_one_trial_3p1_3p1 = LRNN_SE3D_testing_data_3p1_3p1['testing_trial_and_activity'][list(LRNN_SE3D_testing_data_3p1_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_SE3D_one_trial_plot_env_info = LRNN_SE3D_testing_data_0p0_0p0['testing_env_info'] # Same for each



In [ ]:
# 0p0_0p0
plot_unit_activity_over_time(LRNN_SE3D_one_trial_0p0_0p0, LRNN_SE3D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE3D/1trial_0p0_0p0_LRNN_SE3D.png') # plot all units, no legend for now
#0p0_3p1
plot_unit_activity_over_time(LRNN_SE3D_one_trial_0p0_3p1, LRNN_SE3D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE3D/1trial_0p0_3p1_LRNN_SE3D.png') # plot all units, no legend for now
# 3p1_0p0
plot_unit_activity_over_time(LRNN_SE3D_one_trial_3p1_0p0, LRNN_SE3D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE3D/1trial_3p1_0p0_LRNN_SE3D.png')
#3p1_3p1
plot_unit_activity_over_time(LRNN_SE3D_one_trial_3p1_3p1, LRNN_SE3D_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_SE3D/1trial_3p1_3p1_LRNN_SE3D.png')


Figure saved to LRNN_SE3D/1trial_0p0_0p0_LRNN_SE3D.png
Figure saved to LRNN_SE3D/1trial_0p0_3p1_LRNN_SE3D.png
Figure saved to LRNN_SE3D/1trial_3p1_0p0_LRNN_SE3D.png
Figure saved to LRNN_SE3D/1trial_3p1_3p1_LRNN_SE3D.png


### LRNN SE 3D - Average Unit Activity

In [ ]:
LRNN_SE3D_all_trial_0p0_0p0 = LRNN_SE3D_testing_data_0p0_0p0['testing_trial_and_activity']
LRNN_SE3D_all_trial_0p0_3p1 = LRNN_SE3D_testing_data_0p0_3p1['testing_trial_and_activity']
LRNN_SE3D_all_trial_3p1_0p0 = LRNN_SE3D_testing_data_3p1_0p0['testing_trial_and_activity']
LRNN_SE3D_all_trial_3p1_3p1 = LRNN_SE3D_testing_data_3p1_3p1['testing_trial_and_activity']
LRNN_SE3D_all_trial_plot_env_info = LRNN_SE3D_testing_data_0p0_0p0['testing_env_info'] # Same for each

In [ ]:
#0p0_0p0
plot_average_unit_activity_over_time(LRNN_SE3D_all_trial_0p0_0p0, LRNN_SE3D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE3D/avg_0p0_0p0_LRNN_SE3D.png')

#0p0_3p1
plot_average_unit_activity_over_time(LRNN_SE3D_all_trial_0p0_3p1, LRNN_SE3D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE3D/avg_0p0_3p1_LRNN_SE3D.png')

#3p1_0p0
plot_average_unit_activity_over_time(LRNN_SE3D_all_trial_3p1_0p0, LRNN_SE3D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE3D/avg_3p1_0p0_LRNN_SE3D.png')

#3p1_3p1
plot_average_unit_activity_over_time(LRNN_SE3D_all_trial_3p1_3p1, LRNN_SE3D_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_SE3D/avg_3p1_3p1_LRNN_SE3D.png')


Figure saved to LRNN_SE3D/avg_0p0_0p0_LRNN_SE3D.png
Figure saved to LRNN_SE3D/avg_0p0_3p1_LRNN_SE3D.png
Figure saved to LRNN_SE3D/avg_3p1_0p0_LRNN_SE3D.png
Figure saved to LRNN_SE3D/avg_3p1_3p1_LRNN_SE3D.png


## Plotting Unit Activity - LRNN EI Net

### Custom Function for E-I Population Distinction

In [ ]:
def plot_ei_unit_activity_trial(trial_activity, env_info, e_size, unit_indices_to_plot=None, filename=None):
    """
    Takes a single trial's data and plots the activity of E/I units over time.

    Args:
        trial_activity : (shape: seq_len x hidden_size).
        env_info (dict): Environmental information including 'dt' and 'timing'.
        e_size (int): The number of excitatory units. Assumes E units are first.
        unit_indices_to_plot (list, optional): Specific unit indices to plot.
                                            If None, all units are plotted. Defaults to None.
        filename (str, optional): File path to save the figure. If None,
                                  the plot is displayed directly. Defaults to None.
    """

    seq_len, hidden_size = trial_activity.shape
    dt = env_info['dt']
    timing = env_info['timing']

    # Create time axis
    time_points = np.arange(seq_len) * dt

    fig, ax = plt.subplots(figsize=(15, 7))

    # Define E and I indices
    e_indices = set(range(e_size))
    i_indices = set(range(e_size, hidden_size))

    # Determine which units to plot
    units_to_plot = range(hidden_size)
    if unit_indices_to_plot is not None:
        units_to_plot = [i for i in unit_indices_to_plot if 0 <= i < hidden_size]

    # Plot unit activities with E/I colour coding and labeling
    e_plotted = False
    i_plotted = False
    for i in units_to_plot:
        if i in e_indices:
            label = 'Excitatory' if not e_plotted else ''
            ax.plot(time_points, trial_activity[:, i], color='blue', alpha=0.7, label=label)
            e_plotted = True
        elif i in i_indices:
            label = 'Inhibitory' if not i_plotted else ''
            ax.plot(time_points, trial_activity[:, i], color='red', alpha=0.7, label=label)
            i_plotted = True

    # Add vertical lines for task phases
    current_time = 0
    phase_boundaries = [0]
    for duration in timing.values():
        if isinstance(duration, (int, float)):
            current_time += duration
            if current_time <= (time_points[-1]+dt):
                ax.axvline(x=current_time, color='k', linestyle='--', alpha=0.5)
                phase_boundaries.append(current_time)

    # Add phase labels inside the plot after all data is plotted
    ymin, ymax = ax.get_ylim()
    text_y_position = ymax - (ymax - ymin) * 0.05  # Position text near the top

    phase_keys = list(timing.keys())
    for i in range(len(phase_boundaries) - 1):
        start_time = phase_boundaries[i]
        end_time = phase_boundaries[i+1]
        mid_point = start_time + (end_time - start_time) / 2
        ax.text(mid_point, text_y_position, phase_keys[i], ha='center', va='top', fontsize=12, style='italic')

    ax.set_title('E/I Network Unit Activity Over Time (Single Trial)')
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Activity')
    ax.legend()
    ax.grid(True, alpha=0.3)

    if filename:
        try:
            if os.path.dirname(filename):
                os.makedirs(os.path.dirname(filename), exist_ok=True)
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {filename}")
            plt.close(fig)
        except Exception as e:
            print(f"Error saving figure to {filename}: {e}")
            plt.show()
    else:
        plt.show()


def plot_ei_average_unit_activity(trial_data_dict, env_info, e_size, unit_indices_to_plot=None, filename=None):
    """
    Calculates and plots the average activity of E/I network units over time
    across multiple trials.

    Args:
        trial_data_dict (dict): A dictionary of multiple trials.
        env_info (dict): Environmental information including 'dt' and 'timing'.
        e_size (int): The number of excitatory units. Assumes E units are first.
        unit_indices_to_plot (list, optional): Specific unit indices to plot.
                                            If None, all units are plotted. Defaults to None.
        filename (str, optional): File path to save the figure. If None,
                                  the plot is displayed directly. Defaults to None.
    """
    if not trial_data_dict:
        print("No trial data provided for plotting.")
        return

    first_trial_key = list(trial_data_dict.keys())[0]
    first_trial_activity = trial_data_dict[first_trial_key]['network_activity']
    seq_len, hidden_size = first_trial_activity.shape
    dt = env_info['dt']
    timing = env_info['timing']

    # Stack all activity arrays for averaging
    all_activities = [t['network_activity'] for t in trial_data_dict.values() if t['network_activity'].shape == (seq_len, hidden_size)]
    if not all_activities:
        print("No valid trial data found for averaging.")
        return

    average_activity = np.mean(np.stack(all_activities, axis=0), axis=0)

    # Create time axis
    time_points = np.arange(seq_len) * dt

    fig, ax = plt.subplots(figsize=(15, 7))

    # Define E and I indices
    e_indices = set(range(e_size))
    i_indices = set(range(e_size, hidden_size))

    # Determine which units to plot
    units_to_plot = range(hidden_size)
    if unit_indices_to_plot is not None:
        units_to_plot = [i for i in unit_indices_to_plot if 0 <= i < hidden_size]

    # Plot average unit activities with E/I colour-coding and labeling
    e_plotted = False
    i_plotted = False
    for i in units_to_plot:
        if i in e_indices:
            label = 'Excitatory' if not e_plotted else ''
            ax.plot(time_points, average_activity[:, i], color='blue', alpha=0.7, label=label)
            e_plotted = True
        elif i in i_indices:
            label = 'Inhibitory' if not i_plotted else ''
            ax.plot(time_points, average_activity[:, i], color='red', alpha=0.7, label=label)
            i_plotted = True

    # Add vertical lines for task phases
    current_time = 0
    phase_boundaries = [0]
    for duration in timing.values():
        if isinstance(duration, (int, float)):
            current_time += duration
            if current_time <= (time_points[-1]+dt):
                ax.axvline(x=current_time, color='k', linestyle='--', alpha=0.5)
                phase_boundaries.append(current_time)

    # Add phase labels inside the plot
    ymin, ymax = ax.get_ylim()
    text_y_position = ymax - (ymax - ymin) * 0.05

    phase_keys = list(timing.keys())
    for i in range(len(phase_boundaries) - 1):
        start_time = phase_boundaries[i]
        end_time = phase_boundaries[i+1]
        mid_point = start_time + (end_time - start_time) / 2
        ax.text(mid_point, text_y_position, phase_keys[i], ha='center', va='top', fontsize=12, style='italic')

    ax.set_title('Average E/I Network Unit Activity Across Trials')
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Average Activity')
    ax.legend()
    ax.grid(True, alpha=0.3)

    if filename:
        try:
            if os.path.dirname(filename):
                os.makedirs(os.path.dirname(filename), exist_ok=True)
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"Figure saved to {filename}")
            plt.close(fig)
        except Exception as e:
            print(f"Error saving figure to {filename}: {e}")
            plt.show()
    else:
        plt.show()

In [ ]:
LRNN_EI_E_SIZE = LRNN_EI_TRAINED.rnn.e_size
print(LRNN_EI_E_SIZE)

51


### LRNN EI - Single Unit Activity

In [ ]:
#Single trial of each sample x test combination
LRNN_EI_one_trial_0p0_0p0 = LRNN_EI_testing_data_0p0_0p0['testing_trial_and_activity'][list(LRNN_EI_testing_data_0p0_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_EI_one_trial_0p0_3p1 = LRNN_EI_testing_data_0p0_3p1['testing_trial_and_activity'][list(LRNN_EI_testing_data_0p0_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_EI_one_trial_3p1_0p0 = LRNN_EI_testing_data_3p1_0p0['testing_trial_and_activity'][list(LRNN_EI_testing_data_3p1_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_EI_one_trial_3p1_3p1 = LRNN_EI_testing_data_3p1_3p1['testing_trial_and_activity'][list(LRNN_EI_testing_data_3p1_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_EI_one_trial_plot_env_info = LRNN_EI_testing_data_0p0_0p0['testing_env_info'] # Same for each



In [ ]:
## Testing the new func:
# plot_ei_unit_activity_trial(LRNN_EI_one_trial_0p0_0p0, env_info=LRNN_EI_one_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename=None)

In [ ]:
# 0p0_0p0
plot_ei_unit_activity_trial(LRNN_EI_one_trial_0p0_0p0, LRNN_EI_one_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename='LRNN_EI/1trial_0p0_0p0_LRNN_EI.png') # plot all units, no legend for now
#0p0_3p1

plot_ei_unit_activity_trial(LRNN_EI_one_trial_0p0_3p1, LRNN_EI_one_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename='LRNN_EI/1trial_0p0_3p1_LRNN_EI.png') # plot all units, no legend for now
# 3p1_0p0
plot_ei_unit_activity_trial(LRNN_EI_one_trial_3p1_0p0, LRNN_EI_one_trial_plot_env_info,e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None,  filename='LRNN_EI/1trial_3p1_0p0_LRNN_EI.png')
#3p1_3p1
plot_ei_unit_activity_trial(LRNN_EI_one_trial_3p1_3p1, LRNN_EI_one_trial_plot_env_info,e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None,  filename='LRNN_EI/1trial_3p1_3p1_LRNN_EI.png')


Figure saved to LRNN_EI/1trial_0p0_0p0_LRNN_EI.png
Figure saved to LRNN_EI/1trial_0p0_3p1_LRNN_EI.png
Figure saved to LRNN_EI/1trial_3p1_0p0_LRNN_EI.png
Figure saved to LRNN_EI/1trial_3p1_3p1_LRNN_EI.png


### LRNN EI - Average Unit Activity

In [ ]:

LRNN_EI_all_trial_0p0_0p0 = LRNN_EI_testing_data_0p0_0p0['testing_trial_and_activity']
LRNN_EI_all_trial_0p0_3p1 = LRNN_EI_testing_data_0p0_3p1['testing_trial_and_activity']
LRNN_EI_all_trial_3p1_0p0 = LRNN_EI_testing_data_3p1_0p0['testing_trial_and_activity']
LRNN_EI_all_trial_3p1_3p1 = LRNN_EI_testing_data_3p1_3p1['testing_trial_and_activity']
LRNN_EI_all_trial_plot_env_info = LRNN_EI_testing_data_0p0_0p0['testing_env_info'] # Same for each


In [ ]:


#0p0_0p0
plot_ei_average_unit_activity(LRNN_EI_all_trial_0p0_0p0, LRNN_EI_all_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename='LRNN_EI/avg_0p0_0p0_LRNN_EI.png')
# plot_ei_average_unit_activity(LRNN_EI_all_trial_0p0_0p0, LRNN_EI_all_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None)

#0p0_3p1
plot_ei_average_unit_activity(LRNN_EI_all_trial_0p0_3p1, LRNN_EI_all_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename='LRNN_EI/avg_0p0_3p1_LRNN_EI.png')

#3p1_0p0
plot_ei_average_unit_activity(LRNN_EI_all_trial_3p1_0p0, LRNN_EI_all_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename='LRNN_EI/avg_3p1_0p0_LRNN_EI.png')

#3p1_3p1
plot_ei_average_unit_activity(LRNN_EI_all_trial_3p1_3p1, LRNN_EI_all_trial_plot_env_info, e_size=LRNN_EI_E_SIZE, unit_indices_to_plot=None, filename='LRNN_EI/avg_3p1_3p1_LRNN_EI.png')



Figure saved to LRNN_EI/avg_0p0_0p0_LRNN_EI.png
Figure saved to LRNN_EI/avg_0p0_3p1_LRNN_EI.png
Figure saved to LRNN_EI/avg_3p1_0p0_LRNN_EI.png
Figure saved to LRNN_EI/avg_3p1_3p1_LRNN_EI.png


# (Re-) Plotting Network Structure

## Network Structure Plot LRNN NSC

In [ ]:
# plot_recurrent_weights_heatmap_BIC(net, filename = None, mask=None, weights_passed=False)
plot_recurrent_weights_heatmap_BIC(LRNN_NSC_TRAINED, mask=LRNN_NSC_TRAINED.rnn.nsc_mask, filename='LRNN_NSC/LRNN_NSC_Connectivity.png')

Figure saved to LRNN_NSC/LRNN_NSC_Connectivity.png


## Network Structure Plot  LRNN SE 2D

In [ ]:
plot_recurrent_weights_heatmap_BIC(LRNN_SE2D_TRAINED, mask=LRNN_SE2D_TRAINED.rnn.nsc_mask, filename='LRNN_SE2D/LRNN_SE2D_Connectivity.png')

Figure saved to LRNN_SE2D/LRNN_SE2D_Connectivity.png


## Network Structure Plot  LRNN SE 3D

In [ ]:
plot_recurrent_weights_heatmap_BIC(LRNN_SE3D_TRAINED, mask=LRNN_SE3D_TRAINED.rnn.nsc_mask, filename='LRNN_SE3D/LRNN_SE3D_Connectivity.png')

Figure saved to LRNN_SE3D/LRNN_SE3D_Connectivity.png


## Network Structure Plot  LRNN EI

In [ ]:
plot_recurrent_weights_heatmap_BIC(LRNN_EI_TRAINED.rnn.h2h.effective_weight().detach().numpy(),weights_passed=True, filename='LRNN_EI/LRNN_EI_Connectivity.png')

Figure saved to LRNN_EI/LRNN_EI_Connectivity.png


# dPCA - All Networks (EXPORTING DATA)

## dPCA Export - LRNN-NSC

In [ ]:
export_nested_dict_to_json(LRNN_NSC_full_testing_data, filename= 'LRNN_NSC/LRNN_NSC_testing_data_for_dPCA.json')

Data successfully exported to LRNN_NSC/LRNN_NSC_testing_data_for_dPCA.json


In [ ]:
temp_testing_structure = import_nested_dict_from_json(filename = 'LRNN_NSC/LRNN_NSC_testing_data_for_dPCA.json')

Data successfully imported from LRNN_NSC/LRNN_NSC_testing_data_for_dPCA.json


In [ ]:
diff_from_io = compare_dicts(LRNN_NSC_full_testing_data, temp_testing_structure)

Differences found:
{'type_changes': {"root['testing_env_info']['sigma']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}, "root['testing_trial_performance']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}}}


## dPCA Export - LRNN-SE2D

In [ ]:

export_nested_dict_to_json(LRNN_SE2D_full_testing_data, filename= 'LRNN_SE2D/LRNN_SE2D_testing_data_for_dPCA.json')

temp_testing_structure = import_nested_dict_from_json(filename = 'LRNN_SE2D/LRNN_SE2D_testing_data_for_dPCA.json')

diff_from_io = compare_dicts(LRNN_SE2D_full_testing_data, temp_testing_structure)

Data successfully exported to LRNN_SE2D/LRNN_SE2D_testing_data_for_dPCA.json
Data successfully imported from LRNN_SE2D/LRNN_SE2D_testing_data_for_dPCA.json
Differences found:
{'type_changes': {"root['testing_env_info']['sigma']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}, "root['testing_trial_performance']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}}}


## dPCA Export - LRNN-SE3D

In [ ]:
export_nested_dict_to_json(LRNN_SE3D_full_testing_data, filename= 'LRNN_SE3D/LRNN_SE3D_testing_data_for_dPCA.json')

temp_testing_structure = import_nested_dict_from_json(filename = 'LRNN_SE3D/LRNN_SE3D_testing_data_for_dPCA.json')

diff_from_io = compare_dicts(LRNN_SE3D_full_testing_data, temp_testing_structure)


Data successfully exported to LRNN_SE3D/LRNN_SE3D_testing_data_for_dPCA.json
Data successfully imported from LRNN_SE3D/LRNN_SE3D_testing_data_for_dPCA.json
Differences found:
{'type_changes': {"root['testing_env_info']['sigma']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}, "root['testing_trial_performance']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}}}


## dPCA Export - LRNN-EI

In [ ]:

export_nested_dict_to_json(LRNN_EI_full_testing_data, filename= 'LRNN_EI/LRNN_EI_testing_data_for_dPCA.json')

temp_testing_structure = import_nested_dict_from_json(filename = 'LRNN_EI/LRNN_EI_testing_data_for_dPCA.json')

diff_from_io = compare_dicts(LRNN_EI_full_testing_data, temp_testing_structure)


Data successfully exported to LRNN_EI/LRNN_EI_testing_data_for_dPCA.json
Data successfully imported from LRNN_EI/LRNN_EI_testing_data_for_dPCA.json
Differences found:
{'type_changes': {"root['testing_env_info']['sigma']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}, "root['testing_trial_performance']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}}}


# dPCA - All Networks (Importing Plots and Displaying)

## dPCA Display Outs - LRNN-NSC

In [ ]:
import os
from IPython.display import display, Image

# Define the directory containing the images
image_directory = 'LRNN_NSC/LRNN_NSC_dpca_plots'

# Check if the directory exists
if not os.path.exists(image_directory):
    print(f"Directory '{image_directory}' not found.")
else:
    # List all files in the directory
    image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    if not image_files:
        print(f"No files found in '{image_directory}'.")
    else:
        print(f"Displaying images from '{image_directory}':")
        # Sort the files alphabetically for consistent order
        image_files.sort()
        for image_file in image_files:
            image_path = os.path.join(image_directory, image_file)
            print(f"\n--- {image_file} ---")
            try:
                display(Image(filename=image_path))
            except Exception as e:
                print(f"Could not display image {image_file}: {e}")

Directory 'LRNN_NSC/LRNN_NSC_dpca_plots' not found.


## dPCA Display Outs - LRNN-SE2D

In [ ]:
# Define the directory containing the images
image_directory = 'LRNN_SE2D/LRNN_SE2D_dpca_plots'

# Check if the directory exists
if not os.path.exists(image_directory):
    print(f"Directory '{image_directory}' not found.")
else:
    # List all files in the directory
    image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    if not image_files:
        print(f"No files found in '{image_directory}'.")
    else:
        print(f"Displaying images from '{image_directory}':")
        # Sort the files alphabetically for consistent order
        image_files.sort()
        for image_file in image_files:
            image_path = os.path.join(image_directory, image_file)
            print(f"\n--- {image_file} ---")
            try:
                display(Image(filename=image_path))
            except Exception as e:
                print(f"Could not display image {image_file}: {e}")

Directory 'LRNN_SE2D/LRNN_SE2D_dpca_plots' not found.


## dPCA Display Outs - LRNN-SE3d

In [ ]:
# Define the directory containing the images
image_directory = 'LRNN_SE3D/LRNN_SE3D_dpca_plots'

# Check if the directory exists
if not os.path.exists(image_directory):
    print(f"Directory '{image_directory}' not found.")
else:
    # List all files in the directory
    image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    if not image_files:
        print(f"No files found in '{image_directory}'.")
    else:
        print(f"Displaying images from '{image_directory}':")
        # Sort the files alphabetically for consistent order
        image_files.sort()
        for image_file in image_files:
            image_path = os.path.join(image_directory, image_file)
            print(f"\n--- {image_file} ---")
            try:
                display(Image(filename=image_path))
            except Exception as e:
                print(f"Could not display image {image_file}: {e}")

Directory 'LRNN_SE3D/LRNN_SE3D_dpca_plots' not found.


## dPCA Display Outs - LRNN-EI


In [ ]:
# Define the directory containing the images
image_directory = 'LRNN_EI/LRNN_EI_dpca_plots'

# Check if the directory exists
if not os.path.exists(image_directory):
    print(f"Directory '{image_directory}' not found.")
else:
    # List all files in the directory
    image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    if not image_files:
        print(f"No files found in '{image_directory}'.")
    else:
        print(f"Displaying images from '{image_directory}':")
        # Sort the files alphabetically for consistent order
        image_files.sort()
        for image_file in image_files:
            image_path = os.path.join(image_directory, image_file)
            print(f"\n--- {image_file} ---")
            try:
                display(Image(filename=image_path))
            except Exception as e:
                print(f"Could not display image {image_file}: {e}")

Directory 'LRNN_EI/LRNN_EI_dpca_plots' not found.


# PCA (All Networks)

## PCA - LRNN NSC

### Fit PCA - LRNN NSC

In [ ]:
# sample period data for all conds:
LRNN_NSC_sample_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='sample')


# delay period data for all conds:
LRNN_NSC_delay_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='delay')

# test period data for all conds:
LRNN_NSC_test_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='test')

# decision period data all conds:
LRNN_NSC_decision_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='decision')



In [ ]:
# Fit PCA for the sample period (1d)
LRNN_NSC_one_d_sample_pca = fit_pca_on_selected_data(LRNN_NSC_sample_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the sample period (2d)
LRNN_NSC_two_d_sample_pca = fit_pca_on_selected_data(LRNN_NSC_sample_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the delay period (1d)
LRNN_NSC_one_d_delay_pca = fit_pca_on_selected_data(LRNN_NSC_delay_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the delay period (2d)
LRNN_NSC_two_d_delay_pca = fit_pca_on_selected_data(LRNN_NSC_delay_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the test period (1d)
LRNN_NSC_one_d_test_pca = fit_pca_on_selected_data(LRNN_NSC_test_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the test period (2d)
LRNN_NSC_two_d_test_pca = fit_pca_on_selected_data(LRNN_NSC_test_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the decision period(1d)
LRNN_NSC_one_d_decision_pca = fit_pca_on_selected_data(LRNN_NSC_decision_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the decision period(2d)
LRNN_NSC_two_d_decision_pca = fit_pca_on_selected_data(LRNN_NSC_decision_period_trials_dict, pca_components=2, report_var_expls = False)

### Plot PCA - LRNN NSC

#### 1D PCA - LRNN NSC Plot

In [ ]:
plot_pca_trajectories(pca_object=LRNN_NSC_one_d_sample_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_sample_PCA_1D_full_trials.png')

Figure saved to LRNN_NSC/LRNN_NSC_sample_PCA_1D_full_trials.png


In [ ]:
plot_pca_trajectories(pca_object=LRNN_NSC_one_d_delay_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_delay_PCA_1D_full_trials.png')

Figure saved to LRNN_NSC/LRNN_NSC_delay_PCA_1D_full_trials.png


In [ ]:
plot_pca_trajectories(pca_object=LRNN_NSC_one_d_test_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_test_PCA_1D_full_trials.png')

Figure saved to LRNN_NSC/LRNN_NSC_test_PCA_1D_full_trials.png


In [ ]:
plot_pca_trajectories(pca_object=LRNN_NSC_one_d_decision_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_decision_PCA_1D_full_trials.png')

Figure saved to LRNN_NSC/LRNN_NSC_decision_PCA_1D_full_trials.png


#### 2D PCA - LRNN NSC Plot

In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NSC_two_d_sample_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_sample_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NSC/LRNN_NSC_sample_PCA_2D_full_trials.png


In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NSC_two_d_delay_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_delay_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NSC/LRNN_NSC_delay_PCA_2D_full_trials.png


In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NSC_two_d_test_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_test_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NSC/LRNN_NSC_test_PCA_2D_full_trials.png


In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NSC_two_d_decision_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_decision_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NSC/LRNN_NSC_decision_PCA_2D_full_trials.png


#### Add All Period PCA

In [ ]:
# all periods all conds (correct)
LRNN_NSC_all_periods_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NSC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False)

# Fit PCA for al period(1d)
LRNN_NSC_one_d_all_pca = fit_pca_on_selected_data(LRNN_NSC_all_periods_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for all period(2d)
LRNN_NSC_two_d_all_pca = fit_pca_on_selected_data(LRNN_NSC_all_periods_trials_dict, pca_components=2, report_var_expls = False)


plot_pca_trajectories(pca_object=LRNN_NSC_one_d_all_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_all_PCA_1D_full_trials.png')

plot_pca_trajectories_2d(pca_object=LRNN_NSC_two_d_all_pca, data_to_transform_dict=LRNN_NSC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NSC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NSC/LRNN_NSC_all_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NSC/LRNN_NSC_all_PCA_1D_full_trials.png
Figure saved to LRNN_NSC/LRNN_NSC_all_PCA_2D_full_trials.png


## PCA - LRNN SE2D

### Fit PCA - LRNN SE2D

In [ ]:
# sample period data for all conds:
LRNN_SE2D_sample_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='sample')


# delay period data for all conds:
LRNN_SE2D_delay_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='delay')

# test period data for all conds:
LRNN_SE2D_test_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='test')

# decision period data all conds:
LRNN_SE2D_decision_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='decision')



In [ ]:


# Fit PCA for the sample period (1d)
LRNN_SE2D_one_d_sample_pca = fit_pca_on_selected_data(LRNN_SE2D_sample_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the sample period (2d)
LRNN_SE2D_two_d_sample_pca = fit_pca_on_selected_data(LRNN_SE2D_sample_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the delay period (1d)
LRNN_SE2D_one_d_delay_pca = fit_pca_on_selected_data(LRNN_SE2D_delay_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the delay period (2d)
LRNN_SE2D_two_d_delay_pca = fit_pca_on_selected_data(LRNN_SE2D_delay_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the test period (1d)
LRNN_SE2D_one_d_test_pca = fit_pca_on_selected_data(LRNN_SE2D_test_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the test period (2d)
LRNN_SE2D_two_d_test_pca = fit_pca_on_selected_data(LRNN_SE2D_test_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the decision period(1d)
LRNN_SE2D_one_d_decision_pca = fit_pca_on_selected_data(LRNN_SE2D_decision_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the decision period(2d)
LRNN_SE2D_two_d_decision_pca = fit_pca_on_selected_data(LRNN_SE2D_decision_period_trials_dict, pca_components=2, report_var_expls = False)


### Plot PCA - LRNN SE2D

#### 1D PCA - LRNN SE2D Plot

In [ ]:

plot_pca_trajectories(pca_object=LRNN_SE2D_one_d_sample_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_sample_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_SE2D_one_d_delay_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_delay_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_SE2D_one_d_test_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_test_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_SE2D_one_d_decision_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_decision_PCA_1D_full_trials.png')


Figure saved to LRNN_SE2D/LRNN_SE2D_sample_PCA_1D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_delay_PCA_1D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_test_PCA_1D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_decision_PCA_1D_full_trials.png


#### 2D PCA - LRNN SE2D Plot

In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_SE2D_two_d_sample_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_sample_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_SE2D_two_d_delay_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_delay_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_SE2D_two_d_test_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_test_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_SE2D_two_d_decision_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_decision_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_SE2D/LRNN_SE2D_sample_PCA_2D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_delay_PCA_2D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_test_PCA_2D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_decision_PCA_2D_full_trials.png


#### Add All Period PCA

In [ ]:
# all periods all conds (correct)
LRNN_SE2D_all_periods_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE2D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False)

# Fit PCA for al period(1d)
LRNN_SE2D_one_d_all_pca = fit_pca_on_selected_data(LRNN_SE2D_all_periods_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for all period(2d)
LRNN_SE2D_two_d_all_pca = fit_pca_on_selected_data(LRNN_SE2D_all_periods_trials_dict, pca_components=2, report_var_expls = False)


plot_pca_trajectories(pca_object=LRNN_SE2D_one_d_all_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_all_PCA_1D_full_trials.png')

plot_pca_trajectories_2d(pca_object=LRNN_SE2D_two_d_all_pca, data_to_transform_dict=LRNN_SE2D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE2D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE2D/LRNN_SE2D_all_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_SE2D/LRNN_SE2D_all_PCA_1D_full_trials.png
Figure saved to LRNN_SE2D/LRNN_SE2D_all_PCA_2D_full_trials.png


## PCA - LRNN SE3D

### Fit PCA - LRNN SE3D

In [ ]:
# sample period data for all conds:
LRNN_SE3D_sample_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='sample')


# delay period data for all conds:
LRNN_SE3D_delay_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='delay')

# test period data for all conds:
LRNN_SE3D_test_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='test')

# decision period data all conds:
LRNN_SE3D_decision_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='decision')


# Fit PCA for the sample period (1d)
LRNN_SE3D_one_d_sample_pca = fit_pca_on_selected_data(LRNN_SE3D_sample_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the sample period (2d)
LRNN_SE3D_two_d_sample_pca = fit_pca_on_selected_data(LRNN_SE3D_sample_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the delay period (1d)
LRNN_SE3D_one_d_delay_pca = fit_pca_on_selected_data(LRNN_SE3D_delay_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the delay period (2d)
LRNN_SE3D_two_d_delay_pca = fit_pca_on_selected_data(LRNN_SE3D_delay_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the test period (1d)
LRNN_SE3D_one_d_test_pca = fit_pca_on_selected_data(LRNN_SE3D_test_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the test period (2d)
LRNN_SE3D_two_d_test_pca = fit_pca_on_selected_data(LRNN_SE3D_test_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the decision period(1d)
LRNN_SE3D_one_d_decision_pca = fit_pca_on_selected_data(LRNN_SE3D_decision_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the decision period(2d)
LRNN_SE3D_two_d_decision_pca = fit_pca_on_selected_data(LRNN_SE3D_decision_period_trials_dict, pca_components=2, report_var_expls = False)


### Plot PCA - LRNN SE3D

#### 1D PCA - LRNN SE3D Plot

In [ ]:
plot_pca_trajectories(pca_object=LRNN_SE3D_one_d_sample_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_sample_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_SE3D_one_d_delay_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_delay_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_SE3D_one_d_test_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_test_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_SE3D_one_d_decision_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_decision_PCA_1D_full_trials.png')


Figure saved to LRNN_SE3D/LRNN_SE3D_sample_PCA_1D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_delay_PCA_1D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_test_PCA_1D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_decision_PCA_1D_full_trials.png


#### 2D PCA - LRNN SE3D Plot

In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_SE3D_two_d_sample_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_sample_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_SE3D_two_d_delay_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'], save_filename='LRNN_SE3D/LRNN_SE3D_delay_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_SE3D_two_d_test_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_test_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_SE3D_two_d_decision_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_decision_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_SE3D/LRNN_SE3D_sample_PCA_2D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_delay_PCA_2D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_test_PCA_2D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_decision_PCA_2D_full_trials.png


#### Add All Period PCA

In [ ]:
# all periods all conds (correct)
LRNN_SE3D_all_periods_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_SE3D_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False)

# Fit PCA for al period(1d)
LRNN_SE3D_one_d_all_pca = fit_pca_on_selected_data(LRNN_SE3D_all_periods_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for all period(2d)
LRNN_SE3D_two_d_all_pca = fit_pca_on_selected_data(LRNN_SE3D_all_periods_trials_dict, pca_components=2, report_var_expls = False)


plot_pca_trajectories(pca_object=LRNN_SE3D_one_d_all_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_all_PCA_1D_full_trials.png')

plot_pca_trajectories_2d(pca_object=LRNN_SE3D_two_d_all_pca, data_to_transform_dict=LRNN_SE3D_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_SE3D_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_SE3D/LRNN_SE3D_all_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_SE3D/LRNN_SE3D_all_PCA_1D_full_trials.png
Figure saved to LRNN_SE3D/LRNN_SE3D_all_PCA_2D_full_trials.png


## PCA - LRNN EI


### Fit PCA - LRNN EI

In [ ]:
# sample period data for all conds:
LRNN_EI_sample_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='sample')


# delay period data for all conds:
LRNN_EI_delay_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='delay')

# test period data for all conds:
LRNN_EI_test_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='test')

# decision period data all conds:
LRNN_EI_decision_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='decision')

# all periods all conds (correct)
LRNN_EI_all_periods_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False)

In [ ]:
# Fit PCA for the sample period (1d)
LRNN_EI_one_d_sample_pca = fit_pca_on_selected_data(LRNN_EI_sample_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the sample period (2d)
LRNN_EI_two_d_sample_pca = fit_pca_on_selected_data(LRNN_EI_sample_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the delay period (1d)
LRNN_EI_one_d_delay_pca = fit_pca_on_selected_data(LRNN_EI_delay_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the delay period (2d)
LRNN_EI_two_d_delay_pca = fit_pca_on_selected_data(LRNN_EI_delay_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the test period (1d)
LRNN_EI_one_d_test_pca = fit_pca_on_selected_data(LRNN_EI_test_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the test period (2d)
LRNN_EI_two_d_test_pca = fit_pca_on_selected_data(LRNN_EI_test_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the decision period(1d)
LRNN_EI_one_d_decision_pca = fit_pca_on_selected_data(LRNN_EI_decision_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the decision period(2d)
LRNN_EI_two_d_decision_pca = fit_pca_on_selected_data(LRNN_EI_decision_period_trials_dict, pca_components=2, report_var_expls = False)



# all periods all conds (correct)
LRNN_EI_all_periods_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_EI_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False)

# Fit PCA for al period(1d)
LRNN_EI_one_d_all_pca = fit_pca_on_selected_data(LRNN_EI_all_periods_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for all period(2d)
LRNN_EI_two_d_all_pca = fit_pca_on_selected_data(LRNN_EI_all_periods_trials_dict, pca_components=2, report_var_expls = False)


plot_pca_trajectories(pca_object=LRNN_EI_one_d_all_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_EI/LRNN_EI_all_PCA_1D_full_trials.png')

plot_pca_trajectories_2d(pca_object=LRNN_EI_two_d_all_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_EI/LRNN_EI_all_PCA_2D_full_trials.png',
                             add_phase_markers=True)



Figure saved to LRNN_EI/LRNN_EI_all_PCA_1D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_all_PCA_2D_full_trials.png


### Plot PCA - LRNN EI

#### 1D PCA - LRNN EI Plot

In [ ]:

plot_pca_trajectories(pca_object=LRNN_EI_one_d_sample_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_EI/LRNN_EI_sample_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_EI_one_d_delay_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_EI/LRNN_EI_delay_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_EI_one_d_test_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_EI/LRNN_EI_test_PCA_1D_full_trials.png')

plot_pca_trajectories(pca_object=LRNN_EI_one_d_decision_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_EI/LRNN_EI_decision_PCA_1D_full_trials.png')


Figure saved to LRNN_EI/LRNN_EI_sample_PCA_1D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_delay_PCA_1D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_test_PCA_1D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_decision_PCA_1D_full_trials.png


#### 2D PCA - LRNN EI Plot

In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_EI_two_d_sample_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_EI/LRNN_EI_sample_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_EI_two_d_delay_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_EI/LRNN_EI_delay_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_EI_two_d_test_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_EI/LRNN_EI_test_PCA_2D_full_trials.png',
                             add_phase_markers=True)

plot_pca_trajectories_2d(pca_object=LRNN_EI_two_d_decision_pca, data_to_transform_dict=LRNN_EI_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_EI_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_EI/LRNN_EI_decision_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_EI/LRNN_EI_sample_PCA_2D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_delay_PCA_2D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_test_PCA_2D_full_trials.png
Figure saved to LRNN_EI/LRNN_EI_decision_PCA_2D_full_trials.png


# Selectivity and Leisoning - All BICs

## Variables for all

In [ ]:
leisoning_testing_data_set = dataset_testing

## LRNN-NSC

### Sample Stimulus Selective

In [ ]:
# Data from Partition
LRNN_NSC_sample_stim_1_data = LRNN_NSC_testing_data_0p0_sample
LRNN_NSC_sample_stim_2_data = LRNN_NSC_testing_data_3p1_sample
LRNN_NSC_sample_stim_data_dicts = [LRNN_NSC_sample_stim_1_data['testing_trial_and_activity'], LRNN_NSC_sample_stim_2_data['testing_trial_and_activity']]

LRNN_NSC_1D_PCA_sssl = LRNN_NSC_one_d_sample_pca
LRNN_NSC_2D_PCA_sssl = LRNN_NSC_two_d_sample_pca


In [ ]:
LRNN_NSC_sample_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_NSC_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_NSC_sample_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_NSC_1D_PCA_sssl, fig_file_name = 'LRNN_NSC/leisoned_unleisoned_LRNN_NSC_SS' , file_ext='.png', unleisoned_PCA_2d = LRNN_NSC_2D_PCA_sssl, return_results = True)

Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(17), np.int64(4), np.int64(51), np.int64(18), np.int64(55), np.int64(31)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.4955
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 49.55 %
Performance Difference: 50.449999999999996 %
Figure saved to LRNN_NSC/leisoned_unleisoned_LRNN_NSC_SS_1d_pca.png
Figure saved to LRNN_NSC/leisoned_unleisoned_LRNN_NSC_SS_2d_pca.png


### Test Stimulus Selective

In [ ]:
# Data from Partition
LRNN_NSC_test_stim_1_data = LRNN_NSC_testing_data_0p0_test
LRNN_NSC_test_stim_2_data = LRNN_NSC_testing_data_3p1_test
LRNN_NSC_test_stim_data_dicts = [LRNN_NSC_test_stim_1_data['testing_trial_and_activity'], LRNN_NSC_test_stim_2_data['testing_trial_and_activity']]

LRNN_NSC_1D_PCA_tssl = LRNN_NSC_one_d_test_pca
LRNN_NSC_2D_PCA_tssl = LRNN_NSC_two_d_test_pca


In [ ]:
LRNN_NSC_test_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_NSC_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_NSC_test_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_NSC_1D_PCA_tssl, fig_file_name = 'LRNN_NSC/leisoned_unleisoned_LRNN_NSC_TS' , file_ext='.png', unleisoned_PCA_2d = LRNN_NSC_2D_PCA_tssl, return_results = True)

Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(37), np.int64(47), np.int64(52), np.int64(51), np.int64(5), np.int64(44)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.4905
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 49.05 %
Performance Difference: 50.95 %
Figure saved to LRNN_NSC/leisoned_unleisoned_LRNN_NSC_TS_1d_pca.png
Figure saved to LRNN_NSC/leisoned_unleisoned_LRNN_NSC_TS_2d_pca.png


### Match - No Match Selective

In [ ]:
# Data from Partition
LRNN_NSC_match_modality_1_data = LRNN_NSC_testing_data_matching
LRNN_NSC_match_modality_2_data = LRNN_NSC_testing_data_non_matching
LRNN_NSC_match_modality_data_dicts = [LRNN_NSC_match_modality_1_data['testing_trial_and_activity'], LRNN_NSC_match_modality_2_data['testing_trial_and_activity']]

LRNN_NSC_1D_PCA_mmsl = LRNN_NSC_one_d_decision_pca
LRNN_NSC_2D_PCA_mmsl = LRNN_NSC_two_d_decision_pca

In [ ]:


LRNN_NSC_match_modality_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_NSC_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_NSC_match_modality_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_NSC_1D_PCA_mmsl, fig_file_name = 'LRNN_NSC/leisoned_unleisoned_LRNN_NSC_MM' , file_ext='.png', unleisoned_PCA_2d = LRNN_NSC_2D_PCA_mmsl, return_results = True)



Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(35), np.int64(15), np.int64(38), np.int64(14), np.int64(59), np.int64(8)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.25
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 25.0 %
Performance Difference: 75.0 %
Figure saved to LRNN_NSC/leisoned_unleisoned_LRNN_NSC_MM_1d_pca.png
Figure saved to LRNN_NSC/leisoned_unleisoned_LRNN_NSC_MM_2d_pca.png


## LRNN-SE2D

### Sample Stimulus Selective

In [ ]:
# Data from Partition
LRNN_SE2D_sample_stim_1_data = LRNN_SE2D_testing_data_0p0_sample
LRNN_SE2D_sample_stim_2_data = LRNN_SE2D_testing_data_3p1_sample
LRNN_SE2D_sample_stim_data_dicts = [LRNN_SE2D_sample_stim_1_data['testing_trial_and_activity'], LRNN_SE2D_sample_stim_2_data['testing_trial_and_activity']]

LRNN_SE2D_1D_PCA_sssl = LRNN_SE2D_one_d_sample_pca
LRNN_SE2D_2D_PCA_sssl = LRNN_SE2D_two_d_sample_pca


In [ ]:
LRNN_SE2D_sample_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_SE2D_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_SE2D_sample_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_SE2D_1D_PCA_sssl, fig_file_name = 'LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_SS' , file_ext='.png', unleisoned_PCA_2d = LRNN_SE2D_2D_PCA_sssl, return_results = True)


Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(63), np.int64(27), np.int64(15), np.int64(2), np.int64(54), np.int64(48)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.4935
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 49.35 %
Performance Difference: 50.64999999999999 %
Figure saved to LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_SS_1d_pca.png
Figure saved to LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_SS_2d_pca.png


### Test Stimulus Selective

In [ ]:

# Data from Partition
LRNN_SE2D_test_stim_1_data = LRNN_SE2D_testing_data_0p0_test
LRNN_SE2D_test_stim_2_data = LRNN_SE2D_testing_data_3p1_test
LRNN_SE2D_test_stim_data_dicts = [LRNN_SE2D_test_stim_1_data['testing_trial_and_activity'], LRNN_SE2D_test_stim_2_data['testing_trial_and_activity']]

LRNN_SE2D_1D_PCA_tssl = LRNN_SE2D_one_d_test_pca
LRNN_SE2D_2D_PCA_tssl = LRNN_SE2D_two_d_test_pca

In [ ]:


LRNN_SE2D_test_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_SE2D_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_SE2D_test_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_SE2D_1D_PCA_tssl, fig_file_name = 'LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_TS' , file_ext='.png', unleisoned_PCA_2d = LRNN_SE2D_2D_PCA_tssl, return_results = True)


Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(10), np.int64(23), np.int64(50), np.int64(58), np.int64(18), np.int64(14)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.5085
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 50.849999999999994 %
Performance Difference: 49.150000000000006 %
Figure saved to LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_TS_1d_pca.png
Figure saved to LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_TS_2d_pca.png


### Match - No Match Selective

In [ ]:
# Data from Partition

# Data from Partition
LRNN_SE2D_match_modality_1_data = LRNN_SE2D_testing_data_matching
LRNN_SE2D_match_modality_2_data = LRNN_SE2D_testing_data_non_matching
LRNN_SE2D_match_modality_data_dicts = [LRNN_SE2D_match_modality_1_data['testing_trial_and_activity'], LRNN_SE2D_match_modality_2_data['testing_trial_and_activity']]

LRNN_SE2D_1D_PCA_mmsl = LRNN_SE2D_one_d_decision_pca
LRNN_SE2D_2D_PCA_mmsl = LRNN_SE2D_two_d_decision_pca


In [ ]:

LRNN_SE2D_match_modality_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_SE2D_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_SE2D_match_modality_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_SE2D_1D_PCA_mmsl, fig_file_name = 'LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_MM' , file_ext='.png', unleisoned_PCA_2d = LRNN_SE2D_2D_PCA_mmsl, return_results = True)


Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(62), np.int64(47), np.int64(21), np.int64(45), np.int64(35), np.int64(61)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.504
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 50.4 %
Performance Difference: 49.6 %
Figure saved to LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_MM_1d_pca.png
Figure saved to LRNN_SE2D/leisoned_unleisoned_LRNN_SE2D_MM_2d_pca.png


## LRNN-SE3D

### Sample Stimulus Selective

In [ ]:
# Data from Partition
LRNN_SE3D_sample_stim_1_data = LRNN_SE3D_testing_data_0p0_sample
LRNN_SE3D_sample_stim_2_data = LRNN_SE3D_testing_data_3p1_sample
LRNN_SE3D_sample_stim_data_dicts = [LRNN_SE3D_sample_stim_1_data['testing_trial_and_activity'], LRNN_SE3D_sample_stim_2_data['testing_trial_and_activity']]

LRNN_SE3D_1D_PCA_sssl = LRNN_SE3D_one_d_sample_pca
LRNN_SE3D_2D_PCA_sssl = LRNN_SE3D_two_d_sample_pca



In [ ]:


LRNN_SE3D_sample_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_SE3D_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_SE3D_sample_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_SE3D_1D_PCA_sssl, fig_file_name = 'LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_SS' , file_ext='.png', unleisoned_PCA_2d = LRNN_SE3D_2D_PCA_sssl, return_results = True)


Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(19), np.int64(56), np.int64(59), np.int64(28), np.int64(31), np.int64(35)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.502
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 50.2 %
Performance Difference: 49.8 %
Figure saved to LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_SS_1d_pca.png
Figure saved to LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_SS_2d_pca.png


### Test Stimulus Selective

In [ ]:

# Data from Partition
LRNN_SE3D_test_stim_1_data = LRNN_SE3D_testing_data_0p0_test
LRNN_SE3D_test_stim_2_data = LRNN_SE3D_testing_data_3p1_test
LRNN_SE3D_test_stim_data_dicts = [LRNN_SE3D_test_stim_1_data['testing_trial_and_activity'], LRNN_SE3D_test_stim_2_data['testing_trial_and_activity']]

LRNN_SE3D_1D_PCA_tssl = LRNN_SE3D_one_d_test_pca
LRNN_SE3D_2D_PCA_tssl = LRNN_SE3D_two_d_test_pca


In [ ]:


LRNN_SE3D_test_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_SE3D_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_SE3D_test_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_SE3D_1D_PCA_tssl, fig_file_name = 'LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_TS' , file_ext='.png', unleisoned_PCA_2d = LRNN_SE3D_2D_PCA_tssl, return_results = True)


Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(1), np.int64(15), np.int64(54), np.int64(50), np.int64(22), np.int64(30)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.7325
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 73.25 %
Performance Difference: 26.749999999999996 %
Figure saved to LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_TS_1d_pca.png
Figure saved to LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_TS_2d_pca.png


### Match - No Match Selective

In [ ]:

# Data from Partition
LRNN_SE3D_match_modality_1_data = LRNN_SE3D_testing_data_matching
LRNN_SE3D_match_modality_2_data = LRNN_SE3D_testing_data_non_matching
LRNN_SE3D_match_modality_data_dicts = [LRNN_SE3D_match_modality_1_data['testing_trial_and_activity'], LRNN_SE3D_match_modality_2_data['testing_trial_and_activity']]

LRNN_SE3D_1D_PCA_mmsl = LRNN_SE3D_one_d_decision_pca
LRNN_SE3D_2D_PCA_mmsl = LRNN_SE3D_two_d_decision_pca


In [ ]:


LRNN_SE3D_match_modality_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_SE3D_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_SE3D_match_modality_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_SE3D_1D_PCA_mmsl, fig_file_name = 'LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_MM' , file_ext='.png', unleisoned_PCA_2d = LRNN_SE3D_2D_PCA_mmsl, return_results = True)


Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(24), np.int64(21), np.int64(13), np.int64(4), np.int64(43), np.int64(55)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.503
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 50.3 %
Performance Difference: 49.7 %
Figure saved to LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_MM_1d_pca.png
Figure saved to LRNN_SE3D/leisoned_unleisoned_LRNN_SE3D_MM_2d_pca.png


## LRNN-EI

### Sample Stimulus Selective

In [ ]:

# Data from Partition
LRNN_EI_sample_stim_1_data = LRNN_EI_testing_data_0p0_sample
LRNN_EI_sample_stim_2_data = LRNN_EI_testing_data_3p1_sample
LRNN_EI_sample_stim_data_dicts = [LRNN_EI_sample_stim_1_data['testing_trial_and_activity'], LRNN_EI_sample_stim_2_data['testing_trial_and_activity']]

LRNN_EI_1D_PCA_sssl = LRNN_EI_one_d_sample_pca
LRNN_EI_2D_PCA_sssl = LRNN_EI_two_d_sample_pca

In [ ]:

LRNN_EI_sample_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_EI_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_EI_sample_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_EI_1D_PCA_sssl, fig_file_name = 'LRNN_EI/leisoned_unleisoned_LRNN_EI_SS' , file_ext='.png', unleisoned_PCA_2d = LRNN_EI_2D_PCA_sssl, return_results = True)


Created a deep copy of the network.
Found h2h layer: EIRecLinear()
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(56), np.int64(58), np.int64(48), np.int64(52), np.int64(51), np.int64(14)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.4905
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 49.05 %
Performance Difference: 50.95 %
Figure saved to LRNN_EI/leisoned_unleisoned_LRNN_EI_SS_1d_pca.png
Figure saved to LRNN_EI/leisoned_unleisoned_LRNN_EI_SS_2d_pca.png


### Test Stimulus Selective

In [ ]:
# Data from Partition
LRNN_EI_test_stim_1_data = LRNN_EI_testing_data_0p0_test
LRNN_EI_test_stim_2_data = LRNN_EI_testing_data_3p1_test
LRNN_EI_test_stim_data_dicts = [LRNN_EI_test_stim_1_data['testing_trial_and_activity'], LRNN_EI_test_stim_2_data['testing_trial_and_activity']]

LRNN_EI_1D_PCA_tssl = LRNN_EI_one_d_test_pca
LRNN_EI_2D_PCA_tssl = LRNN_EI_two_d_test_pca


In [ ]:


LRNN_EI_test_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_EI_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_EI_test_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_EI_1D_PCA_tssl, fig_file_name = 'LRNN_EI/leisoned_unleisoned_LRNN_EI_TS' , file_ext='.png', unleisoned_PCA_2d = LRNN_EI_2D_PCA_tssl, return_results = True)


Created a deep copy of the network.
Found h2h layer: EIRecLinear()
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(55), np.int64(3), np.int64(38), np.int64(37), np.int64(5), np.int64(29)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.7545
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 75.44999999999999 %
Performance Difference: 24.550000000000004 %
Figure saved to LRNN_EI/leisoned_unleisoned_LRNN_EI_TS_1d_pca.png
Figure saved to LRNN_EI/leisoned_unleisoned_LRNN_EI_TS_2d_pca.png


### Match - No Match Selective

In [ ]:

# Data from Partition
LRNN_EI_match_modality_1_data = LRNN_EI_testing_data_matching
LRNN_EI_match_modality_2_data = LRNN_EI_testing_data_non_matching
LRNN_EI_match_modality_data_dicts = [LRNN_EI_match_modality_1_data['testing_trial_and_activity'], LRNN_EI_match_modality_2_data['testing_trial_and_activity']]

LRNN_EI_1D_PCA_mmsl = LRNN_EI_one_d_decision_pca
LRNN_EI_2D_PCA_mmsl = LRNN_EI_two_d_decision_pca


In [ ]:


LRNN_EI_match_modality_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_EI_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_EI_match_modality_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_EI_1D_PCA_mmsl, fig_file_name = 'LRNN_EI/leisoned_unleisoned_LRNN_EI_MM' , file_ext='.png', unleisoned_PCA_2d = LRNN_EI_2D_PCA_mmsl, return_results = True)


Created a deep copy of the network.
Found h2h layer: EIRecLinear()
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(62), np.int64(18), np.int64(26), np.int64(2), np.int64(12), np.int64(44)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.5105
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 51.05 %
Performance Difference: 48.95 %
Figure saved to LRNN_EI/leisoned_unleisoned_LRNN_EI_MM_1d_pca.png
Figure saved to LRNN_EI/leisoned_unleisoned_LRNN_EI_MM_2d_pca.png


# FPA

## Required Package Inits

In [ ]:
import sys
sys.path.append('/fixed-point-finder-master')
%cd fixed-point-finder-master
from FixedPointFinderTorch import FixedPointFinderTorch as FixedPointFinder
import torch
%cd ..

/content/fixed-point-finder-master
/content


# CHANGES TO CODE FOR BIC RNNs

## Defining a Testing Function to collect neuron states for initialising Initial Conditions

# FPF Implementation for BIC Networks

In [ ]:
fixation_input_array = np.array([[1.0,0.0,0.0]])
delay_input_array = np.array([[1.0,0.0,0.0]])
stim_pi_input_array = np.array([[1.0,0.0,1.0]])
stim_0_input_array = np.array([[1.0,1.0,0.0]])
response_input_array = np.array([[0.0,0.0,0.0]])

## Wrapper for BIC RNN Layers

In [ ]:
class FixedPointRNNWrapper_BIC(torch.nn.Module):
    def __init__(self, rnn, batch_first=False):
        super(FixedPointRNNWrapper_BIC, self).__init__()
        self.rnn = rnn
        self.batch_first = batch_first  # Ensure this matches your RNN's setting

    def forward(self, input, hidden):
        # # Squeeze the extra dimension from hidden state
        # # Hidden shape transforms from [1, batch_size, hidden_size] to [batch_size, hidden_size]
        # hidden = hidden.squeeze(0)

        # # EI-RNN expects inputs of shape [seq_len, batch_size, input_size]
        # # Since we have seq_len=1, input shape is already correct

        # Forward pass through your EI-RNN
        output, hidden = self.rnn.forward_helper_fpf(input, hidden)

        # # Unsqueeze hidden to match FixedPointFinder's expectation
        # # Hidden shape transforms from [batch_size, hidden_size] to [1, batch_size, hidden_size]
        # hidden = hidden.unsqueeze(0)

        # Return None for output as per FixedPointFinder's requirement # not a requirement
        return None, hidden

## FPF - LRNN-NSC

In [ ]:
# Do state tracked testing trials
# LRNN_NSC_STATETRACKED_full_testing_data = bic_testing_w_state_tracking(network=LRNN_NSC_TRAINED, dataset_to_evaluate=dataset_testing, num_trials=200)
LRNN_NSC_STATETRACKED_full_testing_data = LRNN_NSC_full_testing_data

In [ ]:
state_tracked_env_info = LRNN_NSC_STATETRACKED_full_testing_data['testing_env_info']

In [ ]:
LRNN_NSC_FOR_FPF = FixedPointRNNWrapper_BIC(rnn=LRNN_NSC_TRAINED.rnn)

In [ ]:
LRNN_NSC_two_d_delay_pca = LRNN_NSC_two_d_all_pca

### FPF LRNN-NSC Sample Period

In [ ]:
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NSC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NSC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NSC_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'sample_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NSC/LRNN_NSC_Sample_Stim0_delaypca.png', title_for_plot = 'Sample Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NSC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NSC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NSC_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'sample_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NSC/LRNN_NSC_Sample_Stimpi_delaypca.png', title_for_plot = 'Sample Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		1033 iters
		q = 1.87e-05 +/- 4.58e-04
		dq = 0.00e+00 +/- 0.00e+00
		learning rate = 9.72e-06
		avg iter time = 2.78e-03 sec
	Identified 2 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 2).
	Computing recurrent Jacobian at 2 unique fixed points.
	Computing input Jacobian at 2 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(2, 64)
(1200, 64)
Figure saved to LRNN_NSC/LRNN_NSC_Sample_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Termin

### FPF LRNN-NSC Delay Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NSC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NSC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NSC_two_d_delay_pca, ic_period = 'delay', fixed_input_array_ = delay_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NSC/LRNN_NSC_Delay_delaypca.png', title_for_plot = 'Delay Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		6110 iters
		q = 1.22e-12 +/- 7.41e-12
		dq = 2.60e-17 +/- 1.38e-16
		learning rate = 7.96e-05
		avg iter time = 3.19e-03 sec
	Identified 3 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 3).
	Computing recurrent Jacobian at 3 unique fixed points.
	Computing input Jacobian at 3 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(3, 64)
(1200, 64)
Figure saved to LRNN_NSC/LRNN_NSC_Delay_delaypca.png


### FPF LRNN-NSC - Test Period

In [ ]:
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NSC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NSC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NSC_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'test_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NSC/LRNN_NSC_Test_Stim0_delaypca.png', title_for_plot = 'Test Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NSC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NSC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NSC_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'test_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NSC/LRNN_NSC_Test_Stimpi_delaypca.png', title_for_plot = 'Test Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		842 iters
		q = 9.36e-06 +/- 3.24e-04
		dq = 3.03e-17 +/- 2.29e-16
		learning rate = 5.89e-04
		avg iter time = 2.69e-03 sec
	Identified 2 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 2).
	Computing recurrent Jacobian at 2 unique fixed points.
	Computing input Jacobian at 2 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(2, 64)
(1200, 64)
Figure saved to LRNN_NSC/LRNN_NSC_Test_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminati

### FPF LRNN-NSC - Decision Period

In [ ]:
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NSC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NSC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NSC_two_d_delay_pca, ic_period = 'decision', fixed_input_array_ = response_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NSC/LRNN_NSC_Decision_delaypca.png', title_for_plot = 'Decision Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 7.84e-04 +/- 1.06e-03
		dq = 7.62e-10 +/- 1.29e-09
		learning rate = 1.97e-07
		avg iter time = 3.27e-03 sec
	Identified 114 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 114).
	Computing recurrent Jacobian at 114 unique fixed points.
	Computing input Jacobian at 114 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(114, 64)
(1200, 64)
Figure saved to LRNN_NSC/LRNN_NSC_Decision_delaypca.png


## FPF - LRNN-SE2D

In [ ]:
LRNN_SE2D_STATETRACKED_full_testing_data = LRNN_SE2D_full_testing_data

state_tracked_env_info = LRNN_SE2D_STATETRACKED_full_testing_data['testing_env_info']

LRNN_SE2D_FOR_FPF = FixedPointRNNWrapper_BIC(rnn=LRNN_SE2D_TRAINED.rnn)

In [ ]:
LRNN_SE2D_two_d_delay_pca = LRNN_SE2D_two_d_all_pca

### FPF LRNN-SE2D Sample Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE2D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE2D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE2D_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'sample_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE2D/LRNN_SE2D_Sample_Stim0_delaypca.png', title_for_plot = 'Sample Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE2D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE2D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE2D_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'sample_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE2D/LRNN_SE2D_Sample_Stimpi_delaypca.png', title_for_plot = 'Sample Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 1.18e-04 +/- 1.87e-04
		dq = 1.12e-10 +/- 2.53e-10
		learning rate = 1.97e-07
		avg iter time = 4.13e-03 sec
	Identified 32 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 32).
	Computing recurrent Jacobian at 32 unique fixed points.
	Computing input Jacobian at 32 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(32, 64)
(1200, 64)
Figure saved to LRNN_SE2D/LRNN_SE2D_Sample_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desi

### FPF LRNN-SE2D Delay Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE2D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE2D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE2D_two_d_delay_pca, ic_period = 'delay', fixed_input_array_ = delay_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE2D/LRNN_SE2D_Delay_delaypca.png', title_for_plot = 'Delay Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 1.38e-04 +/- 2.39e-04
		dq = 1.82e-10 +/- 3.13e-10
		learning rate = 1.97e-07
		avg iter time = 3.75e-03 sec
	Identified 113 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 113).
	Computing recurrent Jacobian at 113 unique fixed points.
	Computing input Jacobian at 113 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(113, 64)
(1200, 64)
Figure saved to LRNN_SE2D/LRNN_SE2D_Delay_delaypca.png


### FPF LRNN-SE2D - Test Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE2D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE2D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE2D_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'test_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE2D/LRNN_SE2D_Test_Stim0_delaypca.png', title_for_plot = 'Test Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE2D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE2D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE2D_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'test_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE2D/LRNN_SE2D_Test_Stimpi_delaypca.png', title_for_plot = 'Test Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 1.89e-04 +/- 2.69e-04
		dq = 1.51e-10 +/- 3.63e-10
		learning rate = 1.97e-07
		avg iter time = 3.99e-03 sec
	Identified 31 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 31).
	Computing recurrent Jacobian at 31 unique fixed points.
	Computing input Jacobian at 31 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(31, 64)
(1200, 64)
Figure saved to LRNN_SE2D/LRNN_SE2D_Test_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached

### FPF LRNN-SE2D - Decision Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE2D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE2D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE2D_two_d_delay_pca, ic_period = 'decision', fixed_input_array_ = response_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE2D/LRNN_SE2D_Decision_delaypca.png', title_for_plot = 'Decision Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 1.87e-04 +/- 1.18e-04
		dq = 2.59e-10 +/- 3.69e-10
		learning rate = 1.97e-07
		avg iter time = 3.58e-03 sec
	Identified 97 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 97).
	Computing recurrent Jacobian at 97 unique fixed points.
	Computing input Jacobian at 97 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(97, 64)
(1200, 64)
Figure saved to LRNN_SE2D/LRNN_SE2D_Decision_delaypca.png


## FPF - LRNN-SE3D

In [ ]:
LRNN_SE3D_STATETRACKED_full_testing_data = LRNN_SE3D_full_testing_data

state_tracked_env_info = LRNN_SE3D_STATETRACKED_full_testing_data['testing_env_info']

LRNN_SE3D_FOR_FPF = FixedPointRNNWrapper_BIC(rnn=LRNN_SE3D_TRAINED.rnn)


In [ ]:
LRNN_SE3D_two_d_delay_pca = LRNN_SE3D_two_d_all_pca

### FPF LRNN-SE3D Sample Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE3D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE3D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE3D_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'sample_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE3D/LRNN_SE3D_Sample_Stim0_delaypca.png', title_for_plot = 'Sample Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE3D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE3D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE3D_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'sample_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE3D/LRNN_SE3D_Sample_Stimpi_delaypca.png', title_for_plot = 'Sample Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		1696 iters
		q = 1.71e-03 +/- 4.10e-03
		dq = 0.00e+00 +/- 0.00e+00
		learning rate = 1.97e-07
		avg iter time = 3.38e-03 sec
	Identified 12 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 12).
	Computing recurrent Jacobian at 12 unique fixed points.
	Computing input Jacobian at 12 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(12, 64)
(1200, 64)
Figure saved to LRNN_SE3D/LRNN_SE3D_Sample_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired

### FPF LRNN-SE3D Delay Period

In [ ]:


run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE3D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE3D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE3D_two_d_delay_pca, ic_period = 'delay', fixed_input_array_ = delay_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE3D/LRNN_SE3D_Delay_delaypca.png', title_for_plot = 'Delay Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 3.30e-07 +/- 5.99e-07
		dq = 2.66e-12 +/- 4.29e-12
		learning rate = 1.97e-07
		avg iter time = 4.11e-03 sec
	Identified 78 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 78).
	Computing recurrent Jacobian at 78 unique fixed points.
	Computing input Jacobian at 78 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(78, 64)
(1200, 64)
Figure saved to LRNN_SE3D/LRNN_SE3D_Delay_delaypca.png


### FPF LRNN-SE3D - Test Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE3D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE3D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE3D_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'test_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE3D/LRNN_SE3D_Test_Stim0_delaypca.png', title_for_plot = 'Test Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE3D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE3D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE3D_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'test_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE3D/LRNN_SE3D_Test_Stimpi_delaypca.png', title_for_plot = 'Test Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		1354 iters
		q = 1.05e-03 +/- 3.31e-03
		dq = 0.00e+00 +/- 0.00e+00
		learning rate = 1.97e-07
		avg iter time = 4.39e-03 sec
	Identified 11 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 11).
	Computing recurrent Jacobian at 11 unique fixed points.
	Computing input Jacobian at 11 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(11, 64)
(1200, 64)
Figure saved to LRNN_SE3D/LRNN_SE3D_Test_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired t

### FPF LRNN-SE3D - Decision Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_SE3D_FOR_FPF, testing_trial_data_for_fpa=LRNN_SE3D_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_SE3D_two_d_delay_pca, ic_period = 'decision', fixed_input_array_ = response_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_SE3D/LRNN_SE3D_Decision_delaypca.png', title_for_plot = 'Decision Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 2.27e-04 +/- 2.47e-04
		dq = 1.86e-10 +/- 3.35e-10
		learning rate = 1.97e-07
		avg iter time = 3.64e-03 sec
	Identified 101 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 101).
	Computing recurrent Jacobian at 101 unique fixed points.
	Computing input Jacobian at 101 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(101, 64)
(1200, 64)
Figure saved to LRNN_SE3D/LRNN_SE3D_Decision_delaypca.png


## FPF - LRNN-EI

In [ ]:
LRNN_EI_STATETRACKED_full_testing_data = LRNN_EI_full_testing_data

state_tracked_env_info = LRNN_EI_STATETRACKED_full_testing_data['testing_env_info']

LRNN_EI_FOR_FPF = FixedPointRNNWrapper_BIC(rnn=LRNN_EI_TRAINED.rnn)

In [ ]:
LRNN_EI_two_d_delay_pca = LRNN_EI_two_d_all_pca

### FPF LRNN-EI Sample Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_EI_FOR_FPF, testing_trial_data_for_fpa=LRNN_EI_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_EI_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'sample_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_EI/LRNN_EI_Sample_Stim0_delaypca.png', title_for_plot = 'Sample Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_EI_FOR_FPF, testing_trial_data_for_fpa=LRNN_EI_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_EI_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'sample_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_EI/LRNN_EI_Sample_Stimpi_delaypca.png', title_for_plot = 'Sample Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		313 iters
		q = 9.57e-14 +/- 3.77e-14
		dq = 5.05e-14 +/- 1.16e-12
		learning rate = 4.63e-01
		avg iter time = 3.47e-03 sec
	Identified 1 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 1).
	Computing recurrent Jacobian at 1 unique fixed points.
	Computing input Jacobian at 1 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(1, 64)
(1200, 64)
Figure saved to LRNN_EI/LRNN_EI_Sample_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance

### FPF LRNN-EI Delay Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_EI_FOR_FPF, testing_trial_data_for_fpa=LRNN_EI_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_EI_two_d_delay_pca, ic_period = 'delay', fixed_input_array_ = delay_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_EI/LRNN_EI_Delay_delaypca.png', title_for_plot = 'Delay Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		1318 iters
		q = 7.60e-06 +/- 2.78e-05
		dq = 0.00e+00 +/- 0.00e+00
		learning rate = 1.97e-07
		avg iter time = 5.69e-03 sec
	Identified 19 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 19).
	Computing recurrent Jacobian at 19 unique fixed points.
	Computing input Jacobian at 19 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(19, 64)
(1200, 64)
Figure saved to LRNN_EI/LRNN_EI_Delay_delaypca.png


### FPF LRNN-EI - Test Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_EI_FOR_FPF, testing_trial_data_for_fpa=LRNN_EI_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_EI_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'test_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_EI/LRNN_EI_Test_Stim0_delaypca.png', title_for_plot = 'Test Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_EI_FOR_FPF, testing_trial_data_for_fpa=LRNN_EI_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_EI_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'test_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_EI/LRNN_EI_Test_Stimpi_delaypca.png', title_for_plot = 'Test Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		856 iters
		q = 4.28e-04 +/- 2.44e-03
		dq = 0.00e+00 +/- 0.00e+00
		learning rate = 7.15e-06
		avg iter time = 3.55e-03 sec
	Identified 4 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 4).
	Computing recurrent Jacobian at 4 unique fixed points.
	Computing input Jacobian at 4 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(4, 64)
(1200, 64)
Figure saved to LRNN_EI/LRNN_EI_Test_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.


### FPF LRNN-EI - Decision Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_EI_FOR_FPF, testing_trial_data_for_fpa=LRNN_EI_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_EI_two_d_delay_pca, ic_period = 'decision', fixed_input_array_ = response_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_EI/LRNN_EI_Decision_delaypca.png', title_for_plot = 'Decision Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100)



Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 1.61e-04 +/- 2.61e-04
		dq = 2.19e-10 +/- 3.32e-10
		learning rate = 1.97e-07
		avg iter time = 4.19e-03 sec
	Identified 186 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 186).
	Computing recurrent Jacobian at 186 unique fixed points.
	Computing input Jacobian at 186 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(186, 64)
(1200, 64)
Figure saved to LRNN_EI/LRNN_EI_Decision_delaypca.png


In [ ]:
# while True:
#   continue

# NBIC WORK (BROUGHT IN FROM OTHER NOTEBOOK - SHOULD STILL WORK THO)

# DEFINING NBIC NETWORK (LAST UPDATE : 1507_4:07pm)

## LeakyRNN Layer - No BICs


In [ ]:
# Leaky RNN
class LeakyRNN(nn.Module):
    """Leaky RNN.

    Parameters:
        input_size: Number of input neurons
        hidden_size: Number of hidden neurons
        dt: discretization time step in ms.
            If None, dt equals time constant tau

    Inputs:
        input: tensor of shape (seq_len, batch, input_size)
        hidden: tensor of shape (batch, hidden_size), initial hidden activity
            if None, hidden is initialised through self.init_hidden()

    Outputs:
        output: tensor of shape (seq_len, batch, hidden_size)
        hidden: tensor of shape (batch, hidden_size), final hidden activity
    """

    def __init__(self, input_size, hidden_size, dt=50, tau=100, **kwargs): # dt is now required
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.tau = tau
        self.alpha = dt / self.tau # Alpha is always dt/tau
        self.batch_first = False # For fixed point finder
        # Check for stability (/biological plausibility)
        if self.alpha > 1.0:
            print(f"Warning: dt ({dt}) is greater than tau ({tau}). Alpha ({self.alpha:.2f}) > 1.0. This can lead to numerical instability.")

        self.input2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)

        self.reset_parameters()


    def reset_parameters(self):
      # Initialise input-to-hidden layer with Kaiming Uniform
      init.kaiming_uniform_(self.input2h.weight, a=math.sqrt(5))
      if self.input2h.bias is not None:
          fan_in, _ = init._calculate_fan_in_and_fan_out(self.input2h.weight)
          bound = 1 / math.sqrt(fan_in)
          init.uniform_(self.input2h.bias, -bound, bound)

      # Initialise hidden-to-hidden layer with Orthogonal
      init.orthogonal_(self.h2h.weight)
      if self.h2h.bias is not None:
          fan_in, _ = init._calculate_fan_in_and_fan_out(self.h2h.weight)
          bound = 1 / math.sqrt(fan_in)
          init.uniform_(self.h2h.bias, -bound, bound)




    def init_hidden(self, input_shape):
        batch_size = input_shape[1]
        return torch.zeros(batch_size, self.hidden_size)

    def recurrence(self, input, hidden):
        """Run network for one time step.

        Inputs:
            input: tensor of shape (batch, input_size)
            hidden: tensor of shape (batch, hidden_size)

        Outputs:
            h_new: tensor of shape (batch, hidden_size),
                network activity at the next time step
        """
        h_new = torch.relu(self.input2h(input) + self.h2h(hidden))
        h_new = hidden * (1 - self.alpha) + h_new * self.alpha
        return h_new

    def forward(self, input, hidden=None):
        """Propogate input through the network."""

        # If hidden activity is not provided, initialise it
        if hidden is None:
            hidden = self.init_hidden(input.shape).to(input.device)

        # Loop through time
        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden = self.recurrence(input[i], hidden)
            output.append(hidden)

        # Stack together output from all time steps
        output = torch.stack(output, dim=0)  # (seq_len, batch, hidden_size)
        # Hidden is the final step state of this layers activity, output is all step states (including final)
        # change for using bic code:

        return output, hidden
    def forward_helper_fpf(self, input, hidden=None):
        """Propogate input through the network."""

        # If hidden activity is not provided, initialise it
        if hidden is None:
            hidden = self.init_hidden(input.shape).to(input.device)
        output = []
        steps = range(input.size(0))
        for i in steps:
            hidden = self.recurrence(input[i], hidden)
            output.append(hidden)
        output = torch.stack(output, dim=0)  # (seq_len, batch, hidden_size)
        return output, hidden
    def forward_helper_fpf_ICs(self, input, hidden=None):
        """Propogate input through the network."""

        # If hidden activity is not provided, initialise it
        if hidden is None:
            hidden = self.init_hidden(input.shape).to(input.device)
        output = []
        hidden_s = []
        steps = range(input.size(0))
        for i in steps:
            hidden = self.recurrence(input[i], hidden)
            output.append(hidden)
            hidden_s.append(hidden)
        output = torch.stack(output, dim=0)  # (seq_len, batch, hidden_size)
        hidden_tensor = torch.stack(hidden_s, dim=0)  # (seq_len, batch, hidden_size)
        return output, hidden_tensor

## LRNN-NBIC : Main Network

In [ ]:
class RNNNet(nn.Module):
    """Full Network with a Leaky Recurrent Layer.

    Parameters:
        input_size: int, input size
        hidden_size: int, hidden size
        output_size: int, output size

    Inputs:
        x: tensor of shape (Seq Len, Batch, Input size)

    Outputs:
        out: tensor of shape (Seq Len, Batch, Output size)
        rnn_output: tensor of shape (Seq Len, Batch, Hidden size)
    """
    def __init__(self, input_size, hidden_size, output_size, **kwargs):
        super().__init__()
        self.num_layers = 1
        self.output_size = output_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        # Leaky RNN
        self.rnn = LeakyRNN(input_size, hidden_size, **kwargs)
        # Add an output layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.all_layers = [self.fc]
        self.reset_parameters()

    def reset_parameters(self):
      for layer in self.all_layers:
        if isinstance(layer, nn.Linear):
          init.kaiming_uniform_(layer.weight, a=math.sqrt(5))
        if layer.bias is not None:
          fan_in, _ = init._calculate_fan_in_and_fan_out(layer.weight)
          bound = 1 / math.sqrt(fan_in)
          init.uniform_(layer.bias, -bound, bound)

    def forward(self, x):
        rnn_output, _ = self.rnn(x)
        out = self.fc(rnn_output)
        return out, rnn_output

    def forward_for_fpf_ics(self, x):
      rnn_outputs, hidden_state_tensor = self.rnn.forward_helper_fpf_ICs(x)
      out = self.fc(rnn_outputs)
      return out, rnn_outputs, hidden_state_tensor

## Starting From Pretrained // TRAIN

### Training for LRNN_NBIC

### Initialising LRNN-NBIC from .pth // Training

In [ ]:
# Initialising Parameters
hidden_size_NBIC = 64
input_size_NBIC = 3
output_size_NBIC = 3
dt_NBIC = 50
tau_NBIC = 100
# Initialising base model
LRNN_NBIC = RNNNet(input_size=input_size_NBIC, hidden_size = hidden_size_NBIC, output_size = output_size_NBIC, dt=dt_NBIC, tau=tau_NBIC)
# MODEL_IO_NBIC = False
if MODEL_IO_NBIC:
  LRNN_NBIC_TRAINED = LRNN_NBIC
  # LRNN_NBIC_TRAINED.load_state_dict(torch.load('CURRENT_MODEL_PTHS/NBIC_LRNN_MODEL.pth'))
  LRNN_NBIC_TRAINED.load_state_dict(torch.load('LRNN_NBIC/LRNN_NBIC.pth'))
  LRNN_NBIC_TRAINED.eval()
else:
  # LRNN_NBIC_TRAINED, LRNN_NBIC_learning_curve_info = training_with_early_stop(LRNN_NBIC, dataset_training, validation_set_dictionary, max_steps = 10000, min_validation_perf = 0.8, patience = 3, num_steps_for_early_stop_check = 500, num_validation_trials = 200, model_name='LRNN_NBIC', tr_output_mode = True)
  # Save learning Curve now because i/o
  LRNN_NBIC_TRAINED, LRNN_NBIC_learning_curve_info = training_with_early_stop_and_regularisation(
    model = LRNN_NBIC,
    training_set =dataset_training ,
    validation_set_dict = validation_set_dictionary,
    WD_approach=False,
    WD_regulariser=None,
    wiring_beta=0.001,
    activity_regularisation=False,
    activity_beta=1e-3,
    max_steps=10000,
    min_validation_perf=0.8,
    patience=5,
    num_steps_for_early_stop_check=500,
    num_validation_trials=200,
    model_name='LRNN_NBIC',
    tr_output_mode=True)
  plot_learning_curve(
    learning_curve_dict=LRNN_NBIC_learning_curve_info,
    average_only=False,
    plot_loss_components=False,
    filename='LRNN_NBIC/Learning_Curve_NBIC_all.png',
    show_legend=True,
    legend_location='upper center')
  plot_learning_curve(
    learning_curve_dict=LRNN_NBIC_learning_curve_info,
    average_only=True,
    plot_loss_components=False,
    filename='LRNN_NBIC/Learning_Curve_NBIC_avg.png',
    show_legend=True,
    legend_location='upper center')

In [ ]:
# plot_learning_curve(LRNN_NBIC_learning_curve_info, average_only=False, filename='LRNN_NBIC/LRNN_NBIC_learning_curve_all.png')

# Implementing Testing

## Running Testing for Each Network

### Testing for LRNN - NBIC

In [ ]:
LRNN_NBIC_full_testing_data = bic_testing_w_state_tracking(LRNN_NBIC_TRAINED, dataset_testing, num_trials=2000)

Average performance 1.0


# Performing Data Partitioning for NBIC Network's Testing Data

## Data Partitioning LRNN - NBIC

In [ ]:
## NOTE : THESE ARE ALL OF THE SAME SHAPE AS THE ORIGINAL INPUT DICT
# trial_activity_matching
cond_dict_matching = {
    'test_equals_sample' : True
}
LRNN_NBIC_testing_data_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, cond_dict_matching, return_like_full=True)

# trial_activity_non_matching

cond_dict_non_matching = {
    'test_equals_sample' : False
}
LRNN_NBIC_testing_data_non_matching = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, cond_dict_non_matching, return_like_full=True)

# trial_activity_0p0_sample
cond_dict_0p0_sample = {
    'sample_stim_value' : 0.0
}
LRNN_NBIC_testing_data_0p0_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'sample_stim_value' : 0.0}, return_like_full=True)

# trial_activity_0p0_test
cond_dict_0p0_test = {
    'test_stim_value' : 0.0
}
LRNN_NBIC_testing_data_0p0_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'test_stim_value' : 0.0}, return_like_full=True)


# trial_activity_3p1_sample
cond_dict_3p1_sample = {
    'sample_stim_value' : np.pi
}
LRNN_NBIC_testing_data_3p1_sample = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'sample_stim_value' : np.pi}, return_like_full=True)

# trial_activity_3p1_test
cond_dict_3p1_test = {
    'test_stim_value' : np.pi
}
LRNN_NBIC_testing_data_3p1_test = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'test_stim_value' : np.pi}, return_like_full=True)

# trial_activity_0p0_3p1
cond_dict_0p0_3p1 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : np.pi
}
LRNN_NBIC_testing_data_0p0_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : np.pi}, return_like_full=True)

#trial_activity_0p0_0p0
cond_dict_0p0_0p0 = {
    'sample_stim_value' : 0.0,
    'test_stim_value' : 0.0
}
LRNN_NBIC_testing_data_0p0_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'sample_stim_value' : 0.0, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_0p0
cond_dict_3p1_0p0 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : 0.0
}
LRNN_NBIC_testing_data_3p1_0p0 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : 0.0}, return_like_full=True)

#trial_activity_3p1_3p1
cond_dict_3p1_3p1 = {
    'sample_stim_value' : np.pi,
    'test_stim_value' : np.pi
}
LRNN_NBIC_testing_data_3p1_3p1 = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, {'sample_stim_value' : np.pi, 'test_stim_value' : np.pi}, return_like_full=True)

# Plotting Unit Activity

## Plotting Unit Activity - LRNN NBIC

### LRNN NBIC - Single Trial Unit Activity

In [ ]:
#Single trial of each sample x test combination
LRNN_NBIC_one_trial_0p0_0p0 = LRNN_NBIC_testing_data_0p0_0p0['testing_trial_and_activity'][list(LRNN_NBIC_testing_data_0p0_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NBIC_one_trial_0p0_3p1 = LRNN_NBIC_testing_data_0p0_3p1['testing_trial_and_activity'][list(LRNN_NBIC_testing_data_0p0_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NBIC_one_trial_3p1_0p0 = LRNN_NBIC_testing_data_3p1_0p0['testing_trial_and_activity'][list(LRNN_NBIC_testing_data_3p1_0p0['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NBIC_one_trial_3p1_3p1 = LRNN_NBIC_testing_data_3p1_3p1['testing_trial_and_activity'][list(LRNN_NBIC_testing_data_3p1_3p1['testing_trial_and_activity'].keys())[0]]['network_activity']
LRNN_NBIC_one_trial_plot_env_info = LRNN_NBIC_testing_data_0p0_0p0['testing_env_info'] # Same for each



In [ ]:
# 0p0_0p0
plot_unit_activity_over_time(LRNN_NBIC_one_trial_0p0_0p0, LRNN_NBIC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NBIC/1trial_0p0_0p0_LRNN_NBIC.png') # plot all units, no legend for now
#0p0_3p1
plot_unit_activity_over_time(LRNN_NBIC_one_trial_0p0_3p1, LRNN_NBIC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NBIC/1trial_0p0_3p1_LRNN_NBIC.png') # plot all units, no legend for now
# 3p1_0p0
plot_unit_activity_over_time(LRNN_NBIC_one_trial_3p1_0p0, LRNN_NBIC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NBIC/1trial_3p1_0p0_LRNN_NBIC.png')
#3p1_3p1
plot_unit_activity_over_time(LRNN_NBIC_one_trial_3p1_3p1, LRNN_NBIC_one_trial_plot_env_info, unit_indices_to_plot=None, legend = False, filename='LRNN_NBIC/1trial_3p1_3p1_LRNN_NBIC.png')


Figure saved to LRNN_NBIC/1trial_0p0_0p0_LRNN_NBIC.png
Figure saved to LRNN_NBIC/1trial_0p0_3p1_LRNN_NBIC.png
Figure saved to LRNN_NBIC/1trial_3p1_0p0_LRNN_NBIC.png
Figure saved to LRNN_NBIC/1trial_3p1_3p1_LRNN_NBIC.png


### LRNN NBIC - Average Unit Activity Plot

In [ ]:

LRNN_NBIC_all_trial_0p0_0p0 = LRNN_NBIC_testing_data_0p0_0p0['testing_trial_and_activity']
LRNN_NBIC_all_trial_0p0_3p1 = LRNN_NBIC_testing_data_0p0_3p1['testing_trial_and_activity']
LRNN_NBIC_all_trial_3p1_0p0 = LRNN_NBIC_testing_data_3p1_0p0['testing_trial_and_activity']
LRNN_NBIC_all_trial_3p1_3p1 = LRNN_NBIC_testing_data_3p1_3p1['testing_trial_and_activity']
LRNN_NBIC_all_trial_plot_env_info = LRNN_NBIC_testing_data_0p0_0p0['testing_env_info'] # Same for each


In [ ]:

#0p0_0p0
plot_average_unit_activity_over_time(LRNN_NBIC_all_trial_0p0_0p0, LRNN_NBIC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NBIC/avg_0p0_0p0_LRNN_NBIC.png')

#0p0_3p1
plot_average_unit_activity_over_time(LRNN_NBIC_all_trial_0p0_3p1, LRNN_NBIC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NBIC/avg_0p0_3p1_LRNN_NBIC.png')

#3p1_0p0
plot_average_unit_activity_over_time(LRNN_NBIC_all_trial_3p1_0p0, LRNN_NBIC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NBIC/avg_3p1_0p0_LRNN_NBIC.png')

#3p1_3p1
plot_average_unit_activity_over_time(LRNN_NBIC_all_trial_3p1_3p1, LRNN_NBIC_all_trial_plot_env_info, unit_indices_to_plot=None, legend=False, filename='LRNN_NBIC/avg_3p1_3p1_LRNN_NBIC.png')


Figure saved to LRNN_NBIC/avg_0p0_0p0_LRNN_NBIC.png
Figure saved to LRNN_NBIC/avg_0p0_3p1_LRNN_NBIC.png
Figure saved to LRNN_NBIC/avg_3p1_0p0_LRNN_NBIC.png
Figure saved to LRNN_NBIC/avg_3p1_3p1_LRNN_NBIC.png


# (Re-) Plotting Network Structure

## Network Structure Plot LRNN NBIC

In [ ]:
plot_recurrent_weights_heatmap_BIC(LRNN_NBIC_TRAINED, filename='LRNN_NBIC/LRNN_NBIC_Connectivity.png')

Figure saved to LRNN_NBIC/LRNN_NBIC_Connectivity.png


# dPCA - All Networks (EXPORTING DATA)

## dPCA Export - LRNN-NBIC

In [ ]:
export_nested_dict_to_json(LRNN_NBIC_full_testing_data, filename= 'LRNN_NBIC/LRNN_NBIC_testing_data_for_dPCA.json')

Data successfully exported to LRNN_NBIC/LRNN_NBIC_testing_data_for_dPCA.json


In [ ]:
temp_testing_structure = import_nested_dict_from_json(filename = 'LRNN_NBIC/LRNN_NBIC_testing_data_for_dPCA.json')

Data successfully imported from LRNN_NBIC/LRNN_NBIC_testing_data_for_dPCA.json


In [ ]:
diff_from_io = compare_dicts(LRNN_NBIC_full_testing_data, temp_testing_structure)

Differences found:
{'type_changes': {"root['testing_env_info']['sigma']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}, "root['testing_trial_performance']": {'old_type': <class 'numpy.float64'>, 'new_type': <class 'float'>}}}


# dPCA - All Networks (Importing Plots and Displaying)

## dPCA Display Outs - LRNN-NBIC

In [ ]:
import os
from IPython.display import display, Image

# Define the directory containing the images
image_directory = 'LRNN_NBIC/LRNN_NBIC_dpca_plots'

# Check if the directory exists
if not os.path.exists(image_directory):
    print(f"Directory '{image_directory}' not found.")
else:
    # List all files in the directory
    image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    if not image_files:
        print(f"No files found in '{image_directory}'.")
    else:
        print(f"Displaying images from '{image_directory}':")
        # Sort the files alphabetically for consistent order
        image_files.sort()
        for image_file in image_files:
            image_path = os.path.join(image_directory, image_file)
            print(f"\n--- {image_file} ---")
            try:
                display(Image(filename=image_path))
            except Exception as e:
                print(f"Could not display image {image_file}: {e}")

Directory 'LRNN_NBIC/LRNN_NBIC_dpca_plots' not found.


# PCA (NBIC Network)

## PCA - LRNN NBIC

### Fit PCA - LRNN NBIC

In [ ]:
# sample period data for all conds:
LRNN_NBIC_sample_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='sample')


# delay period data for all conds:
LRNN_NBIC_delay_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='delay')

# test period data for all conds:
LRNN_NBIC_test_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='test')

# decision period data all conds:
LRNN_NBIC_decision_period_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False, hla_index='decision')



In [ ]:
# Fit PCA for the sample period (1d)
LRNN_NBIC_one_d_sample_pca = fit_pca_on_selected_data(LRNN_NBIC_sample_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the sample period (2d)
LRNN_NBIC_two_d_sample_pca = fit_pca_on_selected_data(LRNN_NBIC_sample_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the delay period (1d)
LRNN_NBIC_one_d_delay_pca = fit_pca_on_selected_data(LRNN_NBIC_delay_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the delay period (2d)
LRNN_NBIC_two_d_delay_pca = fit_pca_on_selected_data(LRNN_NBIC_delay_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the test period (1d)
LRNN_NBIC_one_d_test_pca = fit_pca_on_selected_data(LRNN_NBIC_test_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the test period (2d)
LRNN_NBIC_two_d_test_pca = fit_pca_on_selected_data(LRNN_NBIC_test_period_trials_dict, pca_components=2, report_var_expls = False)

# Fit PCA for the decision period(1d)
LRNN_NBIC_one_d_decision_pca = fit_pca_on_selected_data(LRNN_NBIC_decision_period_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for the decision period(2d)
LRNN_NBIC_two_d_decision_pca = fit_pca_on_selected_data(LRNN_NBIC_decision_period_trials_dict, pca_components=2, report_var_expls = False)

### Plot PCA - LRNN NBIC

#### 1D PCA - LRNN NBIC Plot

In [ ]:
plot_pca_trajectories(pca_object=LRNN_NBIC_one_d_sample_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_sample_PCA_1D_full_trials.png')

Figure saved to LRNN_NBIC/LRNN_NBIC_sample_PCA_1D_full_trials.png


In [ ]:
plot_pca_trajectories(pca_object=LRNN_NBIC_one_d_delay_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_delay_PCA_1D_full_trials.png')

Figure saved to LRNN_NBIC/LRNN_NBIC_delay_PCA_1D_full_trials.png


In [ ]:
plot_pca_trajectories(pca_object=LRNN_NBIC_one_d_test_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_test_PCA_1D_full_trials.png')

Figure saved to LRNN_NBIC/LRNN_NBIC_test_PCA_1D_full_trials.png


In [ ]:
plot_pca_trajectories(pca_object=LRNN_NBIC_one_d_decision_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_decision_PCA_1D_full_trials.png')

Figure saved to LRNN_NBIC/LRNN_NBIC_decision_PCA_1D_full_trials.png


#### 2D PCA - LRNN NBIC Plot

In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NBIC_two_d_sample_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_sample_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NBIC/LRNN_NBIC_sample_PCA_2D_full_trials.png


In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NBIC_two_d_delay_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_delay_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NBIC/LRNN_NBIC_delay_PCA_2D_full_trials.png


In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NBIC_two_d_test_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_test_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NBIC/LRNN_NBIC_test_PCA_2D_full_trials.png


In [ ]:
plot_pca_trajectories_2d(pca_object=LRNN_NBIC_two_d_decision_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_decision_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NBIC/LRNN_NBIC_decision_PCA_2D_full_trials.png


#### Add All Period PCA

In [ ]:
# all periods all conds (correct)
LRNN_NBIC_all_periods_trials_dict = extract_dict_of_trial_data_for_conds_and_hla_index(LRNN_NBIC_full_testing_data, dict_of_conds= {'network_correct': True}, return_like_full=False)

# Fit PCA for al period(1d)
LRNN_NBIC_one_d_all_pca = fit_pca_on_selected_data(LRNN_NBIC_all_periods_trials_dict, pca_components=1, report_var_expls = False)
# Fit PCA for all period(2d)
LRNN_NBIC_two_d_all_pca = fit_pca_on_selected_data(LRNN_NBIC_all_periods_trials_dict, pca_components=2, report_var_expls = False)


plot_pca_trajectories(pca_object=LRNN_NBIC_one_d_all_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'], num_trials_to_plot=100, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_all_PCA_1D_full_trials.png')

plot_pca_trajectories_2d(pca_object=LRNN_NBIC_two_d_all_pca, data_to_transform_dict=LRNN_NBIC_full_testing_data['testing_trial_and_activity'], testing_env_info=LRNN_NBIC_full_testing_data['testing_env_info'],
                             num_trials_to_plot=20, plot_title=None, save_filename='LRNN_NBIC/LRNN_NBIC_all_PCA_2D_full_trials.png',
                             add_phase_markers=True)

Figure saved to LRNN_NBIC/LRNN_NBIC_all_PCA_1D_full_trials.png
Figure saved to LRNN_NBIC/LRNN_NBIC_all_PCA_2D_full_trials.png


# Selectivity and Leisoning - All NBIC

## Variables for all

In [ ]:
leisoning_testing_data_set = dataset_testing

## LRNN-NBIC

### Sample Stimulus Selective

In [ ]:
# Data from Partition
LRNN_NBIC_sample_stim_1_data = LRNN_NBIC_testing_data_0p0_sample
LRNN_NBIC_sample_stim_2_data = LRNN_NBIC_testing_data_3p1_sample
LRNN_NBIC_sample_stim_data_dicts = [LRNN_NBIC_sample_stim_1_data['testing_trial_and_activity'], LRNN_NBIC_sample_stim_2_data['testing_trial_and_activity']]

LRNN_NBIC_1D_PCA_sssl = LRNN_NBIC_one_d_sample_pca
LRNN_NBIC_2D_PCA_sssl = LRNN_NBIC_two_d_sample_pca


In [ ]:
LRNN_NBIC_sample_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_NBIC_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_NBIC_sample_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_NBIC_1D_PCA_sssl, fig_file_name = 'LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_SS' , file_ext='.png', unleisoned_PCA_2d = LRNN_NBIC_2D_PCA_sssl, return_results = True)

Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(24), np.int64(50), np.int64(5), np.int64(58), np.int64(11), np.int64(14)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.4865
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 48.65 %
Performance Difference: 51.35000000000001 %
Figure saved to LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_SS_1d_pca.png
Figure saved to LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_SS_2d_pca.png


### Test Stimulus Selective

In [ ]:
# Data from Partition
LRNN_NBIC_test_stim_1_data = LRNN_NBIC_testing_data_0p0_test
LRNN_NBIC_test_stim_2_data = LRNN_NBIC_testing_data_3p1_test
LRNN_NBIC_test_stim_data_dicts = [LRNN_NBIC_test_stim_1_data['testing_trial_and_activity'], LRNN_NBIC_test_stim_2_data['testing_trial_and_activity']]

LRNN_NBIC_1D_PCA_tssl = LRNN_NBIC_one_d_test_pca
LRNN_NBIC_2D_PCA_tssl = LRNN_NBIC_two_d_test_pca


In [ ]:
LRNN_NBIC_test_stim_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_NBIC_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_NBIC_test_stim_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_NBIC_1D_PCA_tssl, fig_file_name = 'LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_TS' , file_ext='.png', unleisoned_PCA_2d = LRNN_NBIC_2D_PCA_tssl, return_results = True)

Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(43), np.int64(58), np.int64(0), np.int64(15), np.int64(36), np.int64(47)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.2455
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 24.55 %
Performance Difference: 75.44999999999999 %
Figure saved to LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_TS_1d_pca.png
Figure saved to LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_TS_2d_pca.png


### Match - No Match Selective

In [ ]:
# Data from Partition
LRNN_NBIC_match_modality_1_data = LRNN_NBIC_testing_data_matching
LRNN_NBIC_match_modality_2_data = LRNN_NBIC_testing_data_non_matching
LRNN_NBIC_match_modality_data_dicts = [LRNN_NBIC_match_modality_1_data['testing_trial_and_activity'], LRNN_NBIC_match_modality_2_data['testing_trial_and_activity']]

LRNN_NBIC_1D_PCA_mmsl = LRNN_NBIC_one_d_decision_pca
LRNN_NBIC_2D_PCA_mmsl = LRNN_NBIC_two_d_decision_pca

In [ ]:


LRNN_NBIC_match_modality_leisoning_results_dict = leison_network_for_top_n_and_test_BIC(trained_network = LRNN_NBIC_TRAINED, list_of_task_cond_1_cond_2_data_dicts_for_selectivity=LRNN_NBIC_match_modality_data_dicts, testing_data_set = leisoning_testing_data_set, n_perc_leison = 10, unleisoned_PCA_1d = LRNN_NBIC_1D_PCA_mmsl, fig_file_name = 'LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_MM' , file_ext='.png', unleisoned_PCA_2d = LRNN_NBIC_2D_PCA_mmsl, return_results = True)



Created a deep copy of the network.
Found h2h layer: Linear(in_features=64, out_features=64, bias=True)
Original h2h weights shape: (64, 64)
Lesioning recurrent connections for units: [np.int64(5), np.int64(12), np.int64(25), np.int64(59), np.int64(13), np.int64(43)]
Recurrent weights for specified units set to zero.
Average performance 1.0
Average performance 0.235
Unleisoned Network Performance: 100.0 %
Leisoned Network Performance: 23.5 %
Performance Difference: 76.5 %
Figure saved to LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_MM_1d_pca.png
Figure saved to LRNN_NBIC/leisoned_unleisoned_LRNN_NBIC_MM_2d_pca.png


# FPA

## Required Package Inits

In [ ]:
import sys
sys.path.append('/fixed-point-finder-master')
%cd fixed-point-finder-master
from FixedPointFinderTorch import FixedPointFinderTorch as FixedPointFinder
import torch
%cd ..

/content/fixed-point-finder-master
/content


# FPF Implementation for BIC Networks

In [ ]:
fixation_input_array = np.array([[1.0,0.0,0.0]])
delay_input_array = np.array([[1.0,0.0,0.0]])
stim_pi_input_array = np.array([[1.0,0.0,1.0]])
stim_0_input_array = np.array([[1.0,1.0,0.0]])
response_input_array = np.array([[0.0,0.0,0.0]])

## Wrapper for BIC RNN Layers (and NBIC)

In [ ]:
class FixedPointRNNWrapper_BIC(torch.nn.Module):
    def __init__(self, rnn, batch_first=False):
        super(FixedPointRNNWrapper_BIC, self).__init__()
        self.rnn = rnn
        self.batch_first = batch_first  # Ensure this matches your RNN's setting

    def forward(self, input, hidden):
        # # Squeeze the extra dimension from hidden state
        # # Hidden shape transforms from [1, batch_size, hidden_size] to [batch_size, hidden_size]
        # hidden = hidden.squeeze(0)

        # # EI-RNN expects inputs of shape [seq_len, batch_size, input_size]
        # # Since we have seq_len=1, input shape is already correct

        # Forward pass through your EI-RNN
        output, hidden = self.rnn.forward_helper_fpf(input, hidden)

        # # Unsqueeze hidden to match FixedPointFinder's expectation
        # # Hidden shape transforms from [batch_size, hidden_size] to [1, batch_size, hidden_size]
        # hidden = hidden.unsqueeze(0)

        # Return None for output as per FixedPointFinder's requirement # not a requirement
        return None, hidden

## FPF - LRNN-NBIC

In [ ]:
# Do state tracked testing trials
# LRNN_NBIC_STATETRACKED_full_testing_data = bic_testing_w_state_tracking(network=LRNN_NBIC_TRAINED, dataset_to_evaluate=dataset_testing, num_trials=200)
LRNN_NBIC_STATETRACKED_full_testing_data = LRNN_NBIC_full_testing_data

In [ ]:
state_tracked_env_info = LRNN_NBIC_STATETRACKED_full_testing_data['testing_env_info']

In [ ]:
LRNN_NBIC_FOR_FPF = FixedPointRNNWrapper_BIC(rnn=LRNN_NBIC_TRAINED.rnn)

In [ ]:
LRNN_NBIC_two_d_delay_pca = LRNN_NBIC_two_d_all_pca

### FPF LRNN-NBIC Sample Period

In [ ]:
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NBIC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NBIC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NBIC_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'sample_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NBIC/LRNN_NBIC_Sample_Stim0_delaypca.png', title_for_plot = 'Sample Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100, hidden_key='network_activity')
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NBIC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NBIC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NBIC_two_d_delay_pca, ic_period = 'sample', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'sample_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NBIC/LRNN_NBIC_Sample_Stimpi_delaypca.png', title_for_plot = 'Sample Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100, hidden_key = 'network_activity')


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 9.31e-03 +/- 6.76e-03
		dq = 3.54e-09 +/- 3.48e-09
		learning rate = 1.97e-07
		avg iter time = 3.02e-03 sec
	Identified 70 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 70).
	Computing recurrent Jacobian at 70 unique fixed points.
	Computing input Jacobian at 70 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(70, 64)
(1200, 64)
Figure saved to LRNN_NBIC/LRNN_NBIC_Sample_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reach

### FPF LRNN-NBIC Delay Period

In [ ]:

run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NBIC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NBIC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NBIC_two_d_delay_pca, ic_period = 'delay', fixed_input_array_ = delay_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NBIC/LRNN_NBIC_Delay_delaypca.png', title_for_plot = 'Delay Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100, hidden_key='network_activity')


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Optimization complete to desired tolerance.
		2230 iters
		q = 2.58e-04 +/- 2.39e-03
		dq = 0.00e+00 +/- 0.00e+00
		learning rate = 1.97e-07
		avg iter time = 2.55e-03 sec
	Identified 226 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 226).
	Computing recurrent Jacobian at 226 unique fixed points.
	Computing input Jacobian at 226 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(226, 64)
(1200, 64)
Figure saved to LRNN_NBIC/LRNN_NBIC_Delay_delaypca.png


### FPF LRNN-NBIC - Test Period

In [ ]:
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NBIC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NBIC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NBIC_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_0_input_array, trial_cond_for_plot = {'test_stim_value': 0.0}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NBIC/LRNN_NBIC_Test_Stim0_delaypca.png', title_for_plot = 'Test Stim = 0.0 Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100, hidden_key='network_activity')
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NBIC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NBIC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NBIC_two_d_delay_pca, ic_period = 'test', fixed_input_array_ = stim_pi_input_array, trial_cond_for_plot = {'test_stim_value': np.pi}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NBIC/LRNN_NBIC_Test_Stimpi_delaypca.png', title_for_plot = 'Test Stim = Pi Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100, hidden_key='network_activity')


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 7.57e-03 +/- 4.86e-03
		dq = 4.54e-09 +/- 5.81e-09
		learning rate = 1.97e-07
		avg iter time = 2.88e-03 sec
	Identified 158 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 158).
	Computing recurrent Jacobian at 158 unique fixed points.
	Computing input Jacobian at 158 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(158, 64)
(1200, 64)
Figure saved to LRNN_NBIC/LRNN_NBIC_Test_Stim0_delaypca.png

Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count re

### FPF LRNN-NBIC - Decision Period

In [ ]:
run_fpf_2d_pca_plot_BIC(rnn_for_fpf=LRNN_NBIC_FOR_FPF, testing_trial_data_for_fpa=LRNN_NBIC_STATETRACKED_full_testing_data, testing_trial_env_for_fpa = state_tracked_env_info, pca_object_for_plotting = LRNN_NBIC_two_d_delay_pca, ic_period = 'decision', fixed_input_array_ = response_input_array, trial_cond_for_plot = {'network_correct': True}, use_random_ics=True, use_noisey_ics = True, filename_for_plot = 'LRNN_NBIC/LRNN_NBIC_Decision_delaypca.png', title_for_plot = 'Decision Fixed Points Plotted in Delay PC1,PC2', plot_prev=True, max_trials_to_plot=100,hidden_key='network_activity')


Searching for fixed points from 1200 initial states.

	Freezing model parameters so model is not affected by fixed point optimization.
	Finding fixed points via joint optimization.
	Maximum iteration count reached. Terminating.
		20000 iters
		q = 5.09e-04 +/- 5.21e-04
		dq = 7.12e-10 +/- 1.28e-09
		learning rate = 1.97e-07
		avg iter time = 2.91e-03 sec
	Identified 265 unique fixed points.
		initial_states: 0 outliers detected (of 1200).
		fixed points: 0 outliers detected (of 265).
	Computing recurrent Jacobian at 265 unique fixed points.
	Computing input Jacobian at 265 unique fixed points.
	Decomposing Jacobians in a single batch.
	Sorting by Eigenvalue magnitude.
	Fixed point finding complete.

(265, 64)
(1200, 64)
Figure saved to LRNN_NBIC/LRNN_NBIC_Decision_delaypca.png
